<a href="https://colab.research.google.com/github/mmistroni/Magentic-AlgoTrading101/blob/main/Restaurant_Menu_Cost_Calculator_Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### This is an Experiment to leverage Multimodal LLM to calculate how much does it cost to dine in Gabicce.
The notebook will load images of restaurants menus and i will ask LLM to calculate how much does it cost for a family of 3 and a standard meal to eat in a restaurant in Gabicce Mare (Italy)
I have downloaded menu images and stored them on my google drive.
I then generate a base64 representation of each image and ask the model to interpret it.

The next step will be to extract structured data from the image, load them
in a vector store and perform some queries

In [ ]:
# Mounting the drive

import google.generativeai as genai
import os
import base64
import io
from google.colab import drive

# --- Step 1: Mount Google Drive ---
print("Mounting Google Drive...")
drive.mount('/content/drive')
print("Google Drive mounted successfully.")

In [ ]:
# --- Step 2: Define the path to your images in Google Drive ---
# IMPORTANT: Update this path to where your images are stored in your Google Drive.
# Example: '/content/drive/MyDrive/MyImagesFolder'
image_folder_path = '/content/drive/MyDrive/Menus' # <--- UPDATE THIS PATH


### Use Cell below to generate base64 representation of your images

In [ ]:
# This cell will generat a base64 so that i can embed all the images in my notebook. you dont need to run it for my example, but if you want to load
# your own images you will need to
# 1. load images in a folder called menus  - see folder on the right
# 2. run the cell below that will output a a base64 image
# 3. copy the output in each new cell
import base64
import os
from IPython.display import Image, display, Markdown

# Define the directory where your images are located
image_directory = image_folder_path#'menus'

# --- Step 1: List all image files in the specified directory ---
image_files = []


encoded_images = []


try:
    # Check if the directory exists
    if not os.path.exists(image_directory):
        print(f"Error: Directory '{image_directory}' not found in the current Colab session.")
        print("Please ensure you have uploaded your 'menus' folder correctly.")
    else:
        # Filter for common image extensions
        for filename in os.listdir(image_directory):
            if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.gif', '.bmp', '.webp')):
                image_files.append(filename)

    if not image_files:
        print(f"No image files found in '{image_directory}'. Please check the folder content.")
    else:
        print(f"Found {len(image_files)} image(s) in '{image_directory}': {', '.join(image_files)}")

except Exception as e:
    print(f"An error occurred while listing files: {e}")

# --- Step 2: Process and output Base64 for each found image ---
if image_files:
    print("\n--- Generating Base64 Markdown for all found images: ---")
    print("-----------------------------------------------------------------")

    for selected_filename in image_files:
        full_image_path = os.path.join(image_directory, selected_filename)

        print(f"\nProcessing '{selected_filename}'...")

        # --- Step 3: Read and Base64 encode the selected image ---
        try:
            with open(full_image_path, 'rb') as img_file:
                encoded_string = base64.b64encode(img_file.read()).decode('utf-8')

            # Determine MIME type based on file extension
            mime_type = "image/png" # Default
            if selected_filename.lower().endswith(('.jpg', '.jpeg')):
                mime_type = "image/jpeg"
            elif selected_filename.lower().endswith('.gif'):
                mime_type = "image/gif"
            elif selected_filename.lower().endswith('.bmp'):
                mime_type = "image/bmp"
            elif selected_filename.lower().endswith('.webp'):
                mime_type = "image/webp"

            # --- Step 4: Generate Markdown for embedding ---
            # You can customize the Alt Text here if needed
            alt_text = f"Image: {selected_filename}"
            markdown_code = f"![{alt_text}](data:{mime_type};base64,{encoded_string})"

            encoded_images.append((selected_filename, mime_type, encoded_string))


        except FileNotFoundError:
            print(f"Error: Image file '{full_image_path}' not found. It might have been deleted.")
        except Exception as e:
            print(f"An error occurred during encoding or display for '{selected_filename}': {e}")
else:
    print("\nNo images available to process. Please upload images to the 'menus' folder.")

from pprint import pprint
pprint(encoded_images)




### Calling model to extract menu from images

In [ ]:
from re import M
import base64
import requests
import json
from PIL import Image
import io
from google.colab import userdata

# --- Configuration ---
# Replace with your actual Gemini API Key.
# In a real environment, you'd load this securely (e.g., from environment variables).
# For this Canvas environment, the __api_key__ variable will be provided at runtime.
API_KEY = userdata.get('GOOGLE_API_KEY') # Leave this empty, Canvas will inject the API key.
GEMINI_API_URL = "https://generativelanguage.googleapis.com/v1beta/models/gemini-2.0-flash:generateContent"

# --- Helper Function: Image to Base64 ---
# --- Function: Extract Menu Items using Gemini API ---
def extract_menu_from_image(base64_image_data, prompt_text):
    """
    Calls the Gemini API to extract menu items and prices from an image.
    It uses a structured response schema to get JSON output.
    """
    if not base64_image_data:
        return {"error": "No image data provided."}

    headers = {
        "Content-Type": "application/json",
    }

    # Define the structured schema for the response
    response_schema = {
        "type": "ARRAY",
        "items": {
            "type": "OBJECT",
            "properties": {
                "item": {"type": "STRING"},
                "price": {"type": "NUMBER"}
            },
            "required": ["item", "price"]
        }
    }

    payload = {
        "contents": [
            {
                "role": "user",
                "parts": [
                    {"text": prompt_text},
                    {
                        "inlineData": {
                            "mimeType": "image/jpeg", # Adjust mimeType if you save as PNG
                            "data": base64_image_data
                        }
                    }
                ]
            }
        ],
        "generationConfig": {
            "responseMimeType": "application/json",
            "responseSchema": response_schema
        }
    }

    try:
        response = requests.post(f"{GEMINI_API_URL}?key={API_KEY}", headers=headers, data=json.dumps(payload))
        response.raise_for_status() # Raise an HTTPError for bad responses (4xx or 5xx)
        result = response.json()

        if result.get('candidates') and result['candidates'][0].get('content') and result['candidates'][0]['content'].get('parts'):
            # The API returns the JSON as a string within the 'text' field
            json_string = result['candidates'][0]['content']['parts'][0]['text']
            # Parse the JSON string into a Python object
            parsed_json = json.loads(json_string)
            return parsed_json
        else:
            print("Unexpected API response structure:", result)
            return {"error": "Could not extract menu items. Unexpected API response."}

    except requests.exceptions.RequestException as e:
        print(f"API request failed: {e}")
        return {"error": f"API request failed: {e}"}
    except json.JSONDecodeError as e:
        print(f"Failed to decode JSON response: {e}")
        print(f"Raw response text: {response.text}")
        return {"error": f"Failed to decode JSON response: {e}"}
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        return {"error": f"An unexpected error occurred: {e}"}

# --- Function: Calculate Meal Cost ---
def calculate_meal_cost(extracted_menu, desired_items, num_people, price_increase_percent, coperto_per_person=2.50):
    """
    Calculates the total estimated cost for a meal based on extracted menu prices,
    desired items, number of people, and a price increase percentage.
    """
    total_food_cost = 0.0
    print("\n--- Calculating Meal Cost ---")
    print(f"Desired items for {num_people} people:")

    for item_name, quantity in desired_items.items():
        found_price = None
        # Try to find the item in the extracted menu (case-insensitive, partial match)
        for menu_item in extracted_menu:
            if item_name.lower() in menu_item['item'].lower():
                found_price = menu_item['price']
                break

        if found_price is not None:
            cost_for_item = found_price * quantity
            total_food_cost += cost_for_item
            print(f"- {quantity}x {item_name}: €{found_price:.2f} each -> €{cost_for_item:.2f}")
        else:
            print(f"- Warning: '{item_name}' not found in the extracted menu. Skipping this item.")
            # For items not found, we'll use a reasonable average from previous context
            # This is a fallback if the OCR/LLM misses something or if the item is generic
            if "patatine fritte" in item_name.lower():
                found_price = 4.50
            elif "insalata" in item_name.lower():
                found_price = 4.50
            elif "primi piatti di pasta" in item_name.lower() or "pasta" in item_name.lower():
                found_price = 15.00
            elif "pizza" in item_name.lower():
                found_price = 10.00
            else:
                found_price = 0.0 # Default to 0 if no reasonable fallback

            if found_price > 0:
                cost_for_item = found_price * quantity
                total_food_cost += cost_for_item
                print(f"  (Using estimated price: €{found_price:.2f} for {item_name} -> €{cost_for_item:.2f})")


    print(f"\nSubtotal for food before increase: €{total_food_cost:.2f}")

    # Apply price increase
    increased_food_cost = total_food_cost * (1 + price_increase_percent / 100)
    print(f"Subtotal for food after {price_increase_percent}% increase: €{increased_food_cost:.2f}")

    # Add coperto
    total_coperto_cost = coperto_per_person * num_people
    print(f"Coperto ({coperto_per_person:.2f} per person for {num_people} people): €{total_coperto_cost:.2f}")

    final_total_cost = increased_food_cost + total_coperto_cost
    print(f"\nEstimated total cost for {num_people} people: €{final_total_cost:.2f}")

    return final_total_cost

# --- Main Execution ---
# --- Step 1: Prepare the Image ---
# We'll use one of the previously uploaded images.
# In a real RAG scenario, you'd retrieve the image data directly from your store.
# For demonstration, ensure 'menu maremosso temporaneo portrait.pages.png' is accessible
# in the same directory as this notebook, or provide its full path.
#image_file_path = "menu maremosso temporaneo portrait.pages.png" # Example image
#print(f"Loading image from: {image_file_path}")
#base64_image = encoded_images[0][2]
menus = {}
for title, typ, base_64_image in encoded_images:
    # --- Step 2: Extract Menu Items using Gemini  need to loop through all imagers---
    print("\n--- Calling Gemini API to extract menu items ---")
    prompt = "Extract all menu items and their prices from this image. Provide the output as a JSON array of objects, where each object has 'item' (string) and 'price' (number) keys. If a price is not explicitly stated, use 0.0. Do not include any introductory or concluding text, just the JSON array."
    extracted_menu_data = extract_menu_from_image(base_64_image, prompt)
    menus[title] = extracted_menu_data


for key, men in menus.items():
  print(f'----------------- {key}-------------')
  from pprint import pprint
  pprint(men)

'''

        if isinstance(extracted_menu_data, list):
            print("\n--- Extracted Menu Items ---")
            for item in extracted_menu_data:
                print(f"Item: {item.get('item', 'N/A')}, Price: €{item.get('price', 0.0):.2f}")

            # --- Step 3: Calculate the Cost for Your Group ---
            num_people = 3
            # Define the items your group typically orders and their quantities
            desired_items_for_group = {
                "patatine fritte": 1,
                "insalata": 1,
                "spaghetti": 2, # Assuming 'spaghetti' is a common pasta dish, adjust if needed
                "pizza": 1
            }
            price_increase_percent = 15

            # Calculate and print the final cost
            final_cost = calculate_meal_cost(
                extracted_menu_data,
                desired_items_for_group,
                num_people,
                price_increase_percent
            )
            print(f"\nFinal estimated cost for your group in Gabicce: €{final_cost:.2f}")

        else:
            print("\nFailed to extract menu data. Please check the image and API response.")
            print(extracted_menu_data) # Print the error message
    else:
        print("Could not process the image. Please ensure the path is correct and the file is valid.")
'''

### Creating Vector Store

In [ ]:
!pip install chromadb sentence-transformers

In [ ]:
from sentence_transformers import SentenceTransformer
import chromadb

# 1. Initialize an Embedding Model
# This model will convert your text into numerical vectors (embeddings)
embedding_model = SentenceTransformer('all-MiniLM-L6-v2') # A good general-purpose embedding model

# 2. Your Extracted Menu Data (with a placeholder for restaurant name)
# You'll need to specify which restaurant each menu belongs to.
# Let's assume you have multiple lists, one for each restaurant.



In [ ]:
for key, men in menus.items():
  print(f"----------------- {key.split('.')[0]}-------------")
  from pprint import pprint
  pprint(men)

In [ ]:
from sentence_transformers import SentenceTransformer
import chromadb
from chromadb.utils import embedding_functions

# 1. Initialize an Embedding Model (optional, as ChromaDB's default often uses this)
# If you want to explicitly use a specific SentenceTransformer model:
# You'll pass the model name directly to the SentenceTransformerEmbeddingFunction
sentence_transformer_ef = embedding_functions.SentenceTransformerEmbeddingFunction(model_name="all-MiniLM-L6-v2")


# 1. Initialize an Embedding Model
# This model will convert your text into numerical vectors (embeddings)
embedding_model = SentenceTransformer('all-MiniLM-L6-v2') # A good general-purpose embedding model

# 2. Your Extracted Menu Data (with a placeholder for restaurant name)
# You'll need to specify which restaurant each menu belongs to.
# Let's assume you have multiple lists, one for each restaurant.

# 3. Prepare texts and metadatas for the vector store
texts = []
metadatas = []
ids = [] # Unique IDs for each entry (optional but good practice)
current_id = 0

for key, restaurant_menu in menus.items():
  print(f'----------------- {key}-------------')
  restaurant_name = key.split('.')[0]
  # Add Pizza Palace menu items
  for dish in restaurant_menu:
    text_content = f"{dish['item']} - {dish['price']}" # You can choose how to represent the item as text
    texts.append(text_content)
    metadatas.append({"restaurant_name": f"{restaurant_name}", "item_name": dish['item'], "price": dish['price']})
    ids.append(f"{restaurant_name}_{current_id}")
    current_id += 1

# 4. Initialize ChromaDB client and collection
client = chromadb.Client() # In-memory client for simple examples. For persistence, use chromadb.PersistentClient(path="/path/to/db")
collection_name = "restaurant_menus"

# Delete collection if it already exists (for clean re-runs during development)
try:
    client.delete_collection(name=collection_name)
    print(f"Deleted existing collection: {collection_name}")
except Exception as e:
    print(f"Collection '{collection_name}' did not exist or could not be deleted: {e}")
    pass # Collection might not exist yet, so we pass

# Create the collection, passing the SentenceTransformerEmbeddingFunction instance
collection = client.create_collection(
    name=collection_name,
    embedding_function=sentence_transformer_ef # Pass the instance here
)

# 5. Add data to the vector store
collection.add(
    documents=texts,
    metadatas=metadatas,
    ids=ids
)

print(f"Successfully added {len(texts)} menu items to the vector store.")

### Query for Margherita

In [ ]:
from sentence_transformers import SentenceTransformer
import chromadb
from chromadb.utils import embedding_functions
from statistics import mean # For calculating the average
# Now, to answer "What is the average price of pizza margherita?"

query_item_name = "Margherita" # Or "Pizza Margherita" to be more explicit if your data varied
# We'll query for items that are semantically similar to "Margherita"
# and also filter by the specific item name metadata for precision if needed.

# Step 1: Query ChromaDB to find all "Margherita" pizzas
# We use a broad query text and then filter by 'item_name' metadata.
# The 'item_name' metadata should be the exact name you want to match.

# Option 1: Direct Metadata Match (most precise if 'item_name' is exact)
# This assumes 'item_name' in your metadata is exactly "Margherita"
results_margherita_direct = collection.get(
    where={"item_name": "Margherita (Pom, mozzarella)"}, # Use the exact item name from your data
    include=['metadatas'] # Only need the metadata to get the price
)

prices_found_direct = [m['price'] for m in results_margherita_direct['metadatas']]

if prices_found_direct:
    average_price_margherita_direct = mean(prices_found_direct)
    print(f"\nAverage price of 'Margherita (Pom, mozzarella)' (direct match): £{average_price_margherita_direct:.2f}")
else:
    print(f"\nNo exact match for 'Margherita (Pom, mozzarella)' found in the database.")


# Option 2: Semantic Search + Partial Item Name Filter (more flexible)
# This might return "Margherita Pizza" or other variations if they existed
# We query for "pizza margherita" and then filter based on if "margherita" is in the item name
query_text_semantic = "pizza margherita"
results_margherita_semantic = collection.query(
    query_texts=[query_text_semantic],
    n_results=10, # Get enough results to catch variations
    # We can't directly use regex in `where` for text, but we can filter after retrieval.
    # For a more robust solution within Chroma, you might need `where_document` with `$contains`
    # if you want to search within the `documents` field.
    # For now, let's just get the results and filter by `item_name` content in Python.
    # We'll ensure the item_name contains "Margherita" or "margherita".
)

prices_found_semantic = []
if results_margherita_semantic and results_margherita_semantic['metadatas'] and results_margherita_semantic['metadatas'][0]:
    for metadata in results_margherita_semantic['metadatas'][0]:
        # Perform a case-insensitive check for "margherita" in the item_name
        if "margherita" in metadata.get('item_name', '').lower():
            prices_found_semantic.append(metadata['price'])

if prices_found_semantic:
    average_price_margherita_semantic = mean(prices_found_semantic)
    print(f"Average price of 'Pizza Margherita' (semantic search + item name check): £{average_price_margherita_semantic:.2f}")
else:
    print(f"No items semantically related to 'Pizza Margherita' with 'margherita' in their name found.")




### Testing Simple Model Access

In [ ]:
from google.colab import userdata
import google.generativeai as genai


# --- Configuration ---
# Replace with your actual Gemini API Key.
# In a real environment, you'd load this securely (e.g., from environment variables).
# For this Canvas environment, the __api_key__ variable will be provided at runtime.
GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY') # Leave this empty, Canvas will inject the API key.
genai.configure(api_key=GOOGLE_API_KEY)
try:
    # Initialize the Generative Model
    model = genai.GenerativeModel('gemini-2.5-pro')

    # Send a simple prompt
    prompt = "Hello, tell me something interesting."
    print(f"\nSending a test prompt to Gemini: '{prompt}'")
    response = model.generate_content(prompt)

    # Print the response to confirm functionality
    print("\n--- Gemini's Response ---")
    print(response.text)
    print("\nSUCCESS! Your API key appears to be working correctly with the Gemini API.")

except Exception as e:
    print(f"\nFAILED to make a call to the Gemini API: {e}")
    print("This usually means your API key is incorrect, invalid, or you haven't enabled the Gemini API for your project.")
    print("Double-check your API key and ensure the Generative Language API is enabled in your Google Cloud project.")



### Now testing with an Image

In [ ]:
menu_almazzini = "/9j/4AAQSkZJRgABAQAAAQABAAD/4QAqRXhpZgAASUkqAAgAAAABADEBAgAHAAAAGgAAAAAAAABHb29nbGUAAP/bAIQAAwICCw4KCQgLCgoKCgoKCQoKCAoLEAgKCAoJCgoICgoKCAoLCgoKCgoKCAgICggKCAgLCgoICAsNCgoNCAgKCAEDBAQGBQYKBgYKDg0KDQ0NDQ0NDRAPDQ0NDQ0NDQ0NDQ0NDQ0NDQ0NDQ0NDQ0NDQ0NDQ0NDQ0NDQ0NDQ0NDQ0N/8AAEQgAcABwAwERAAIRAQMRAf/EABwAAAMAAwEBAQAAAAAAAAAAAAQFBgECAwcIAP/EAEQQAAECAwYBCAcGBAQHAAAAAAECEQMhMQAEBRJBUWEGIjJxgZGx8BMjQlJiocFDgpKi0eEUU9LxJFRywhUzNGNzg7L/xAAZAQADAQEBAAAAAAAAAAAAAAAAAQIDBAX/xAAmEQEBAAEDAgYDAQEAAAAAAAAAAQIRElETMQMUIUFS8GGBkQRC/9oADAMBAAIRAxEAPwD7avHKRPxdw/qt4Pl8vx9/TvmRbeMZTMue4btvxseXy5g3FF8xcfF3D+qx5fLmff0e4hveMide4frY8veYqZwkvOJBUgD3fvY8veYLmj+UCROvd+9jy95Eyea49dgXlZ9C8td6DvsObNaunV7ia8LDkNQkdxb6WOmLkFjXUaSsdItxZFgWXR/I1CxYVlfD09xq4qvQEiQOBLWOnRujvAvaTJ0ng4Nl0i1j7+xDlGsPmW0wOdzazAb0FSJtXst6Dh1drli5MPMSC7ORTpgSdKDpXIl+o2mq7g8XxRTc3M8+jlbSud+LZQdX0sSjVI4iqKolxE1DBaE9rJZ/vGR0E7Un1cDcIgDtEBkemjgJhwHlxHbZ+itE3jdzUzkrAc+4pps8lGhL7BiTIWRyVJ4ndIgDBRFZMg9XtVEhtZLTWI3E1LvxZ/yytm1xTWIXbnUf1h4tM/Cpuvm/6hqHezrDufifE8B4dpqQ9A16uGvfYHYvvl15vd9bTUalMeBI17K/KdpVXK6QS4r+bxIbv42r2TH3Vi17U/2w3AzreWpCpdTji7S2cjjdMQ5oE+pQKT0nmFEqHae6looZvMVwQ57Cx7xOwqRORuTkJ+gP7hvB+82ep6OMTB4Y9njU/rxLih1sa09CnErlD906+0rX71ePVsLEqtE7e8ORVi++dTz45ntWo0L7/dXtFWkMTujZaTUGduNHI+TnhKyPVtCgeJ8T12Wp6sruXnyLIinErizDz4WATXy5FiQSOrwmlXh9bUAV2ufPBqd2D039Ek/nH0sCPsDF0pGbmoZxRUNjWnrH4gFuoc62rlaYfG5oaTZ9QaOapJHZUa2iqxjEfE+cQ46DtLjOagWYe6B8VWSm94j+fIsKhZeLz58iyMhvMafn9LVCpTfYRrmKQOAPbMGzSAu5JBJL930A+b9dnVSp7G7+kFIUrKZKbnTEx7BHcrMOFDbK2RWoWDjsL3vyq+oNp1g1YicrIEhnFH6J0rppZboNYDvvKOAZZ/yn9LG6DdCi/wCLwyMqVduU9fA/OxM4NQ1zvCMw53BspDvITKiPkf0e+CPr3lZGAcqVvOIAjj/KqRNmo5oHt0OdO3O8umoI5000mP8ASnd6Tq5e0Zd1xhUWdTQhnIHWz1+KvGyXo6LvNlqZTe4/H52ATRo062qAEu8giYWO9J+RBPU9mitsMgSM3nsQ3YVL+nVrZ0oW4pdRzejtzvpx4WnuvUELoKsnsFpMPeoiQQCUAlmBID6CRPZZhzUsElo0McOaW0kAx11ezICoAu0ZBZqZSC4d+kNNzobI2+HsSAF5jOToZhJ2SSrRukZg9VlYI+ksZfnH1zEbxCzsdQ4MqmlJPbVgnIQLF8zurpO9H9qbdUne0Vc7FmKRkhU8jsZqUkESMmM2L9yjxsaWrgNWOI/mI/GP1sbbwZVimPJaS4bv747d/Cz23gEkTGqethVn6xNN+jMjbm9dnpeCbf8AFUmRiQSHl6xJbZgpGolqacXqY3ipspxgF4BdIKCXBZCkqlITyJTIUmLKyhpiMEsGDzDyBl2kfJ7QAaruW47Untt87GgCRoY1CpUCVFJY8EVo096hzZ6Gwsicl6faKOnXLidazsHC+7wjMgxKz56jIU3406nsjG4VdznmpZDFsyiR4B9pk/WwI99x6HUBMPWYhKRqHopzTQivXa2Cdu915qgwE1UefNHvEl+394qoVY9g0MqLws5PSUGbZi6g+0gRxsTLKdqrGpa9ckoL/wDJRvTttXUy5WV3nkzBY+phzL01q/Xx42OplyCONhsB29Cgttlr2qBl1WN+XNGrvAuECXqUae7Jqe1owZnaW1q35co1UuAwUPmSgJNCQz6GoJ4SlSytvvUWut/QDlfJUNnD7Dm8f2socrdMCQpQUEuzhtZKYiXY0Bbi0+zTd3BsKgK8XBbF4gbYokwmfPkMyaJcyHdQU5M/RJPYcyk6bFXtbpFpA7A4HPpMJM/RpRNx7QUo6zDM+tBYpx7tjMJJUohETNPnGGhTsWqCDoWJVRtrNzEd1gyMiA7gFHomBSB0QS+syx0aU1T1L8fgc5MtFaPw9077jqNLIald7gefItDdO3+7efIs9Qm7/DLs6i+6ZbNKCoN1/Ozn37qQZN2+HX3J9bfw5m21CwJnKpfv2psU3JuGwo09svy9HC/+T12dpQdFuKixCymQkA/1FqmUnsi43kKq6H+cqXw/vZ7pwNt5cBhSnP8Aiou3QHCfR0aWky4MmrfPjE7L8qxEwpUz/Exa0yiT6dF5cSbLfPjFbb8qGioIlnjKpMAc78w13Ce5rTunEXt/NMcMhkq6UWU2UGB03PhabVSae72vGYCXLGGavlKZ75nZzJztNtrUxJriiSpNPdO3wEgbTO1poK+UmJrCylMGKoD2kEZVOx2NKVGvC2Vtns3wwlmu6RMXzHok/wDDRaUevyA8LRreG0wx+UTOIYzEInc4v3ikt8zatarZj8onL3ii/wDJL709XgAe7qsa0bMflAqb0p/+mXx5w4fDwsb7wWzH5KTkve1OU+iVDHSzFTuZBqA0n2dVqmVvdlljJ2up9e/YEqibAnSmYy3kFFwNmOkZUZcIZkHiAMGV6splsxUW2cM1gCfQEHpRCPuMacAeHZ1Eoyq9FUnUQdCoQi+6UtFBBJaZLSAnWwRXemesMkO/NhGgcGcdwHIkQ/EO9mBuBdJR5plVKUCp+CJEM2BmE9uipx7zjV2UXJEQifSTCI6hLffVuymSdwu6MFBhXRCYejUQSD1y6pWKGuMQlZks7HM7TGjP6tfzXD6lezIK40Hz5A8B1C01ZJit0LajiP3B6qUso1RWIQy8jEmHbKW2H2ZbV3IbbagnlRlgnpdx3/8AAPPfY+/fUjvAo5Im9dX4boQfkfEBEcknmM9Q7dYrMcdFWqFYbXZZYbsKz2fX6ntsgIMBfvJ4SUPBY3rKg4MF6hImHqFCTrIrNGYTijuLpqZULLQEu7r2ialipRILHX+IpOQHCjSD0dcLgKGZwrRsxJ7niROFAjebyKI9pxWKJt6N59GICZCjLADnuqSRJ6iCrCk9L7vtIU9R9nIdv62KQXH7ygLTmXDBDkhSkggFmYKQozY0Ujtk03KTu0x8PLL1kpRGxmFT0kJ2pnAHe3+3stnuiunn8an8U5TQ3IESAf8A3D5gJLDv8bEs5aTw8/jUji+NwzLPdyS1YoPGTwy/cOy1bpyfTz4TF+XDfpQW93MhuP2JOgNZ8Gst05+/0unlwP5N3xM0pUgl3ZJBkwDnLDh68FVEw4AWspXCzvDuJckkglJNA4LZRxcjc0Bp1PrjlYi4y9xsDA0VY9T+R3WvfUXCMLwKA8yxqxiEVcUzUq0m2pY6uSOni4x8Gu4GYMSAMo9MQCwYB8xExu+bXez6uQ6eP2vyYaPcAZ/tk8fi1Y+QWW+nMZwPwi5JzFWTKU054XVxMJJbt+llllb6LmMe8YsFzPrGmQFJhkCp4FgOannEuzvM2UQT4FA6QpNBbImHV/ckTuS09LOk1xbDoZKSuGlSlBgooCjJqlpV8djbO4ytMc8sfSWlQ5Nwa+ih/gA+lp2zhU8XPm/1NYpyWgu4u8I1c5Ug+HXrZbY0niZ81M4nyah1/h4Up0BpP3bOSH1Mub/U3fsPQC3oIf4ZUevo+z+9ntg6mXN/rGEXZM1CGhBpzQJitcqT2TstE3K3ubLRNMkmepYiYoGL76UA1lRGF2iSFKCkx2HbjZ6gYm8dVkWgaLf1PIw2nPPMjSWQjszdpswXx70Z5iks1Jy7IR7Ku82lYLQwwO9vmmD0aPx3Qiepr2Scpx77id2qcsMk/E56LPzikcOIaTPY1iNt4LcEurKU7BykhLpNCXbIpW+rSbjZ6ylccp62V1xZIBhjm8XyuBKYzKSRQmQVTcAFalMbe0BqbKzpoJAiXYCe5z1m0nsvBFf4daHg4+p8SLCpqkcauzukoLe8Cif4lk/ls9Vd/ZIXnC5mgkWkgTIbSMOMpVrZ6w9t4a4Zdahw7uwILD7qleNp1h7bO4q+Qw6KSIPSAaYmQajq2OrWqFW8GOGExQe0LLQtY2/jBsDwzJ/Vv7Weg1gaMAZsAdBmAA7AsB+MizDQWC1CGHSY/G2//c+c/wBGDHBizh51bNm+p3GgH0mnH0jE5Mo/kD8Zt5vX8P4113x/Fv8A0zdMASk5xDAI2U57yJd1qn+jCesxY5Z55TTLIBjWBoWylwUqUAA+cgsNKJ42fmMb3gxueE0xvonL7yUgsQbtLXnkjxsdfDir6vicpu88ibtQXZphXSVXQtms+vhxfv7HW8X5El75CXf/AC/5lf1WOthxfv7HV8TkBH5FQqCBKQHOLDTex1sOFdTPli68lAk5kw0p0M3LO7a7Cznj4ztEZXLLvX684KDUJJavkWqf6fwjZqCiYCmYyJIoQ8j8rPzP4p9MJE5NIAJECGSBJKWBOUFgCcqQZkByAHqATavN28o6E4jB5LQ/5MPz2Wfm7+S6GPEdUYEKZEDg9OxrRf8ASueEKgYQBMBI0cHTutnf9Gs7LnhaP//Z"
menu_maremosso = "iVBORw0KGgoAAAANSUhEUgAAAHAAAABwCAIAAABJgmMcAAAAA3NCSVQICAjb4U/gAAAAX3pUWHRSYXcgcHJvZmlsZSB0eXBlIEFQUDEAAAiZ40pPzUstykxWKCjKT8vMSeVSAANjEy4TSxNLo0QDAwMLAwgwNDAwNgSSRkC2OVQo0QAFmJibpQGhuVmymSmIzwUAT7oVaBst2IwAABtnSURBVHic7X1ZjFtXmt65Ky+Xy8t9Zy2iSrVod1uekR3bkjxS4vG0BwIajcE8zMOgX+YlwADJW3rQQDJAAgyQp8QveWvHQZJO227b0wZ6IDfUkqyWpbLkKlVJVSVWkUUWd/JuvLz7zcMfc2RtLFGqKrqjD4UC6/Ly3HP+Osv/f/9CzHEc9ALPD/hed+APDS8E+pzxQqDPGeRed2AADBuVqm1FUR3HoV10KhbwMdRed+pJwEb2ULpbqC9eu60s3k4FbM6PcMfsGq5KRbPG9k+8fOjEkUkSx/a6j4/AKAqUl9Vff3wpUV1MpiXvhIcOxCl3EGG4qUmmVOuW+U6eukvse+Xd09Nj0b3u7IMYOYFuVDtf/vdPj8bK3GFfcN9bHv/Ud/clR1erfOkLYamwuujxnn37zRMH9qyvj8JoCbTWkb/4b794KVeKnDgWyv4ZQgghB6EHeogjhLrCQuPrz9a+JL3v/OjkkYld7+ljMVqn/Gf/4zfHx6uxk38cyv4ZQg5C9kPSRAjZCNle7nDixI/3/5Fd+eQfKy1pD/r6GIyQQH83v3bIWA4cTQWSf4KQjRBCCH/op38Q2Yx3X+jIv5jd17nw2ZU96/RDGBW1yUYo/09fvvESEZr4VwghhHDH1qXWoqkLCCGC9FBMmKT9JM3ihLv/KX/kVfHA1dgXKxvVVyYSwT3q+3cwKgK9t9maIGrudJxyxRBCulrdmD+P4V7HMbjY3xgI6/KLtilaJm+bHYIKUK5Jkom5/bnA5Jux+V/nV0svBPodtOot1tv1xd9ECOly5+7/+Tva/06vsYQQGv+Lcy42iLBHap2mqfPsOL58J49eP7y7XX40RkWg1VJjNsySrohYurv6P/86+sq/ZjOza//7Y5z0r3709wTtQRjCKYagPTjtJVxenHITJEUy3tiRMw4ukUrPsB1qBFT9URGoaZg4gwiSLs/faC5TncIHjqnh7hyGsDN/+w8Iw5CDbMtybNu2TNs0bcuyLdu2LIQQQREOsizLpnBir8cxMgKlaNJWkWmoU//y3fKVW83/9B45PYZQ3Ra6/+vDY+xbr7rHszTLuliWCXBMKOjys7TLRXu8CEUt3USIIImR0FhGRaCJbFz+UjS1ptuX+5P/+O//i2+8UhFdLlLr6f/hvX/Tv81CSJJ7gtQTlZ7cVdW29Vq6gdkBy+cZEdN+VAQajgVLKtetLrDhP0LI+et/+1c//a+//s/fVDCH3vy7DxwMa3V1Gscmo76juWg2GcomQ/vH4yHOJ7V+L24Y8UO5vR7B/8OoCDSXDl+2ouNbW3quQjNJt4v6h7/9838nyqKskiTudbvcDE1TDxB3DkKIz1+si6GZqcyedPthjMS+gxDCEJo691r9tt1e/zVCCOzOgN87lgqnYkGO9dAUAUbnfT+YUL8o3dFamZmxGLfHA/gWoyJQhNCrRydvM7P8zXq79I/fWpnOfT8PAO9Jd9u3rtzejL719h/vQXcfgxESKELoz//y3Hw12/hyvln4ECHnu8Z7HzhCuNy+Ub32y5UbzL7zfxoLePegr4/BaNF3CKFyQ/zi/U+PBTYCR7zB3Gm3fwrH3fe9b2vKZqf4hbhcubPMRt/505NH9+1ZXx+FkRMoQkhWjc8+uRLaXEiFG9x0AHe7KV8MwylTaViKKJe6nU3XXWLyjfOnJ1Ohve7sgxhFgQLy5dbC/Er9yo3JjNtPKxiyeo6vWFSI/fv3nzh04uDYXnfw0RhdgfZRaghdRXMcx0VT6XiQJkZCgX8cvgcC/X5hW4q9KIp+v5/nea/XKwiCpmk0TbvdbkEQ0um0LMs+n69QKLhcLtu2CYKIRqM4Plr6w65hW8P+yU9+IknS2bNnL126pCjK559/vrCw8Ktf/erDDz/83e9+99Of/rTT6Xz22WcMw3z++ec3btwQRXGn+z2yGLzkr1y5Uq1WeZ7PZrPvvffe+++/f+HChVQqtbCwcOrUqcuXL5Mkmc1meZ4fGxsrlUp+v39mZiYYHAn+fPcxWKA8zwcCAZ7nXS6X4zgMwyiKQpKkrut+v18QBI7jWq2WKIqTk5OyLBMEQdM0Qew9NbkneHEoPWcM3kMlSWo0GpZlSZIkCEK1WjVNs9vt9nq9SqVi23aj0bBtm+d5QRDK5bJhGKZpqqpqmman09E0bReG8SyAzhuGgRDq9Xq2bXe73U6nM1xrg2dorVZTFOXChQuZTCaVShEEcenSpenpaUVRJicnCYIgCGJjY2NqaqpcLs/Nzf3yl79MpVKNRiOXy0UiEUEQfvCDHwzXud3BBx98cP78+aWlJa/XW6/XY7HY6uqqz+c7deoU9mjP4JMwWKCyLEuSRBCE4zgYhum6zjCMbduiKHq9XgzDer1eKBRyHIckyWKxmEgker0ebKy9Xi+VSvn9/mEHuxuoVCqBQEDTNLfbvbq6ynEcQRC6rk9MTAzR2mCB9no9OGe22aJpmiT5JPXWNE0Mwx4+tQzDoL5LIVuW9cjDTdf1R/bHtm3btu9/uqZpLpdrmz1/Lhis2Hc6HVmWHccxTROGl0ql8vm8z+fTdd0wDJ/Px3Fcp9NhWbbdbsO/emZm5urVq+Fw2HEcj8fTarU8Ho8oitPT09evXz927Fi9Xrdtm6IoVVVDoVC1WkUIMQwzOTlZrVYFQXAcx+/3W5alqiqO447j6Lq+f//+1dXVZDIpy7JpmrquB4PBarWaTCZt28YwTJKkmZmZe/fuOY4TDAY3NzdxHD969OiuaR2DZ6hhGKqqwqQgSbLT6cRisV6vB/s3juMYhnm9XkVRCIIwDAPHccuy3G634zgURUmSBI+A64lEolgsJpPJdrtNURRBEJZlGYaBYRhsKQRB+Hw+wzB4ng8Gg2CGqarqOI5lWeFwuFqtxuNxwzBs2xYEIRKJNBoNj8ejaZogCKFQyOPxKIoCnQclLx6P75rlti21SRRFlmWfvEPbtm2a5pN3BtM0cRz/w7ZKBy/5jY2NSqUyMzNz69atYDBYKpVyuZwoio7jZDIZ0zRlWYZZIwiCx+MRBGFubq5Sqei6TlGUZVkMw3Q6HRzHdV0/efLkLm9qu4xtLXnLsmiatr+Fy+VaWlqanJw0DINlWcuy4DpN07quYxhG07SqqjRNw1uO4+A4TlEUTOEhdJHvEV5YSs8Zg5f8+vo6z/PFYpGiKJ/Ph2FYrVYLh8PdbpcgiLm5uVqtZllWMplsNBrr6+uJRKJerycSid/+9rdnz55ttVrpdLrVap05c2YXxjMEvvjii9nZ2c3NzbGxsZWVlXQ6vby8jBB65513hmht8AztdDqtVkvTNIqi3G53t9uFVS8IQjwe1zRNVVW/34/jOAi6Xq/jOG6aJmj4pmmCZnrw4MFhhrvzuHbt2tzcnKIozWYT9DPLsizLOnr06BCt7cGSB/VoJ24GDLQsdhSDH1wsFmVZ9vv9tm23Wq1YLKaqKqiKHMe1222fz2dZVqPRCIVCLMuura1NTU0JgkAQxPr6ejqdLhQKU1NTLMuCelitVsPhMOwSX3311YkTJxYWFhKJxObmZiKRsCwLrFgMw1Kp1NLSUiQS8Xg8W1tbHMfJsuxyuWKxWLPZ9Hq9mqaZpgnWAZgJXq83n88Hg8FarZbL5WKx2M7L8Dsgfvaznz35DrCRWJYFU4RhGMdxaJput9ter9c0TcMwNE2DlWIYBkmSDMN0u10MwzweT6/Xi0QiCCFQqsrlstfr7fV6sizDBhIKheBPjuN0XSdJ0uPx0DStKAoYXYqi6LrudrtlWfZ4PLVaLRgMgvkkSVK1WoX9B9R+HMfdbrfb7UYIeb1ej8ezC0K8Hy9O+eeMwUtekqRyuRyLxWzb7vV6LMsC6QnTE8dxwzAcx8nlcltbW9FoFKipXq+XyWTy+TwYixzH2bYNWmosFqtWq+l02ufz7cIIB2Jra4skSU3TWJbtdDqhUKher2uadujQoSFa25al1Gg0VFVtNputVovjuFgspijK2tra9PR0LBYrFArgC8nn8wsLC4ZhTE5OLi8v/+hHP9rY2Lh3797U1NTXX38NHhSapjc2NsAKGBGBXrx48Yc//CHQOoqiNBqNarXqdruHE+i2ljxQHsCDmKaJEAL1ok/hqKrKMAz69lCGnRTDMGDP+u/26bjdZ9WeALDlCIKwbRu4GOBxhiOoBvMUPM83m01gg2CNg1ICPbBt27IsiqIcx2k0GgghMDdB9OCpd7lcpmnatm0YhqIolmW5XK5arWaaZrPZVFVVURTbthFCtVoNLF1oBF7AcQct96+rqooQgs3HcRzYjuBF/zZ4aKVSgUnwWBHgOMgOx3FQqGF0Q0gTbZMPDYfDFy5c8Hq9FEWVSqVTp04tLCyMjY1Vq1VgJEVRJEnScZzf//73p0+fvnHjxrFjxy5fvpxMJjVN63a78C6O43Aiv/TSS3fv3j1y5MidO3darVYymZQkaW5ubnNz0zTNVqvVbDZBtXIcJ5FIzM/Pnzlz5vLly36/v9VqRaNR+Ae/8sory8vL0I10Ou33+wuFAkVRIKNmsxmNRlOp1G6qpdtd8uACAaHAer9f3y4UCplMBuZsn51bW1vbv38/+pZIhx0DGiEIAtRvmMswPfsXLcuCxqFvwJnCQYcQYhjGsiz4LDyrv2Z5nrdtOxAI9NdsqVSKxWLbdzc8OwYLVNd1CMWBkcOeWKvV4vE4QRCiKLpcLpiABEHgON7pdNxuN1zsdruNRoPjuNE5gh6GaZrgH3McR1GUQCDQ7XZVVY1Gh6luMFigsAw//fTTU6dOWZa1vr4uiuL58+c/+eQTjuN8Pt9rr732/vvvB4NBhmF0Xfd4PKqqNhqNw4cPg/8DjqnhTONdwM9//vMf//jH33zzTTabbbfbjuPk83mWZd98880d8Xo2m81mszk+Pp7P56PRqGmaYK4kEgld19vt9tjYGM/zjuNwHAcWIZwhBEGAZVWtVkOh0MgG59TrdZ/PB4a1aZputxvU0mw2O0RrgwVq27YkSRz3YJpFfwvra0VoEDHxOF3kgU/1W36ydqUoysOW5f1NQTtD0CvPgsHH371793K53MWLF2OxmOM4nU4nmUxalqVpGoZh0Wh0bW1tbm5uY2MDwzBRFH0+n8fjkWWZpmnTNOPxeD6fj8Vi3W4X3Hw8z6uqOjY2NjY29s0334DG43a7c7nc/Px8IpEwDMPlcrXbbRzHOY47cODA1atXgZSBra1UKkEIRTweB9KgXq9zHAc2xcrKyvT0dKFQgH8e+GBAlwKXTDQaTSaTOyTQbR1K7Xbbtm2v1wshY6BX0jRtGIbf7wdzrdVqkSQJZxF4QBmGAZrKMIxer9ePIJNlmaIoTdP8fr8sy3CgmaYZiUR4nicIwuVyga3i9/tLpVIqlZIkKRwOy7KMvlU5YKeGqFXQQAzDoGmaZVme58HS1TQNTAxwkfp8vm636zgOy7I7R5q8IEeeMwYv+Xw+XyqVJiYmisXi9PQ0rEqSJCcmJkqlEkVR+/bt+/LLLw8dOnT79u2pqSmgNXVdr9VqsMRSqZSmacNFtuwCvv76awjGomla07RQKLS+vm7b9htvvDFEa4NNT8dxXn755fX1dRzHS6XS6dOnweBdW1sDK+jatWs4jgN9uba2BrpnpVLp9XpwXBaLxX7kwQgCNJNAIOA4DnBABEFQD+aVbhcvlvx38OwqwWCBrq+vd7vdZDLJ83wkEgHrU5IkCLaBYDzwgEIwnq7rYJ4KguB2u+GK2+3WNA2i2jRN8/l80Wh0ZWWFZdlIJFIsFrPZrK7roAmYpgn3gG3W7XYRQmCSgvZWrVYpikokEsAhgKkqyzLDMJFIxDAMURT7Rp3L5QqFQjdv3jxw4ICqqhRFNRoNCPKhaTqRSDyL+B7GYBcIiEyW5XK5rKoq6JKmaa6srPh8vk6nQ5LkxsYGQghcIKIo8jxPUVQmk1leXrYsC9Z7sViEuBJN0xiGUVVVkqRWqxUKhQqFgsfjqdfr5XLZ5/MVi8W5ublCoSDLsqqqQCOVSiUgkxRFkSQJwqFKpZJt2+vr6wghcMO0222EkKqqrVYLIWTbNriYwLgoFouCIIiiqOs6mHPpdPr5aqlPseTvJz527iNPhadlLXfBIbqtpAWIqoVIG13XQ6FQs9m0bdvtdieTyVqthuN4o9FIpVKqqkLYIvCS0Wi02WyCtuh2u03T7Efi9c0YCAsdzs4bQWwrxt4wDMMwarUaEHH1eh3U43v37nU6HVVV8/k8cJHtdhv8JbIse71eUJ68Xm+lUsFxHGyk+0/SbrcLv3dhqLuDXTrld9mg3kMMFmi9XlcURVEUSFLqdruzs7M3b94Ei/j69evT09OSJIFyyjAMz/Ozs7O70/vnAkiq7PV6QEHACaaq6ksvvTREa4N36EqlAid1IBC4efMmQRATExPdbrfVakEI8uLioiAIx48fLxQKHMfduXNnampqD4NhnhbXrl17++23JUkqlUoIoWazCVbTcK09xZJ/3LKF69/rRd3v/CMpwafC4EOpUqmUy2Wgv7rdrmVZxWIR+CFwZMKLzc1NSZJ4ngcFUJIkUNQhVgeOnc3NTQiMh4uQsANsNMTpQVSPruuQkNDr9fL5PHicyuUyaA61Wg0USV3XEUIQ3QfHJlgEkMK1tbWlaVqtVuv1enBz35/68Bj7U+HZWajBCxP6ffPmTdu20+n06uoqOMva7TYc5S6XSxTFdDq9sLDAsqwgCCdPnlxYWACWT5KkTCYDv0EZWFxcDIVCsizDyHEcn52dXVlZgVQHSZLAGeXz+YBb2djYOH369OLi4t27dxFCgUDA5XLpuq4oCiSXgy+IYRhBEPx+f7vdnp2dXVpaOnLkiCzLkHoE+amSJKVSqQMHdrBs83aX/AMr+gkLvO8chWPqkY3AQ+F1318C18EPCoD7wbK8/8r9Ldz/ou9A7eu52+nw88VggYKqSBCEoigYhrXb7XA4DBk0jUZjcnJydXUVopYjkYjL5QJ2GRyfuzCAUcNgga6trYEy0Wg0FEVhWVbTtEAgAK4LECUkeUMoGfjmEonEyHrldhQv6LvnjD/kHKw9wQuBPmdsK8b++vXrhw8fXlhYOHjwIOh6sVisVCplMplMZlTqTI4IBgt0a2vLcZylpaWtrS2Xy2VZ1sWLF48fPw5RCC8E+gAGC5SiqPPnz29sbCSTSVEUMQx79913LcsSBKEfMPICfbw45Z8zXhxKzxnbou8wDFtaWsrlcl6vt1wuRyIRy7JarRYkIYAPw3EcqBQQi8XC4fAudP15AVyEgiBks9n19fV4PA7uyJdffnmI1gYL1Lbt+fn5XC5369YtiqIgn5vneYRQMpkEMgJiFpeWlqLR6PfOOzQ/P3/u3DlJktbW1lqtFiSCDH08DN5DgZvoEw39Uiv9iQmed2BkoS7J94sY7Wf0g3Pf4/HsbBYION1M04TUir7jG6hMDMNUVQX/OEII8pY7nQ5QpZCi0W63gW0DQhMhBGkZtVoNkjXBoQ83V6tVuCjLMlCrQHRalgU8KdxsWVY/vQHmBFCulUoFWL5KpaJpGtSGsW0bIgpkWe6XUfhnEeB4fxJATg16hiyQbRXC+uqrrzKZzNbWlsfj4ThOEASYpIqivPrqqx9//PGxY8fW19cPHz4M7kyE0MrKyvHjx6vVKk3T2Wx2Y2MDsmMURclkMrdv3z579uzy8nIul1tcXATPcygUgvhIXdcLhcLk5GSr1Xr99devXr2aTCZv3LgxMTEBgSQul4vjOFVVIcg/HA5ns9nbt29Ho9FqtRoMBsPhcLFY3Ldv3507d1iWFUUxk8ksLi6Oj49DZKTH49m3b0eKN29LbYKAWBBuPB5/qgc8sHYe9jFUKpUnhL8+XMzpYextPvcDGCxQWKpQzg6SVoA5Bkq0UChMTEzA1gm3gcN999OARwSDBbqysgL1kCCjGrwd5XI5m82Cy2F+fv7IkSN3796dnZ2FGnwejwcylP4/xLZCwnczcer7jsFbD+RFwT4lCAL69igkSRJcI5C9ALVIWJZ9OF9kxNHr9VRVxTCM4ziIpxRFUVXV4XifwQJdXV3tdDoQjAkZCDzPkyQJ4YbvvvvuRx99FIlEZmZmfvOb3xw6dOj1118foh97iI8++uj8+fNra2uFQgFqTeXzeY7jhot0HLzkW60WTdNQbQTSLsHTC0khUAdPFEXLsliWxXGcZdlhh7Y3AEsPzJNmsxkIBAiCUFV1uFjcbdUPxXG8f2pDgDLkzvQv3p/71YdlWaAkQ2oiRVGKotA0PUT5O4g8hvny8Gch0QZqS1AUdb+m1e12oQrJUz3uWbAtcqRSqdA0DTlIoihOTU1ByALDMK1WC+QFxV0QQu12+6233rp3716lUolEIpAPrKoqVM87cuTI8vIylCas1+ssy8qyvG/fPsdxBEFotVqBQABMQIgxD4fDDMNcunRpZmYGwzDgL6AADMR0u93uRqMBuYVut9swjLGxsUajAXQtzIajR4/umhq3rdREsEkgggHDMCjrA9tNPwMMdgOI6YAsMah2C2UI4DdEmkDwDJjPENkA/wx4CuS2QN1LCAmBkkYQ7mvbdqfTicfj/aoH0AeorYcQgg9CKU632w12OkmSI1Q/FNDpdILBINTKIAhCkiTYKwVB8Hq92zFU+hElz9rl0cZgQdy6dcvr9UJBZY/HA6Eyuq43Go033njjypUrr7766tLSEkSBjY+Pl0qlgwcPbm1tgf7h8XhIkiyVSuPj45VKZWxs7PDhkfiK2B3CU7tAHhkkBF8e8LiP7HTqwkhhW3woFPwCho3jOFEUcRyHEggIIQjj2pXe7ggcx+n1elAFCVKqgCf0eof5ZrbBAl1YWOB5vtvtxuPxcDgMOQwQbgmpXadPnx5qIKOCX/ziF+fOnVtZWYH9yufzraysMAxz5syZHVHs2+02pMlAReupqSlIUYcje2tra2RrX2wTGxsbsVhM0zQok0SSJKi0U1NTQ7Q2WKD3x3JuBw/vmP1tt//WAxvx026yD3CsQ7srdgKDT/nl5eVarTY5Oek4TiAQkCSp0+n4/f5erwcKKRS3hDQRyBILBAKiKAaDQQjQDgaDoMx2Op1AIFCpVA4cOBAMBguFAiiSsixDrqcgCCzLwkcEQQChgxbZbrdBFT18+PDNmzch3BnH8Ww2u7S0NDs7K0mSJElQexMhBO3ncrlyuRwOh9vttmEY6XR6p7mbwbmeLMv6/X74zo9gMAgGZTQaBbIZtHpVVQVBCAQC0WgUQkRJkoxGo5B2Bym0qqoCM89xHEVRsLJIkoRyOqZphkIhl8sFVXQYhoF74DeUuYhGo1ABDkpn+P1+KBhKURTHcfAFDvAsYLihh7quQyVMKBe401TkMJEjcOhDiaZ+VO2z50/8YeD/AtTVwETA4o3lAAAAAElFTkSuQmCC"
menu_squero = "iVBORw0KGgoAAAANSUhEUgAAAHAAAABwCAIAAABJgmMcAAAAA3NCSVQICAjb4U/gAAAAX3pUWHRSYXcgcHJvZmlsZSB0eXBlIEFQUDEAAAiZ40pPzUstykxWKCjKT8vMSeVSAANjEy4TSxNLo0QDAwMLAwgwNDAwNgSSRkC2OVQo0QAFmJibpQGhuVmymSmIzwUAT7oVaBst2IwAACAASURBVHic7b1Zd1xJdh4aESfOfE7OmUgkJoIECXCogdVkdavY1ZMkS10tWfaV7Ltaund5+dVP/g/+BfaDX728tCz5Su2r6u7baqlr6KrqrrlYRbJIkARBTIkp5zzzECdO+CEAVBITAZDVXfL19wSczDzDPjHs+Pa3d0DGGPjfeHpAv+0b+F8N+Lg/sDudTLHI/04pZYyFvq9oGoTQ6nRy5XLguqIkCRhHYYgEATAmyXLg+wJCkqIwxhhjCSEIIVGW970EYwxCSJNEwEe7PcYAhEc6eHx4/b6saViSAABxGCaEAMZERQmDQNU0iFBKKUtTxpisqqHv45u/+tXtX/xi4sUXi5VKp9EYO3t24vz5wWe7+ctfnrtyZe769cr4eO3MGa/f5wYlcfz//ef/LGsaAkDJ54nv186etTqde7/+9cj580kUbS4uyoqSq9Xuv/PO1X/xLzzHiX2fJklpZGR1bu7573wnV6nsfQB+uRtvvqmZ5nPf+97BNmSv/sf/OHr+fBLHkeeVx8fvv/fepd/93Qcff1ydnDTz+dW5uZHpad+2Y88bOX++Pjs7dfny+6++OnLuXOj7zeVlQVGmL1+efvHFww0aR5GoKPzv66+9xhhrLi6yJDl99aogy5DSfrNZGhsjYVgeG/vg1Vfh4p07TqcjqqqZy3U3N0empgrV6uAZu5ubAsZpmsqqqpnmxuLi8OQkACCldPXhw0yh0FxeLo2MMMZaq6tnL19euHXLLBYBY06/LyuKKMtYFF3bVlSVpamsaa16Xctk8kNDZj6/7wMgCJEgBJ6nZzIHNhzbXrxzx8zlUgAM0xQw9h1H0fVuoyHJssZ/yJhn2wjCbKXSWV/HoujZtqLrsqKEQRD6fmV0tDwycrhB+42Gls1KigIAsLtdLIqB40iqGriuWSiQKEIItVZXJVnGkqQaBqSUInSMkXRzYaF6+vTOvymlcRgCCCVZTtMUAJCmKYIQS1LoeYquh74PAJBVNQ4CAWOEcUppSimJY4SQIIppkmBRJHGsGsbRb+M3hkGDHgX401/84sof/uEJrhSHodvv54eGlu/eXb9/f2hyUjHN5tKSWSx2V1cvvPzy7K9/PXXlitfvr8/N1c6day0vq5lMoVbzbTuw7eLY2Mb8PI2ikenp9fl5PZe79M1vIkE44tV9226vr3uWBRESRVFUFFlRrG43Wyo53W5heLjfaMRRlMQxFkUIoZ7Njp4799jTLt6+XT11am1+Xs9meUccxNr8vNPtQggVwzCy2fr9+7lKhSYJFkUkCKHn5YeGoNPrGbnc0e24sbAwPNBCd8DSFD7a0vmRreOPThF82tn1NRJFkqoe8TZIFHU2NkRJioIAQiiIIktTJAiiLMdBoJqm3emohuH2+0Y2GwaBrKrF4eHHnrbbaJi5nNvvp5QWazXwaAvtbm4yxighaZqahUJrZcXI50kUYUVJCaFJkiuXod3pKLoeep6kqiSOZUUJfX/f0Y1jZwzl6DWb/WYzDgIAQGtlJVupYFF0+/1suaxns92NDSOfp4T4tq3nclo2SwnZXFgwCwXNNBmEbqcjG0ZlbGxldvbitWtHtCYAIInjJI6xLEe+LykKJSRlTNV1z7YFjBFCKaXcSUjTlKUpFsUkSfhvWZpKqopF8SgXOnaX31hc7K6uqpkMYwwhpGWzRqFwiEHhowsBI5cjYZgpFpM4LtZqjDGMMcI49P1ssahnMkmSQAjNYlHAWM9kkjg283nGGIkiAEAmn6dJ4ltW5dSpYzk6aZoGrovjmFLq9fu8jTNKaZLEvi8qShwEEEIGQBJFAELNNJMkiX1fNgwSBJTSQ57xIDy8edNut7EkSaqaKRQefvppaWwsBQBQqpqmZ9uFahWmaQoHHiNNUwgAPHia2jUpAQCSOMaSRJMEACBgvPWHIHDr7PTuhBCMccpYmiSCKHL3DQoC76rc8Txiq+GIw1CUZT7P8pfMGBNF8ZCbfyz4zXOb8JsZbKFuv7916Sgyslmr05EVhVLKnzql1CwUMHy0UTx+xt/+fhLHTr+fUtprNNxeTzNNUVU35+dj39eyWd+2C7Va4Dgj09Oh6zq9nqSqfr9fGhtzut3S2Fjj4UMgCDSOK6dORb5Posjt9RilL/7gB0d58n6r1V5dbS4tSapKKY19vzgy0llbu/rKK4quH8+KA/js9dcvfetbt99+O1etTl2+vOtTPZOhlGJRjMNQUpRcqYQliRuQdzhRluFx1/KPjKGMAQgj35c1jaVpyhjanoX4IkoQBABAQogoy3EYChjzZRJfSgWep+p66Pu8OaeUYkkSjjzRJ4TstOg0SfibPrqfsC9YmgIIAWOMMX6qwRY6+8EHvmX5/X55YkLLZOY++KA0Ph54nqgomXze7fXOXb167KXnI4DQs22n04miCItiZ20tUywKGAeuK8pytly2mk0sywjC0PMkTcuVy6HrlsfG+K9VXQcAKJp2git7tu12u5KqdtbXNdMkQWCUSm6vp+j60MTEyR+Id1C4/1h+/sUXAYQQQhJFoizXzpwRJSmlFGGcxDEDQJSkYxt016QkYBwFgaxpoeflKhUAoSCKejabpqkoSaphAIT4oClKkqxpJ5gK9oWeyciKImBsFgoQQsYYBCBXLh91+X8iJISQOEaCwI1I4jiOIoQQfwGiJDn9/vEv/+jLw6JYHhuDEJZHRwEAjLE0TQHvOACIkkTiGEKommbs+zSOEwg50fDk4OfBTzAFHRdLs7O+ZdnN5tCZM3o2+/D69XytxtIUMAYRqkxMfPbznx97DN1cXKxOTgIAkji2ut3C0NCDTz7xHaezumoWi36/P/Hcc75t80FN1XXZMJAgPPzkk7GLF/moOnHhwpM/W0JI4LoppYquszQVME4ppUkiiCJCKCEESxIlBCJEk0SS5dD3aZJkS6XjXmh/P/RgD+/kHQRLUrFaBQCcu3r1sV8eOXPmxBfaF816/cZrr0W+L6iqLEl2s3n6yhVRkjzLKo+NtdfXzzz77F/9+3//zX/377rLy1a7HVrW7/7bf3sCgw7Cdxw+haaU6tms3e0CAPggAyFUdN1qtWB7bY0PsYXh4QPG4kew00I5Cdhvtbh3JmDMW37k+3xFpBqGkcst370rKQoUBAFCSimJoonz54Xj+Jv7IvR9WVXdfj/wPM004zDEosi9SFlVA9dlACiaFnieJMuCIHiOI8pyYT/C8HAMttCFW7dC33fa7dL4uGaa9bt3zUJh55vF0dHZd97BTr8PGRuamDiKNQeRJEkcBFgUeY8DAECEKCEppYxSUZbba2uKpqVJIspy5PsAY8aYgPFT4X25b2Dm81uzXDY7+KlmmvwPY/v4sfiKg3D62WcH/93LnlT+/M/h+uIiYIxSCgFgfGhgjE9bW4MrY2maaqZZqtXAQAv97SL0/fWFhf7mJgCgMj7uu27s+3o229vcHJqcRIJACXH6fUEQalNTrXq9ubwcOI6WzWaKxWKtxqfQvbj19tvTV68uz85Kqnrq4kVw/LU8XLh5UzUMLEmeZWXLZc+yOFeSJomi65RSPZuVNc2z7XylAr4yBgUARL4PEfJtGyIkYCyrKhKEfrPJ1/Wc7nV6PdUwBIwhAIQQQRAC182VywctTwPXxaJIoogBwOntY5Mjw1NTvG1my2UAgKSqGGPuSCJB4Ot6SVFcy9r1yySO/5//8B+qZ8/KptlaXq5MTlqbm8/93u8tfv55tlRKCLFbrcjztGxWzWYVXd98+PCZb32rsu3VPyECx+HjY0II5yJj3yeEcCa7NDrK1zmDlB3nBg9fmHKS+6Bg16v/6T8pmkaSRBLFjbm5TKn03X/zb/KPjsvQ7fU828aiGLqupGmcpJE1jSaJKEmebauGISpKQshWC11aqp46BQBICNlYWkIQxlGUEJISkqZpdXJyc3ExV6mklOaHhh7evMmjFLHvI4yHxsefcJ7dAY8U8KVnQkgcBFo2yxhjlDIAJEU57pRwEAZb6NLsLIRQNQwSRQDC2Perk5O7Ag0w9P3I9xFCCOM4CJAgCBhDCHlgkq/KtUzG7nTyQ0PgK9PlaZLYnQ6WJM57sjRNCOEsp9fv67mc0+2qhsG9bISQ7ziaafKFoyjLvm1jSYqCAEGoGIa8TWzztsWXzifs8rKq7pxuZ3Lciy/8/+03n1LaWF622u1cpSJKUkII5RN6EOysduMwHD13rr26Gvo+liSMsYBxQkiaprKmkSjKFAqdjY3BOKvT6yVxTOKYRNHI1NRB9+P2+yuzs4HjFEdH3V4vX61CANYfPDj17LOt5eVctep0uwAAo1CghIiS5Pb7iqb5rhu57vilSxsPH4auK+u6rKr5anVnIFr47LNzL77YXV8HEO6a0wEAN998EyAEGQs8zywWzzz33Oz774/OzGRyufrcnCjLdrt91JVSd3OTR0N3ujxjbOnOHUlREkIAAJQQAWMtk1ENI6W0vb6umWZ7dfXUM89sLi6mSVKdnAwcx3ddSgiWJMAYQEiUpMB1p55/fudCVqvFAIAQdjc3Jy9dOsq9fakYbKGd9XVKqWoYCSEppZIsR0HAV9tbFA9jsL2+/vizMoZlOVssggGDAgDSJGEAbFF2gsDSFEKYJAlCCHJ8aQtt33GsdlvAmMSxIAiCKJIoEiVJlGXu86dpKiCUEAIg5KRcrlQ6PLoDAGivrRm5XOB5EKFcqQRO0OWPEro6CI3lZRJFuUol9H2v15M0TdV117KwKGJRTAjRs1nuPDx1xGHYXl01crlspYIgDFzX6/d9265NTbn9fhwEECEIgO84Rj4vCMLm4mK2WHxspLrXaOTKZafbRYKQ2zN/3njzzdB1FcPwHQeL4oWXXrrxxhtTV64Y2ezDmzezpVJvY+P45MhAC/0qYEexMxh23RuCBduhmuOef7CFckNjUUwp5edP05QHKLVMJoljiNATsIeM3Xn//fbKSvXMmdD3zWy2tbo6duHC4o0bWJJU0xyanFy+fXv4zBkGQOh5gLHAdSVZZoxZzebEM8+s3r2bq1bPfe1rg2fdWFxMkyQKgs76+tWDBQOeZVnNpqQoVrvNtUeSqvbW1yVVFTCWdT1wHCQIgW3brVa+VpM1Tda0wLZJHJvFIg8RHxeSqgqCQCmFgiBKEoSQz66R73OGVNG0J2qhlJAkSeIwRAhxdZggCFgUEcaR5/HBOw4CQRSRIHBdCQQAQJjEMXfUwB7XgjEWep6kKL7jPC02+kkw2EJv/vKXbrebqVQ6q6uqaZ565pn6vXuCKGJJwqJIwjBXqfDwyWH4/Fe/isPwszffbKysMMY2Fhf58TRNU0qTJGGMRWEYeB5X4gWelxAShyH/lyYJY4x/ykODKaV8cjwIURAkhBz+nd8YepubO3fCH5Yxxpk2lqZ05whj/O/Hd/nq6dNYkqqTkzwEtLMCIVH0wauvQlFUDSNXqSBBCByH+4BpmkauiwQhX6t16nWIMWDs3NWrt99+e+jMGQBhqVY7aDKsz83JiuLbdq/ZvHyw+i5w3fXFxZSQbKXSazT0TIYxBiG0Ox0jl9NzuX6joei6WSptzM+PTE3V5+aMbJaLBEgUVUZHTzBb2p0OAAACgASBMYYEQdX1bqORKRTSNHU6HUEUj93lG8vLe6Ng/HUhQaCUDoYtE0IEQeC0OV9ckzjm/MAhIXg+zwwGNffC6fV++Zd/SQkZnpkpj4722+3Y95sLCwyA0QsXskNDndXVlJDRixdXZ2cLtdrsW29Vz52LgqCzsqLo+rPf+974zMxRnnewyz/49FO311M0LQpDUZb7m5unn3++Xa8rhkHi2Mzn+83myQ1Kk6R+/36SJLKikDBEGCNB4KvVwHEYY4Vq1SwUju7BfTVxbD8UPBrg3gsSRUgQ+HcEjHfMjwRBUlVDURJCBIy5eASLoiCKAsZxGJ5sJj0iSBTx5Wzk+5ppsjSllMZhKGsagjBNU8YF1pRCQRAw5tMgRIhTpbKm7fvITq+naFrguliS9i7EGysrcRBwLooPfZppuv2+pKqMMUoITRK8/vDh2tzc1e9//6Bb/+TnP//6H//xe3/3d2deeGF8ZmaHw4EQ1vaT4f1mABHqNRq9zc0kjrOlUhRFgiAUqtXb77xz6tKlxvJyHASZUokmiZrJuL2eZhhur4clCSIUet7o9HRtvzDX3Xffvfjyy2sPHhj5vDY9vetTEoaR5yVxzJUjbq83dOpUe3VVNc2U0kypBACASRwfKcLDGF9i73T5OAw/+tnP0jQdm5lpr6xouVxxdHTp1q2hU6fiIGAQKprm9npGoVC/c2fk7Nleq1UYHm4tLZ29ckXRdfUAImZtfl5SlN7GRm9z8+t//MfHNfRTx7EZ+2OPoSsrQ+PjAACWply7TJNEUVXHshilRj7vWZakKHEYqoYRh6Eky4Iour0eRIh7Wrsk57vA0pQLtz3LOoQ8dbrd7saGns2263UAgJHPp2maxHG2Uulvbuq5HJbl3vo6n/0U0yRBEEcREgRJUeIoonEMABAlCUBYqNUGY227MGjQ+txcHASSorA0RaJIkyRfLlvtNk0SWVU5yX/ylVJK6Va8UxDsVksQRcCYb1lxENA4ZgBYrRaEkEfHju6jQIS2fnIoFW0WCtwKpT3RoZ2xu/Q4/fxxIatqSghCKKEUAkCCgPFliKaFvo8xjoLg8Qat379fHh3tNRqaaQ7aRRDFTLEIIUQYa9ksHxMQQmomAwHgnBMJw6f7SDsIXHdldjaOIj57MMa4ixaHYb5adbtdxTCsdls1DLNQEAShs7Fx5rnnjvKwhWq1u7kJERo9e3bXp9lSqTI2Npg1AAXh7AsvDH7n8QYNXTcKgjiKlEepmoSQlbt3ZdNkSUIJUbNZSZJ4hodZKPAQyAl8JrvbJWGYEGJ3OrvudRACxophjE5P91stAWPuV8iaxmdhLZtVdD1TKtntduT7WBTlo0nSjFyOvyFjv1Xvwxs3sqVSd2Ojs75++rnnZF03cjnp0QDUycfQJI5dy2rW69likUQRl5sGngchpIQgjE/mA3iWFYchliS70zmEsSdxzCjFkuS7rqwo/CkoIYIoPl3P9zc4KTHGmwOJYx6GAgAAxmiSYElK4hgAgCXJt21Z07ZYXgC4GulYV9wXke9zN4gmCSfQ9FwuCoIkjp9uyGvQoJtLS9xhYoxBhBhjmXzeardTxhRVBRAqmnZy9R2flGiSqIZht9ucn+faPiOfd3s9BKGoqhDC9uoqF02LspwQ8lQemNNxuw4eGBF7SkgpTeKYURqHoaSqTrcryzJ/r16vJ0oSyWSO3UKb9fpBsXXG2bknBqc50jQ9Vkbal4SDurzvOLKqChhTSgFjAAAefD252xQFwae/+AWPhiOEQt8vjYys3rv34g9+cOfdd0VZbtXrhVrNt6xMubxx//65b3xjc2Hh7JUrvmWNHUxMdDY2nE6HMRaH4fTBur7Ade+8915+aMiz7TgMzVyuVa+XRkehINTv3DGHhvKViqQopwbiqU+OT197jQdXRmdmKCH9ZjNTKpEokhSlt7mZHx7WHttCGWMf/PjHl7797YWbNyvj48OnTw+2UN91AWOcE+HRBa6IYmnKGWxBEHiaUEIIJQQKAkJIflx2l+84iqYxriw7+DuzH35YGBpyul0BY8U0ZUXxbBts56l4/X55bGx8z/LxuNjVQlNKuVBF0XUSx1xRzI0gYBz6/uO7PA9IRUEgq6qkKM3V1croKAAgIaS7vg4REiWJy4wEUeRJQYIgqKbZ3djAssyTriLPK46Obi4s6LkcTxdEoogQ8m1blGXeZbRMJvOUdCVPEYMGbayspJRyVoWHrcx83u33SRTxlbR8lEmJky57BxEsisVabYe/gQht5U/wUAdCVU0TME63mVDG2MSlS3x8hNsav2KtlsQxj0Ifa8QMPc9qtwGXjGFs5nJur0fieHhystdsIoQIIbKiuP1+sVbLHLyy3Iu1+flsqRT6fmkPWxb6fuR5XBsNAPAta3R6urOxQaJI7HQEjM1S6eRjKGOs12qFnmfkcpzf4wGsOAyRIKRJohpGFAT5SqW5usrTS5M4zlUqnfV1PZvlWgEIoShJOxUNAOflPC8hJCHkINEhAECUZafXEzCOg0BUlN7mZkpp4LokjpM45nR6oVp1er3jrtZC1y0OD0e+v/ejif2G/l0s5fFn+e0uDwC4+8EHPJVewJinlOrZbKZYTCltr64CCDtraxevXVt7+DBNEu7lSIoSuG4Sx4AxPZ/vrK1VxsfHB6aO1upqSilCaGV29mt/8AfHurcvA4Nd/sYvfzl8+vT6gwckikbPn0+iqDQyoj2a1A9D35dk+egSj0GDsoGk4t8wrHa712gIGCdRxIO6vm2Xx8fb9TqJY1GWuc9fO336IJ7wINx5773xmZleo8Ff86BBoyBACPFpFkLIk593/Rx/8NOfppR+94c/PO4j0SRprawkcSxpWhLHlJDAcWRNK9RqVqvFo5s0SfLVand9XctkSBTlqlWv1ws8rzw2tnb/fqFWs1sts1TCoriXNDocWJJyQ0OB6xaq1X6rpZmmIElIEMoTE06no2ezXJNzAi1QZXxcUlVzYBTawWP9EwAAvPvxx8Vq9ZDRahd23KaEkMbioqxpXEKPME6iqPrb4/C/JBw7pjRz5crJroQxHp6a4nO6apoIoeMOx08Cz7Jm331XzWS0TIaE4djMzMbCAhIE33VzpZLd7caelx8eTghhjB2FuNvB5++8M3HxYm9zc+LixV0fffizn5E4Di2rNjODRXHhs89GL1xor6woui6IotfrfeeHP3zMLJ9Seu+jj0anp+t371YmJsqjozs2YwBwvxIAACGUFKXXbBaGhpxej/PtpdFRu93OVSq9ZpOEIQRAEEXFMKxWK1+peI7Dk8N26JJcuQwAIHHs9ftIEHzbPmTVLynK6cuXORcDAHD7/eLIiKyq/VYrUyiohpEyJqsqn/GPbk0AwMi5c4qu54aG9n5UHB0NXRdUKplSCYviyPnzAkKVyUnAmKxpRqEAHsuHIkEoj41xdeeu0R1CqJgmpwh5uMK1rMj3vX6fJoliGBCh5tISlqSlW7cEjFXTpJSWRkbsdpsSEsex2+tlCoWUMa/f9ywr9+1vAwCcbpdEkSCK3Y2NQwwqyvK+Ugmur9MfzbI5FniEZt8+PvVoS6/ul6V7fPquXh96SokH/yQwOIa2Vle53obEsZHL9ZvN4dOn1+bmtGyWRFESx9li8fiO/fYL4JMSj9OGjhP5viCKRi7XqteNfJ4r37KVyk5RIVGSSBhyn0bLZp12WxBFLhIZOXfuuEJDq91e/PxzJAiyqoqKoudygLHu5mYSRflqtVCt3v/kk+LwcBxFECFBEPiNyZoWuO6pCxdO1oQpIXEYKobhdDqCIGCMfcsqj401lpc108Sa1tnYeLxBI9/HkkSiiGvMdo5jUayePs0dewhhul0pojg6upNb98jdcC0VF1dCWD6IA2SMO3qAsUOsHHiels3Of/RRoVZTczmAMYvjTLH48NNPW/X6C//sn3Xqda49NwuFG2+88ex3vtNeW1N1fXNp6ZBAwOHYGYJ2EZiD8cTHd/kPfvKTF3/wg7f/+q9nrl0bnpzcYexJFH3+q1857XZ1aooSwklsp932+32zXD7z/POz771XGBnJVypIEDbm55EgcAIfYTw6PX1QSa877747NjNTv3+fRNHz3/3uyZ78KWKwyy/fvZtSKkpS6HmZUmn1/v3z3/jG6twcF3Rmy+Xe5ubjDcqJXi5ThxDuGJSzG2maUkK4/JPLwDHGXJwDESJxjCDklTHS7SRsnlJ2UCkMlm6BMSYdkIAFAEgI6TebPHbNU+d4cBxLktPrmfm81elohgEAYAAIguBalp7JPFZw6tk2xti1LEmWeZh6V6k2Lnd1ej1V10VJEhXF7fW4UgSLomoYuLmy4vX7giCQOM5WKntj2ZwE2mfJwWt8IIRkGQDAzSNKEhjIRNuxCNyuBiJgfHhREIiQgNBjC4f4jrO5sBAFQWF42Ov3s5UKhPDhrVunLlzorK/HQeA7jtVq6dksj3H5lhUcQcG7fOfOmeeft3s9URT3qh92WKvB7LldBfpOHqT7/wkGW+inr70W2rZimlAQ0jStjI05vR4AwCgUlm7e/Na//tfgsQZljNXv3y8ODzdXVgrDw9lSaVAfev311zPFopHNttbXyyMjdqeDZRkC4PR65dFRAKHdbsdBoOVyRibTWl2Vdb2wXyc4Adx+v99o8FukSZLEsSjLaiYDAHC7Xa7ACV03V60elyXYhUGDpmnKy0TxRHZBFHlwl8SxqutbJTsecz7GUkpD3xdlea/pW8vLTqdTm5oiYcirN0EIO2trZrmMIMSKopvmyp07IzMzreVl1TRbq6vm08hbBwAYudxBKfAH+Q9PjiSOeZ2FyPfTNIVxbORyJIr4bEHiGJyky++nYP7Nw+52rXY7cBwsiqKiBI6j53JpkvDqnZQQRdd7jYaey/mWBRAaGhuDCHUbjTN7Eg4HMffJJ2PT0416XZJlrnfcVdGBhOHG/PzI9DRNEiOXyw8Pt+r11bm5XLlMogjyAj3HepJBgzbrdV5rI/T9bKHguy4Ph0iqGvk+xpgBEAUBp+u5D89Vu5zhlxWl326TMDQKBZamke9nCgUuiuKk/SEcGInjXqORUporl+MwDH2fpWm2VIIQRkHAL0qTRM9kAEJWuw0Zw5JEKT08v9zudlXDsNptLIqcWziEbeIBO7BdtYHXAXsi9d36gwc8CZ2lqdvtWq1WcWSEMeYvLfHiCsXRUavd1gzDarUKw8MMAN+yaJJo2Wzkuvnh4fX5+dB1s6WSpGmh5+mZzMbDh8OnT3fW1kLXPcSgLE3NXA5LkmtZsqLomQxCKA4ChHFmD5WpHlmrwufxvdEkDl6DmZfa46O2omlbyg8IEYTgn26Xj8PQsyxRlnniPARAzWS4yuzpLzvEmwAAEUFJREFUStEHW+j8Z59ZrRYlBEAoKYqayUzvIT+fqA7X2vy8IAgIY8aYkc3anQ6n4/gkGwUBT/uuTkx0Njb0TAbLst1u82qpnLLzHUeUpMESMu21NT2TsbvdJI5H9ggKdyApyj7d8JjRjuNib3nBvXi8Qd9/9dUr3//+W3/1V5e+/e3B2rYsTXlpO1GSKKVc6sTTgH3XzRYKke+TMOy3WrlyubGykq9UEMZWq1Wq1axWy7dtvhjtt1pD4+M7kim339ezWdeyeKj2K4Xu5iYSBC41FGVZM00BY8+2I98vDA/zRc3ju/xWbWrPw6KIJamxtDS0k96dpryW7M7YnFIKnrhE4lFAk8RqtWRNSwgxcjnPskgcK5oWet6X1+VX5uZCx4l9v3b2bGNpSVJVhLEoSYIgGIUC9whxr9HI78dO74D3zR0NIttuSjxIl1IqaZpvWRAhWVV5p84PDX1JWd1fPGe7ff2NN2gYVsbHh8+cWX/4UJCkxvy8pKrf+4u/+JIuOj5QFvugTAHIiYyjn/Qrkt6dENJtNFRdp0kSR5Eky1wqwqvTP8ULHTtId9z6kIOlvm+/++7a3bv5kRGzWIyCAAKQUqqZJl+npoyZ+TxfrvHCQ4qm3fvwwzgIhk6dklQ1SZJ8pbLr9WwuLzNKA8dpLC39zp/8yYH3LYr7Gu6Qsim/GTxBejdjvVYrUyh4lsULiMiqGgWBpCgkiliairLMiTuv32eMibIsa5rb7yu6DgEgcZxSqmjarlAVV9RIisKL5B90GySOP3v9dbfbzQ8Pu72eJMtIkkLLOvv1r2/Mz2OM3V4vSZIoCCYuXnR7vdj3s0NDhWq1WKsd3oY++cd/vHjt2sr9+9Nf+xo4gST87ocfzjxuP4xBDBZxcbtdXh6UMcYozQ4N+ZbFGVKuCi/Wak+l0t2+4JfmtYkRQjRJdirM8QorgiiGvs8T2LkyVjvCZg58BbkTLDh2lz+oitZjAXldCEI4bcxV9FwFxhjDkrRTF/nLQEqpsJ2BytI09H2eC0KiSDWMyPchhIHjCBhjWUaCwNIUpynnpZIkUTSNRNFWSCZNsSiGnscDTXyJPLgpz7GAjyUCGAQDoNdqcXkTpVSUJB6eTilNCEn6fSyKnuPkj1lhMqXUcxyWpkkcH5JzZ3U6H/3kJ2omUzt3jjFGeNli37ebzZGZmcUbN66+8sr9jz9WdL29sjJ6/vzynTuKqgqyDAHIlMv9RgOkKUBINc04CKxmc3Rm5tyVKwCAO+++e+Gll3zbPplBj8CH3rtXrNVaq6uFajVTLO50+TgM6w8erM7OTjzzjGoYbrcb+j6W5dU7d3LDw7lKhTGWUrqrpMhj0Vhe5tn3K7OzX/+jP3rs99Ptir1wIJuCh2T4kV1yNl59AQ3spsHbxEEp2U85rSZN05XZWbNY9C3LLBRylcpXxG2yu93I9/uNhtPr8QyzwHFERdEyGc+yZFWtTk4+FWXgl56nNFhMsLm0lBCiZjLBdtHlyvi49gSqjaMj9DzO0Tn9viTLvC4/lqQ4DIUDPKqT4SQFCE4GLIq1g8mLLxuKpkmKggThIMnCoFrgaV20sbIShyHYHkb2bYtbFzuGdHb7LDRJ7n30UeR5pbExHmT2+n1F1zkx3NnYKI+M8IkrWyy2VlYopVzsCyHkcrAoCDTTlDWNhy0ZYyNnzix+/vnQxESv2Qw8b+rgCZPE8c233iqPjT346KPC6GjouqeefXbt3j3ftvNDQ2alErmu2+sNTU4ihA5hrfbi01/84sK1a8uzs3uTekRRzOTziq7HYUiTRDXNNEmgIOyIPCCEeOn27SgI4iCIo+hrv//7ey/w/o9/fPX73//1j350/tq1oYmJHU8IIpQtlVixKKsqgNC37UK1KioKr7okq6pn2xBCURQRQrlKhRfeUnRd0XUBIUnTeP1BPgXxWi8AgEypxAvBHq6WQYJw4aWXAIS8siMP5VcnJnjpRx784Ylox91I7OI3v4kEYd+5NCHE6nTaa2taJsNLywuiCBHiSbuh54myDEPf5+pm33H2zZYIXFc1DM+2RVmWZPkrUj8UAODZNq+JSOJYFMU4iviGbDtsrGaafEswSZb5Jnc8AYXHYBhjEAAuyBg8LRdjJYRwPmjXBlWFanVvybfB/o13ZM4H5Z5sCQT322/v/scf251OZWLCabdr587NX79eGh31LKt6+nR3ba0wMtJYXOSJGp5lZUolCOFjd6FqrKwAxkLX3VhY+MbBJTI8267fu9deWcmUy+2VFdU0zWJx5fbtyRdeCG07jiItkynUat319YQQEgSl8fEoCPobG0kUadlsyhiJoksvv6xlMrv2avjsjTcuffObjaWlvYLbJI5n332XU8AQwtDzyuPj6w8emIUC34ogTZKT77QAAOCbvm21cYSQICi6zkPVoe/r2SyP0evZLM9JDlz3scUgaZLwNEq33z+8mMaXBC5V25nKHqkfurGxc/++bWv7NTLcazSQIPAuYObze6cmu9tVNM3udHZtjZJS6vf7XCyYRBFDCCHk9fuSovCankkURUHAKPX6fe4eSopitVoJIappHkQLCRjzqxxuzcBxFm7dEhUl9n2zWIx8X8/lIt+PfZ9LGKdffPEQadQhQAiBg1IdGGsuL2+JQ1stM5/nDcUsFnubmwAAQRBwZ22NUoplud9oXP7e9+Aesn1ldnbmxRedbndXnjtESNI0/jIx3/2NMR414ytlHmGmhLA0ZWlq5vNREAiiKMryk7vcoixPXLwYh2EchvlKJfR9Sqkoy6hQEESRz0snM+ghIHGsmabOd97R9cC2zUJBMQxKSKZU4lvjPVGXtzodJAiSLPNRXFIUAGEcBLxEDNeBkjiWVfXoz7ZVwjmO0yQ5pFgNYwxs7/gLEeIb8/B3uSNy4dNLmqYYYwAh34VXUhRefpNnGaE9ZXhpkiSEkCjiypRHJqWNDV6QHgCAEPJsW89kSBxLihJ6nmqaVqv1BKmJaXr3vfcUVY2jKF+tirK89PnnE888015dHTp1auH69czQUOx5JEkuXLtWPPJoeO+DD85cvjz/6ackDK++8spBX/Msq1mvr929i2VZkmU9nzeLxZRSXuD64aefjp0/DxBy2+2YE4xpWqhWe82mmc+v3rs3cekSgJCnEO+qiHXzrbcuXbvW3dzcK/UhcezbNqepwHZNC1nToiDgJScRQk/UQn+bYIxvjMWj1jsroh2OI/Q8vrEbp0J4K+b1UCRV5Q45AABCyABLWSrAL8a6rbAjAGCP25QfGgo9b9C35aqxnX9P3kJpkmwuLCSE6LlcYNuAsUy57DtO7PtbnB4hRrEIGPMdB/Cq15QCCEemp8Un36MKwi19FoS8MtUXvRsAp9fjm5ClaRp6XqZQ4B2ZS3O52ofrhyVV2QhWunGzItcqyggAgNudJwXsumYSx3MffYQVRcSYkwYAgEypNBhnPblBebLB1j/bCsUvO9i5g9DzNhYXm4uLsq7zHanMYhEh5Ha709/4RqteX7t/f+j0aQCA1+9XJibqs7M88NWp10+/8EKrXk+i6PRzz2uncjo2Z51PRrUtycGNN944e+XKyuzsM9/61q6LYkma/vrX+d+8UP/eG4N3P/gAQEjCUM9mT++3dr7x5psXX3rp5ttvT1y8WB4d/WI/Jcbmrl/XDAMKAteI8TTzwHGiICiPjcVBwMvUJISY+bxrWaquS6raqtfNQmFkampf2oJXPiNRFEfR2BF2M34S8CIC8+6dgHqT+oyBv/ArdxY/g12+vbZGgiAKQ9+ytGw29LxsqWR3OlomkyZJrlptr67i4sgIFARwsKB9/MIFUVFGZ2Z2+7GMVcbH+42GwBjk/jDfJ0IQFMOQFMVutymlimHwqD3fSIKlaa5cDj3vIM+JC3t4zclDbJEQ0lxZYYztBFpESUq3t0cG3J/N5XzHsbtdxhjGmBc2rUxMUEK4/8Dn9ynjkRVRv9nk5P/eWAOEcHhqiiYJH2F4pZnhM2e4CoQxNnHxIn5s2ix3sL/IGhuYxGRFqZ05w6kmmiSSoqi6HrquIEmiKFbGxpAgHLGk1w52FLmHjx4QQt52SBzTKIrDcHRmhmcgkyhKCHG7XXFmpru+7loW36IiDgKn0xmbnoaH3tL6w4cTFy60Vlb2GpSm6dr8PNgTAhjMcz32LL8TvWKMeZaVJomkqp5lceEngDCJIgAAwpgzWieLzHx10Gs09J18ed/nNVYAAGma8g19IEJ8rbxV/eM3mUL8TxFuv6/qOp/Qu+vrvIacIIqSLEuqGjhOvlrtNRoIIVFRAABwY2GB+72DdoUDpPQjx+HWO9j1GvY9fuBJtry//RnvL8DY8MF7LQau22s0DlnCbpWK4VG8AfBdStK9t4oQ2A7hDZ4EIQEwlqaUDWwcDRGCADLGT/MFd4cEAYZRFERhztxNnPQdGwuCoe3W/vasvq7p0qMBwpgQx3OLud1ZQI7nMsYyxm4epO/Yiiwr0lNeaw8iZWyjualIcjH/yPrV9f2e1SsViqr8SJhoo9UkhIzVRgZfEQNgvbEhiVK58MjA5YdBt9fLZbO77LOwuChMPX/h5p1by+v1sxOnd7Lbfv3x+91O64Mbn9iuO17bmrX8IPibv391fWP949uf5TLZXGaLUV9aXfnHd97QRPwPv35r5vQU3naG3nzvncWVxbXN9fnlxalTW15ekiR/87O/Q2n6xq9/qWpaKf+ljLBRHP31T3/0YPHh7bnZMIzGR7YeYXl15Sev/30ahq+9/854bczYpnvefO8dTcJJHL/98ftnJ8/wlkgp/cnrf7/e2PjwxnXXcyfHtqblzWbjR//w48X68s17d4q5wo4dXn/99Z/+9Kcon8lePjuzur56f+EB/2Btc51RqivaaKF8Y/aW7Tj8+I07tzRZmayNXp25dP/BvZ2O8fp7b//BtW89c/GZM8O165/f5Adb3bYiir//0su/9zvfzOn62uYGP37r3h1ZFDOmiVP2yY1PvqQR/P7DB8+dnfm///mffufq7/ieYzs2P/7RrU9nJqdUWR4ysh989vHWrXbaIKWlbK5aKlczubsP7vPj9x7OhXH0B9e+/X/+4R/Nz91ztt24z+/fmRk99Y1Lz58bGV9cWuAHbce5c+fOK6+8gp6fPn/5uctTtVHb3trGj9L0QX1JVdUojqqZbBRt1T1CCF48fcbQNVmSmpsblNLt7yemboA0zZmZjbU6PxjF8emx8b5tWbY1OTJq7Wx1T4iCBBLFiqK4vR5N6ZdhUAaAAGEYhla3u7Fa395QE6RpOlEbqQ4NT0+dqy8ubH+Z3VteaHQ6lmP/17/9yx3/FwKYJMnq2mqj2Xz/+vvp9q1mDVPEGAImCsLG+urWlxG6d//+//jRj4R/+Wf/0rb6t+/e/u7L31MVFQBgmubDpcWNdlOXZMfzvn5la7GlKOr12zdc2373o/fPnT1/ZrsXY4TnFucVLP7l3/63P/3nf2boBgAgY5g/fv0fAsdptdv/4+9f/bM/+TP+5UIu9/O3ftFuNd//9MOrl1+cPvukten2RSGXe+29X/33//e/f3zzk2cuPPfspa3cpIxuvHv9w8mR0f/y1//1L/7V/5XL5gAAuqZvtpqqInuO83Bx4Yf/6s+xgAEAxXzh7vxcfXPtb1/9m999+feuXN7KT5Ak+eHSwn/70V/dvHvr//jBn5aKJQCALElIklubG3C9sdFutSZPTXJDcKRpWt9Yo4RMTjyiv3B9r15fMXRjbPSRbJ9Gu9lqNsdGxrKPhioXlhfDIJg+Oz2YexzF0Up9RcTiqYlTT9WMj4CmaafbYWk6VHlEE+l6Xq/fy2Uy5qPz8Mraqm31p89OiwPzbcrSdqeTUlodqu46Sb/fM00zm3nkeesn2KDqf+Nw/PYrnv4vhv8JulMmxYa8snQAAAAASUVORK5CYII="
menu_belsit = "/9j/4AAQSkZJRgABAQAAAQABAAD/4QBoRXhpZgAASUkqAAgAAAACADEBAgAHAAAAJgAAAGmHBAABAAAALgAAAAAAAABQaWNhc2EAAAIAAJAHAAQAAAAwMjIwA5ACABQAAABMAAAAAAAAADIwMjM6MDY6MTkgMjA6NDg6NTUA/9sAhAADAgIKCgsKCgoLCwoICgoKCAgKCAsIDQoICggICgoKCAoLCggKDQgICAgICAoKCAgKCAoLCggKDQ0KCA0ICAoIAQMEBAYFBggGBgoPDQoODQ0NDQ4NDg0NDQ0NDQ4NDg0NDQ0NDg0NDQ0ODQ4NDQ0NDQ0NDQ0NDQ0NDQ0NDQ0NDQ3/wAARCABwAHADAREAAhEBAxEB/8QAHQAAAgMBAAMBAAAAAAAAAAAABAUCAwYBAAcJCP/EAEgQAAIBAgQDBQIJBwkJAAAAAAECEQMhAAQSMQVBURMiYXGBMpEGFCNCUqGxwfAHCAkzYoLRFRgkNESSssLhQ1NUcqSz0tPU/8QAGgEAAwEBAQEAAAAAAAAAAAAAAAECAwQGBf/EAC4RAAICAQMCBAUEAwEAAAAAAAABAhESAyExBBNBUWHwBSJxgZEyodHhFMHxFf/aAAwDAQACEQMRAD8A+Y704/BxmaHhbp9hwDIun4v/AAwAQjDA8I/H4OEMiRhiPCMBLInDEQOKGQNLDsk58XOCxHnZHDsAzIcUq0wypUdFcFagpu6iorKVKuFIDqVZlIaQVYjYkYBFCLhMR9Dh+joyH/F5z/pP/nwsBdxgFT8wThdyc3noBg97JC4LfSyw+gfMRE4FFFttclZ/MN4UP7TnzvtW4aJjffL8p52w8UTmy1fzA+FxPb8Qvf8AXcO5+WV+wx4nDwQu4yafo/OFn+0Z+ena5L32yRwYIO4yur+j/wCGAhe34heb9rkoEDn/AEMH3A+MYMEPNltP9H7wsf7bPN51cv8A5couHigyAKH5l3B9Iec2VOn2q9Ie2iuPmAey42PlIglKnuipJxdMIpfmZcFvPxmxgzmU6TeNpAJvfFUiLYblvzJ+CsQAuYJIkD4wJMYeKJyY1y/5i/B+dPMeuYe/9374w1FE5su/mQcFBA7GsZ5/GK33OPsxWMRZyCP5j/BNzQq+P9JzP/sxLSDNi5vzMuBQD2NdZ3DZqoI7qtzfYKwnpB8MZ7GknJM/VnZYszM3xDh601AmNTAsS1JZMRMspBJtYXJi/LEVRo5OQHliJuFM27z0ekR3SCbyIuOmGiQ2mgGyqP3k/wDP7cUSSoqJ2UciZXrtZjecAyGYyvyiW5NyXoOZMj90GeewwykFtlMIDPV/g7oRUpgkLAuWJ7qBZOmpTiQJPzZJsJwkqRcpOTtkaPCak7GwESXjY9cxfe1gbXLWxRmxlkMm+5XwAOoEEebsDPWPU4dktDWlS8CPOPuOAmjnYd9d4g87eo1jw+Y3mNwWFBXxOMFiEHFchpk3Oti1tVpQLFlb6EzY3ttGJKu6NnXqQpboC3nAmMTZolbozvFOJAhSygd51g1GX2SFPsrDDVAIaBHoDN2U41sDdukxoUg2Pyxnwtz5SSRcfOscUIY5GCJWmI2kMpn1EyQImTN+e+HZNBdLKTugHqPSLX9SPXbDCiNZBrUaQTDQe7qFuV5vF4B2vgKoJOXwWFGZf4PaFKwCNS6SdNwlCnTDMDQcKZp3Cqdp1ANoEJUbTeW/vlk8vkFJEry5rTvYb/0feAZgj2tttLsyoaLkRz+xTPvpD6p9MUS4l1PKLyEnyE/UBgDEjmqIDp68xz82BPoD9eAMQ1UwWLEXcZoHukfSvAJtB5BgY5c99ok4kMUPYwFUZrjqgEBdI9o+1TEFzJMNTe5a8gb74llFGUINp8gGpmQBvHZDcENFxPQe0xDjKZW/+qyPdS6eP3YYWHtQi5J/HkMAWR3Ybxf/AHkbdNOn+8R4XjDALFLAMXccyshf+YG4Y7X+arX6bHowwMAAZU+cAA92ryt9E7+RNrk8pAZ0cv4R6PvPTSLfdhgX2E29yt/C+HYFOYpzpiQJB9mp9cEaf3wQOYOCxE3p4YCf4QINN+TA7Kf8RH2g/XiQHTPiLLxM5xyo2sASO6OdWLsw2RSD0mQfKFmbY6RTk6FRphjbxzM3PMwJt5gcomcNNiaQzy3DXmSxgbjVWHLxMGYudJPiJxVkjbL0Ao5/3qj/AOKSPTDA4LsDeL7q/TrqAHqp8I3wWFB4wwozlEMqDtCQdVP26rJtQQNDS2qSrErYFtRuVLNnHjc11Kb297svGcA5gn9rMsIvbbrG8dR0m7M6Ll4qEBHcgXOuvJHW7BoA5CYm1owWFHaXHlO7UwBMw8nmNgvUDn16XBUSzWaU6DIIJse8fURb1a2AKZKrWHXBYsRPx6uAhOoDa50x7Q+kCPf7xOCwxGfE8/oWd+8oO+xNyImSBJA5xjJukawjk6Fj8eYELChioJlkkMVBiDWUkSRy+44EwkqbO5T4RTuUEASNVIkTz7uYaFFt482GzsmhnkuIhtiDG4WCR0khiLj8WOHY6CTmPDBYUCHi51hYt5Gduur7vtBw8h4hP8o/iMKysBVxripgRrN79miMdjyewEwJF7+ZDE4is55z82v5mlSAvaTPL50+J8sAUMcrVgXptN/aCyItaOVpuSb3PQGEBjyp+8D+GAAbO0qh07KJ+kVt6e0P2dsAjr8HJ3b3D/U4AM3xPJIxdCdpBOpRdFpMb/NPyy2I5GYkTOW9FOG1+/H+DZV0DCGAI6MARcEHfwJHkcZ8iTa4M5nsz32GoDaJR4Fo3FMyLE+1YxtIkESo8RY27Xp8zME9bDuzI3vM2O8B2IccPpsbliVvYrVVpt9JzYX5eREXdjGanDsKB1B1zeI/a08/29M/uTtforHQwD4djoT5/PK2pSgfRUCj2zc0lfV3KTlYFSLjxm4GEpDlCkn5/wAtFVCghn5JQZ3+VmRc3NEWEC4kG+0EF2TiMaXDaYsEXxsCPr/BwWPEILDBYYsX8TzgUTbcbx97L9uDIMWdqZoYMisGKuLEEar61VlUjkGKkiJgyUXcHbxMzYU0mijiXH+zAtMzAmNlJ6HewvG/kDzOdHXHQyEee4wupr3kWNcLzHzYOk9BvqgWmcXkY9ugilXBF5Ngf1oO8dSN4sY+0wZB2w/LcUC7KAYj21NvVjbbbqMGRWAUnHCTsI5nULHpABvgyDD0J5bOS8wurSO8A2oiTbVpEqOk2JNryXkUoDD4ycLItQE/GaYiwnU4Zu6WvoC6v1FaIVVA7qgACCLAiYSjdAdOowM6Tz+YeUQf6sp22AI/hVkqIfRzDxzHgBH+QXjzv0wWXiWJWY7yB5kb+u2FYYlPFKjFbSTbbf8A7if4hh2PFHatU4LCkLuIElWHKDM7ERz8OuFYOKOcVyxdeUiI9SJO4uFB5j12xzM2i6YlzWYOpr1B3jHcy0b7gkyRHzmi0yNp0MHz/wALqWe84i0pRm0SLHcTfkLXExgoLDsvXBv7S3jSqRv62n1vc4Qw+hmSBYeQJUfYtsA6J0cydUnaOTsbz9HSBt86Z5RzxQJBXxzDLoz2f4iwL6iQO2XRNQ0xp7OmsA3kFw/dEBmPLUcEeWGpVR+n+2DDiF/bN+tdiN5IgAz7gALRcE2ZV7suy+bpmQXYwZGlqxtEXtJE7i4sJmLMaSGOW0CCJ8CS038Dt7sItRRDiWcsdvVWb6lIOENrYrPELDy8R9R28jgFQJmc7gFRw8VxOJDkYrMcVGpjC737jSQSRBOnn69L6hFUY5BmX4mIG2rewMExuAAIuf8AQcniOwynnyNhF+lS2+0el7emDEWQUOJVOWn1NT7Cd/SfLbDxGpMjlOKtqJIAMRKqb3HzjcyZOm/WcViCkw08XOHiVmxXxvi57sTuNivX9rcdeceMYaiS5MS1eLHqRyvUpja8WFj3TPMXvh0RkwzKfDmnq7MsiuF1RrQys6ZLWvqVhB+iTe8FIpSGY43IBBkG4INiDzHUHBSKyBc9xfum/LqQPUi49L4KE5FKcWsL8hsSRt1O/riQsoq8U8cAWEqX/E/wwYmXcQhpZ6rJkpHhTafP9bt1taRgxJ7gTT4qejEzB0qN7ctU8/HY4dC7hcOJn9r3Ly835YdB3Dvx5jsG9w93tYeIu4ToUqhv3gNgCu5nf2piPffww6F3Apsk/X6h0mPaw6H3BZxrIN3d/aHzQef4vaN+WChOdipeHNqCz3iJ0hKerTYFtN2KAlQSBbUATMSUicmCZL4LOlY1/lTqQUihSkIFjqkMIggSADsSLNhY72V3DQ/FHNtLCerUo9dLMYMbqD6YdDzZytwpr96P3gPrKWwUGbI0eGk8/ewJ9Y9+ChZMXcW4Q1iDAHtQWkntEgRsRAbUZkAwPaJXOfoa6clw/fP9B/wq/Kzlssyo7s1ZgD2VJWd9Okv3glqZZEcqrsCwRioeDjOWtGOz59DfQ6LV1oucaUfOTSXKXjzu1dcWrqyPwY+FGUzQY0XJKEdojGur0i8supC40q4kq8lHWdLEDFQ1Iy4MtfpdTRxc1s+Gt0/OmvFeK5XiOKVSkpsw1bHUWYT4BmIVpG8jn4xoclFv8pJIXZiSQFAJMReyATsYmY58sMQU/EtvG9iTb3CI9MMDtPMzcknlYGCB0I873PPwwwKq+ejYTbkTItI8FNtiAQPdgAS8c4swgEHuxEsqgG3Uib+J1TyElgBbV4gpiOyeSQRUqMRMctKkFiCZtfUZMMVwhhGVySwS1NJ9nu6xN73Z5aG5EWI5i4YHavYrACp3iBTLKukkwFhoYSZA3JMiwkHHJLqtGLxclZ1R6bVkrUXRfma5sJgm45k97cDs789jJHIQcdFnPQNluLLsNxMgaQN9h0I3CsAb4LHRVmuLFdxvEQBI5XgrFovB5bYBGH4VUqg1Sgp02qvFXPPTYMzCkqOEGlBRctTYK5+MCpUBVhT+TDPGC3SHLVnJKLbpcenu3+WOOCcGUZ5a2X/q3xM0ahIqg1WFSmaVTTVOmpIWowrKpLaizO+tYwcHnl6bnatdPpnpPnJSWy2VNPfnfbbg2WayoaO+y6iSflGVWkEd0CsIgX0qtjcKCMaHCTzOREAVGZRAstV1AhrEw5jvdHuBdYsWTRdkss2yLrFzLVNe53PcAN+rQPC0UABmONwlWorq9ND3uyJq1CVklKaUBVd2XTGlVYyIBBBnDU19PT2nJL6s1hozn+iLf0RfkOIqzdktXLvXUgNTp1qRqLqCsissjS5DoRTAkh17t7C6jTyjDJW1a9V6Ph/Yv/G1cJamDxTpuuH6+X3M7xD4Z0laurswqZcoKqlVMPUBOiVQaXCrqcAQiSSVB1Y4db4hHT1HDFuvt+PPb7XtdndH4dqdrS1W1U28Unb2dW0uFeyv18j1fxX8r+f1M2Xy1GrlzHZ1VavCqxGlm/VCKigqEpuGL6QvabNn/wCpp3JP5a81X9/tXqbv4Prxjpz+WSneOEsnS5b2SX5b/DHXAPyuVKlFTXpOmbABqrleyNMnUG7peodxbQx9oPaoqtHB1nxWEoduD3e18L677+/z9LpPgmtGXc1EqV7O2/2Xv6mzbjNHLUFq12UXNwQ60+0LFYNMsVpgalSoYhQFLSBj4NtNwTTa8FTW/G/7HY4OcqSdeqauudn4+LX34M3xb8ruUSKXylaqFSsFy6PUapRrOArMopwGUwSWhggV5YNGPRdBratJanG/m+N/6/H2+R1nSRyuFJ2lvSVt1y3x4+iT8j1r8LPygdrUDBqtPKqSDQAK1GZQwYFZhyzhSodSRrZbQQ/J12tqak2tKTWy2bcUn61+f2PTfD4aPSdK46nTx1NRyfzfJP5Vw4Xs73W0kuJU3xsqX5UFD0URXioqlWU6WOqmXaQXCsaS06naWL6tIVCZBfQ/E9dp/wCVGKptfLfg6T3b2a39TxfUdNl1MtDpoylSUvBUnXnX6W6aVteWzP/Z"
menu_sottosopra = "/9j/4AAQSkZJRgABAQAAAQABAAD/2wCEAAMCAgoLCgoKDQgIDQsICgoLCAgICwoNCwgICAoLDQgNCA8KCAoICAgICgoICA0LCAgICgoJCAoNDQsIDggICggBAwQEBgUGCgYGChAOCw4PEA0PEBAQEBAQDw8QDw8PEA8QEBAQDQ8QDxANDw8PDg8PEA0PDxAPDw8QDQ8PDQ8PDf/AABEIAHAAcAMBEQACEQEDEQH/xAAdAAABBAMBAQAAAAAAAAAAAAAFBAYHCAIDCQEA/8QASRAAAgEDAgMFBAYFCAgHAAAAAQIDBBESBSEAEzEGByJBURQjMmEIM3GBsfAVQpGh4VJiY3KClMHTJEVUg7LC0fEJFhclNDVD/8QAGwEAAgMBAQEAAAAAAAAAAAAABAUCAwYBAAf/xAA/EQABAwIDBAYIBAUDBQAAAAABAAIRAyEEMUESUWGBBRNxkbHRFCIyQlKhwfCSk+HxI1NicnMGFYIkY8LS4v/aAAwDAQACEQMRAD8AMKQLm33fL8dvu/cePqS+XQjHYvsPLVSCzYQRn/SJSP1fSK5AMouL7EKLXBJVGT9IdJNw38NkF8TByaPidw4ZnRNcFgDiPXdZm/edw48dEP7+K2uSNqSiZYFTARunhNdL4vaKZJmljmpqtlMTowVDUZhUqopJFThR0LgqHWelY49ZUdMk5MyghsEFguI72FoAOmq1AGCnRAa0ZD6nj95pr9zdYgqYtRjgrqMSQuNRoWW0dZMC6xqVlUPZH/0nncqM4nbF5qgcafGdH1q9B2EfsPHuPPuixm2drbMkWzhrSkb8dRwz9oEzqB9wD5mxUm652zmluosiP1RPO4/XI8TE7X6A+a8UdG/6ZweDIqEbdQe868f2jIfM8Ulx3TeIxILQdhu5uvacz8hwQCSoVdyR8x6n/D934ca0uDcys6GkmwSOp13baw+Q628vs/PXyoNYmzQiG0QPaKQxxuxtb7fMn+A+wffxY2i99zZRdWYxOXRu7gW5khCIPidza3rbbY/L8R0mXU6OVyqJfVtkFpPejTrklGFkZDi1TsVDf0ZAIa3yuCfNTa2XxnTbRZnrHh7I568rcVosH0K913+qOPtHlpz7kEg0iWdryOWJ8mvYfZvbfpfqfU8ZKviqtczUM7hoOwLX0MLSw4imI46nmoi+jt2N5faesiI//GpAB8izU8o8v5I4dYelNNhO/wA1SaoD3t7D996vfR9ngPLyH5G1vL1P3XF2bIaSEM95KOUukj046XqglVDbUsnwAJJUyFwNhGp8Zvc+EXG58PTrdipLPWO4b0ieQ3PNObs52paAxK8p5CSuwdxcw87FSpJthTrlO5dmOBcHaOFlbI9N/wCmC5z8Zh5LiBLdbTJBm5y9Xha5haDozp9rWtw9eA0TDtNIBGgzk98RKmnV+yEcyskiXWQKHxIKzoOgfJG6gBckwlUAAPZU4weH6XqYUAC8ZXgg/ofvRa+rhG1DOU52sVA8Gpwlpgs8cghlkiZo3De9iOLr4TjmvRhf4r8fbei8cMVhWV3WJFxxHnmF8s6Qwhw+IdSGWY7D9wk02qEmyggfLckfuA6eXz68H9Y99moMU2tu5aYKC+5vfyv9u/2fn7r6eGLruVL8Q1tmpwaN2VLH0+Zt+/rYefS/yHBuwykEC6o6ol1T2opadhGgEs5NrDdEbp4z8JP80Emw6cZ3H9M06MtmTuH1OnjwTvBdEVa/rRA3n6DXw4qAvpI9qKmSSpgeZjGsdPaEABcnkbmbAXIICizG1h03a+AxmPq4ow8w34Rlz1PO3Bb7BdH0cMJaJd8Rz5buSff0cexuVOTb9Yb+mwHp/D/CumwuRL3BqsZo3ZEDoLkeVug87Hax6bi+9xwxp0EE+soE0/SOR2yhAG1VGQ3UnFtPltck3JzhUb3/AMeNJSYBhtr4SPED6pE9/wD1Pa36/orkCksSbdQPTyLff5+e32b3H95XlyWwJx4qMqjOu9mWkZWErrjiGA6FVN1ta4JJO4a4Nh0tvqm0coWOfUnNaO2XbyGlXFrSTMpaOkR4xJIAQL+8dUSNSd2dhsNgx4HxvSNLCNjN24fXcisD0fWxbpFm6k5ct5UVwd7naGWCTTwsMUFQQyNR1VD7RS0RjAaCmd61E5ZJQlzi6AyBCsYRYvmVbo+nia5xTxDjmADszv1vv3m+a+mYd4w9NtAGQBAJN4+8twst/c32GOnuYsawQ1RXL2hqErBOARGQKetkkHOAEZxjbL3RJVVu2s6OY2lU2S4Q6BrY6ZgZ5dyz3So62mXtF2zzGvdn3qe6TSL9B6H8/kDjdspNZmvnb6xdklWo6tBT/WPd7XWCOxc33Fx+op9XsPmeFeN6VoYWzjfcLnu05o7CdG1sUZaLbzYffYmzW9rqipOIAjhvtGh3e38sjqfkLD7b8fPsZ0zXryGeqOGZ5+S3mD6Go0IL/WPHLu80b7Kd3Q5imy3AG528z877HzuL39QLJWUyU9c4BRT9ICiU1dX5nl0qrjuLxyNfpffHEn5enEn0Km1DWk9gKlTqs2SXEDmFP/cDqdLDRrzKqljNgbTTRqR165OLHb9m/rxocLhXhg9Uz2FIsRiWbRO0I7VJi96+mLYfpTT9rfBUxEbD5Ob+VrDyFvXhh6LW/lu/CfJLzi6P8xveFXXvH7b03/mXT62OeJ6aKKnWaojbJY2L1KS5WubpG6OQAdmFr8NGYep6M5padqZiL+75JbUxFM1mua4EQQSDbXXmrFVH0lNHH+sYjb+THOb/ALISP2cBjDVvgKIOIZ8Q70kk+lhoy9a1v7NPVH8Kc8d9DrnJvzA8So+k08p+R8lTus74PaKuLTaGRDPUOY3riucFMQrFigyUVDqBa6sI1bqzFWQ3YrpNz5p4XLV3/r59yjg+imgh2KsdG6893YnVpf0YKiJJJZtXjkcN4yaKKSSTGokhmJDyrvEYsmSN5ApurNHGvPZTTFAO9amXE5kuI+/uE3qur5Uqga3QBoMcFI3Zb6PDTcxTqIKmWWKRjptGmTLVT0t0ymmF+ZQPJy1KsY3p8Sjuw4JqYilTiKUWB9txzAPDRw5zmgG06tSZqk5j2W6GNx1HO2STaJ3AtJSiolqlAejWpwSkpELS4VE5jGdKWjZaf2WexAYCSUEW5eE6mLptcWNpjOLucdw+KM5+XGYsw9WJdUPINH03R8+SDWOwNQYZ4o9SqoxTysgqX9mM8gWkhqAGb2ZY1xWcrcRCUpDcy3ctxVicfijTAa+xvxiSInl28UVhejcI55cWXECNJgGY55ZcIVSu+DsNLSV3ISrrWBiSR2mqHd82klU5sMbmRY0qBkMsZxuwKkr8NhetEuAme1HYyt1MNadMt3LReaLTOvWWUn+c7H8WPGjoYJjfdHcFmq2MefePeUejl/J4atZs5Ja6ptZras9v4cWgKEiICzWq4kuSvjUcdBUCVrkm47IXk5+5yijl1OghkRXjlqY0kjcXV1YkWYeYJttwHjKjm0XFpgxp2hFYVodWa12UnwK6EP3C6OvXSNOt/OgjP/EvGWGIrnKo78RWl6ikPcb3Bc0NA7vm0+t9naoWCWmlKPWQ5sIiAbPFgFlZJFIx2Q4yDIJ4rPKNFgoy1u1Iyym+XektatUFYNc7ZIOecW+o8bqS31JGDJL2orCpZFwWKvdXSMl42YNOqWiluyizEM3MFiTx1ocLtww72eU5eSqfszBxJ7nn6whmt61EkRMWs6hJJfmCMrURLzJnVpy15WAlJvK0gZuZIlySSp4upse58VKLQMp9UmwtplpGgVb3saz1Kzy7d6wFzfzzue1MqfXpTe88xubnKRzc4hLm7bnALHc74AL0AHDEUmDJo7h2+N+1Ada74j3n74dlkD1akEnxC/ndt99vXe+w/YPTih9IGyvbVIvKT0GmKgsFAHyFuICkApOrE6o9oUyCRS8TSoMrwozKZDg2PiUFlVWxdiovgrcSe07J2SAd50uoNcC4bQJG4a24cbngnyvarSWtho9SwXItesluygWF8ARHiSrEi48VrWtcI0sWPaqtH/EfUI0VMKfZpOPM/Qngs9J1+kXcdn5pQGkIZ6mqIxdmxRhHBgRB9UrCzMY7uWbO/HMqmxrgcm+c3z57oXWOpgyKBPaXHlEEcP1TCrX8TeDDxN7ve8e58JuARh8PiAO24B4aBsAXm3fx5paXST2nlw5ZLAzWAPrf7rcTyEquZKzSovsfuP58uOXNlKQLp191enSpX6ROUtHPqVIsT3X3mFZEkoADZDHK3iVb3Frg8LcQ8OZUZqGknhaQmdBh2mPixNjvg34rpDWaQrMWEdTe9rF2wIViylBzSqfIgA42UiyBRjxAkGb7lqXTaIXN7UNOg/SjQmpaqDPK8hkSoaQrymaPnezQrIZLBXIpEKRqoPhjF+NDg8ayvhi6jaIANgM4963fmeKXdKdGOwuM2KxmSSReR6sjK+toyATxr+yFKV8FIuTKwYrTa8xRlQlSmQjQlmEcZv0yyJcFi3W4itq/jnS/XihjhqJyZwyf+nDzSpey0IdQtDHcKRidM1d9wytcrJOGeVQq3uDdJrEsGI4519Qgy8/mUx9Lfouej0xEMH5bzuvn81rre74OjRGnETMYi0kWmVKtGHlUOscstdyLIWjU5AjCQqCuBCdbjCw7ck52NRu7UBs7+Y4rjsGxwLIANrimRroZj9E19W7qo4U5ssuopGLXkNCAr5NZcC1ZZb7LuHGW9yHWxbMa6o7ZY1pP9/jby7LIR+BbSG09zo19Xla5+aj4yIst1Z2RJAVdkVWZFbYlc5FViBfEvIAdiW82ZBc24vH3uSoOAeDNgRfhO5OnQtbRtQWVZGRSWKyXipSlqdgd0JgguQR4SQ17dX4Bq03NwxaRJ3XdPrd5+9EdSqNdig8GBPAR6saWUpSdqW+ATyHED6zXKNccSTY8pHtiqtkW8ASxyyIyTigIkt7qTj4kcvCE69IvDTPbUaO3KeekaofNroCG1VGDkCSdelPxtdlVI4SpQXZC17nLmE2uBa1pB9g/kjxJUC5hHtD80/RR9W9sKJiX/RbEyFizy1tSzlmN2LmwvISTcm973O+RLj0euIirbKzG9yS+kUTnSvn7bvJCdb7RwSIFWhjiYNcyxyzMSviuuMkjRgXZNwMgIwAQGYG+lTew+u8uG6AN15z/AHQ1Woxw9Rgae0nfaPvKEG9oA6X+08FW0Qtzmnt3fzASabJggMGrUoMiwMGcCaOVebPfluykMqw2DhFyJKqtlde3WCTemTEiBYizc+eWmab4Z1mGBZ8ZXvvdlrlnkV0qqtOTmN4ADk5yWCS5yYn4lbAksSxO2VybC54wwJGXgtbE5+K5Idm9VaGrqJjk5Ms6gmSSNiuTIhyheOQFVC7BgpAsQQSOH/RuEDcO1gtYHIHjkZGaB6Wx5fin1X3MkZkcMxfIKRT3wL4saKPxW+sqq+TE28R8VWBkbsQSPCGAOdiWZDAzG07uawfRJP8AcYnZb3vcU0qvtLKzu/MkXNnIVJJbIH6qpaRnKBbJ43YlQLluGTaLGtDYB5DyhLjWqOJO0e8+aRtXMeruftYn09T8h+weg4tDWjQdwVRc45uPeVg8l7fIWF/Ib9PQXJO3mT6njoG5cJOpWMU1iDYbEGzAEGx6MDsynoQdiNjxIiRCgDBlHND7SgVSzsOWBlcUkcIw90UHKSRTALmxbJT1Zh4rHgWrQml1YM/3E75uRfs7skVTr7NYVTb+0DdFgZHbzUgT96tMfgkrxa2KrSaVHcKxKjwq9lUMTj4lLbY23Cv0KqREN/FU79E19OpAzLuTWW+SYveH2njqJVkQ1B92qv7QtOpBUmwjFMiJgLnd1z9SbDg3C03UmbLozm0nvn9kJiqraztps5RePlHbqmzzODtpAFt1iW46uLMPxKVGEQ0zXBGF8IutRBNzbm6iDO62+Ehsw9zupSw2duBarNqb+6Wx2x5Iqi/Zgf1NdPZ+6611HaiEOyGVQwNipvsWAIHTzDKfv+Rtgg4Cy2wac1yk7rGqkmd6alWaSOIs8bwrMFiEkZZgjXybIIvgDPiz2/WK6iq2kaYbVdsgnfF4P3eyz9J9UPLqQkxe02kdl/uE5+12galUlC+mCPlKQvIp0gUiR7AvYgOxYWDE3K+LcNke4erhqEhtSZ3mcuShiKWKrxtU4jdA8Sg//o7qANjRSgk2CsYwSxXIAAyBiSu+w62HUgEodIYf4x8/JCf7fiPg+Y816ndBXkEikeymzHOLbZTv730ZT6b8e/3HD/GO4+S96BiPg+Y801a2lZGZGFmjZkdTbwuhIYbEg2II2JH28HNeHAOGRugnNLSQcxbuWumgyZVFrsQBc2F2NhcnYD1J6DiwugSqw2TG9EZ9CEcpilcIUvmyWkAOGSWKtiwYlVurEWa9zbinrdtm3TvuBtqreqDKnV1bAZkX0lOPtH2e0tEkMWp1EjqG5KPTPHmwPhyJ8Kqdt8gepIXZSvZWxZcA+mANfWBj5o91HCBpLKhJgwI1008kw2lvww2pQIbC9DceldIXhbiShC9Dcdle2Ulr28LD1U/hx4G6m2xXXmh1liiMJSMo42AMTsBnGp2KkZXJJ67dPLj55MfstyAuW3dxqNKsrtJJDyjZUWqpZZhLEXJLe5lilppMFVbjmA84oyFQ5GlqdZUpN2QdrMw4CDHGxE9mWaSUzSp1Xh5tkJaTN87ZW75T5btHQbe80q4K7DSavG1/Fe9bdrCxxIs3iXYYk0inX3P/ADG+SudVw39P5ZXkvanTrMxagkc32OkuOo9XrW3NjYkHHK5DY4HopYgWG0B/kH0AUevw+Z2Z/wAZTNGhae7O7agyXd2EUenMAQZDYRWrGWJSpyVWBCIMSWYAOxFbEAACnNgJLxu1tfj3pcaeGcSS8i5sGnfz+9F5DolBbfUanrYY0HkG2P8A9gOq3OPUNtc2ubDVr/yx+P8A+VV1eH/mH8JQlWjSdSkkjRoyFZGRUc2xLeAvKisrZKLvIrYgn4ioJEuaQ4QSCImRrF7IJwDXggyAQZiNQcr/AHwRukrY5dQV+WXjklB5TU48Yw6GGmJU2I+GFje199xwE5j2YctmHAZzx+I/VHteypig+JaTkR/TuGd/NSA2lUSkXpqO2wGVHrZHuuaCVLQXlEnu4zkCQQXYK8RXhS+pWPvn8VPhuNvvROmYeiL7A/C7jvHlvvZCNX7M08iBEhgQsoC1NPR6yxXkso2D0wEgcrLG7qk0gb4wCc0sp1qjXS508C6mBeeNtIFhuUamFpuAa1oBOuy7TkO8m/FRjr+npHIUWbmBRu/Lljs9yGUrMqyArbqV8/IggOKTy5skRzB5yEkq0wx2yDPIjlBQ8NxfKHhZgH0PHbqJhEdNjJhq1CufcozFViIVEqYAxkL+9RLuv/xjkXxzBjEjIPUMPYTvO/PZPLTXlfMqkJY8DcN2hGevdzXTvut1JptN06cLOefp9DKeXIgAMlNGSCHIxAPmtySfltjqrCyo5u4kdxWsFh62a5j9i0obP7U1eLGPkihEBuvi53M55FiBy8Mbi5bLaxXRuNUAdUG8dqeWXP7zQN6qT1s8Ij68kW7QfovlHkHWefcY+1rRCC2Qzy5LtNfDLGwtna+1+J0ziNr+JsbPDan52zUaow+z6m1tcYj7hNPgyUCQtcv7x+HHNuCpFhIWxTccW7UqothZ8i235HEovCqkRKMdk5bTxEslsty8zwpYg/HJGrSRKemSqfToTxRiNrq3DwG18jYojC7PWtJtfU7Oh1GX2NVLep9pARcVkN7Hca/WGx5UhUkPTE3DhSqra0gRG2kZlQCmfhP5Ld/atGXs+MfmOQbtnrjiH3VfDG0RZ35Os1c71F/iCI8Ua5FjkCuJYbHIqz8X0GDah7CZ302gDiTKoxLhszTeLf8AcJJ4RvlRDU1jMxdmZmcks7sWZmPUsWJZmJ3JJJPDkAAQEmdJMnNbKaPz4uaNUO86BKw/F5KHhODS6iD2adXNNzD9SJaaSSUnEfUyq4Sn3FrurEFj+q72Aqh5qNLZjWHADmNfvcj6LmtpPBI2jlLSScsiLC411uugXcE4k0nTQEhJjoqaP3hbK0SFdrAhlAFsdrG+/TjMYqk4VHO0k+K0mHrBzBvgeCop3T6WrROxSnY84qFn0ypq8zy1sFeBgEAyF49mVnRySJEAOxDiHAAnLR4ZruPj5IfCsaWkkA31YXabx4eadM3ZeNlaLlU4El1SoXRK9HUykjwF6kiFlvGyM+fULl4GJp60tO1JMXjrW+V+P6ol1Cm4FoaBNgerMj73/soxPd5WXbGirmVciH9mnXKNTs1ilxkLHG5Ivbe3DcYmlAl7fxBIzh6skBpPI34/ogDJ6jcbG/UEdQfMHggAG6okiyzTiwKoyimip76LzLSxgBQCTd16BvAT5AN4Sdjtfj1UgMceB8FGkCXsHEeIRPthoMr1M6x08rWdWKwwEYK6ApdIuYsQcXIAYqbEgngShUa2izacL7zx3mJRtem52IfsgmIyHAbpQV+w9Z/sNb/d5/8AL46cRS+NvePNdGHq/Ce4rWOxdX/sVb/d5/8AL456RT+JveFPqKnwnuPktyd39aemnagf6tLUH8IuJdfT+Md481EUKh909xRCLu8r7D/2zU/s9jqv8i/BDK1OLOHeEM/DVNr2Uqp+7LUG6aXqf9qkqVH7WhA/YeJmswC7h3qs4d40S0dy+pn/AFZW/fEw/G3EPSKe/wCR8l0Yep8PzHmrtfR+0+aLTKSOWKSOWFZFaOQWZcZpMCf6y4uPkRwsfDnGMimbAWtAOYA+7KnndbKnIxMtKC0ruVk1Sqo2GKrYukUXLcKEyEgk2At4CtiJiWnbkA5AWY1wzOpPHJHYR7diJEyT7RaflwCc36VhIAafTzgUVmGs6l48SjM4AUm4PUqCSwNgDEch+rfo135TFd1rNXN/Md5L1NVjydfa9OMZKFUOr6kpDA/EJBGJHDo4Uo4IiMYa/gZV71ZgHYdP+Jnhlz1nv8KzMg9sf5DPfx3edoq7Qa5Tuto6COJww98lRUSZKA2QIlcqc2IbMjMY2uQdndKnUbdz5G7ZA8PDJJKtVjrNYAZzkn77UB4KQqV6S4EsROIUSRli4JUKHGWYFiUAvcAglbgdeIVASxwGcHwU2We2d48Vcr6EtePateePFxbTygjPhewrdlJ8iQRc7XueM/iwRTpB0z62eeYT+iWufULTaR2ZKz1T2qksbUs56+a+QJBPi3U2CjDNrndRsWVXHulGbI3rfJMfU8MAAqiky1G/Xi0hcSfVKbMbHcdPn8j+duJU3bB4KqozaTZepI8Jvcev5/78MQAbhAkEGF8tVx3ZUUk1ZAwA99/uSQd9tyGBAPW99iOo86njLPkrGmFz17L6XpTQqair1BJyWzSCGN41XOyFchkxKWZgWWxOxboOPfiA6KTWkcTfx3qtjKGzNVzgeA7tD4rT2h0/TVhJgqK55wyACVEWJkP1vROZdSLrd1yVhdVKMGspOxBd/Ea0N4G/DVV1W0A3+G5xdxFuOg0TTHByDWQPEpUSs146ur0rx5cVtf8Aw9pCDrBuBZNP3PyNd6An9x+w8I+k/c/5f+KcYHIhWybWj4/fU/hFz4W8O/63vPXb9nCcOjUJpHAodJrH9NT/AHBjuLXHx/P5ncGxvxcKh+ILxbwK2QasnnIlwPFje1/O3UgX9SbDqeCQ8Rmq4K1U/aBDIUyF9iPsIHX7zt5H8bQA5u01Vl0GClGo6aHHzHRv+vy/PrxKnULDwXHsDgmlqYKbFsTvvjkD4Tby3G1x0uRjsW4Nc8Fu0Cgg0gwQs5oQ1r72N13OxsR5fIniwtBzUQSFRjslSwcmLKLs8zctSZKutZJLsb2mQOMZRcocUuALXuq8KahqbRg1OTZHI7kWwUy0S2mbam/MQb70SSCEp9V2VAuwu1TJmpZ3A3SQs6mwZWxssdrWtdehz9TV7uzTxXC2nnFLv+sKKdX0jlOY+dBLiF97TPnGcgDZWxXIr0NhYNceR4d037Y2oI4EQe5Jns2DsyDxBkJIBxaq1tVOOrycHYmdBOmYpcWDKzVqGSFLqbM6hlLG4Ci7WBbcEDimuCaZ2Zn+kwT8ip0iBUG1EHPaEgccwrQfQNPj1ogqN6PEwr4bZ1tjEp6JY3RTey2G9uFHSHs088jnc+7nx3plgvaeOPLM5cNytDLWmx97McTZvc9Rff8AUuNgdx5HoSycKQ7ie5N44fNJ4S7XPNf08UaqR06XUbHz2Iv9mxTGk6/JQNtF7T00l/rj1GwVPXp0Ox6fxuTbsneomNyb+u0T8w3nc7C3hT08/Dv99/x4OoNOxmgqxG1kiGk6yRZWN/Rz/wA35247Uo6tXadbRyJahTKy7gn5r1F/T1+ze48jtwM15YZCJcwPEFNDWI2Rb3Vb4kNIDjYkXDAEFTa69bISCcgLMeKgc2R80CaZaYK//9k="
menu_pescepovero = "/9j/4AAQSkZJRgABAQAAAQABAAD/4QAqRXhpZgAASUkqAAgAAAABADEBAgAHAAAAGgAAAAAAAABHb29nbGUAAP/bAIQAAwICCwgIDQgICggICgoICAgKCAoICAoKCA0ICAgICAoICggICAgICAgICgoICggICAgKCQkICAsOCggOCAgJCAEDBAQGBQYKBgYKDw0KDg8NEA4QDw8QDQ0NDw0PDQ0PDw0NDQ8PDQ0ODQ0NDQ8NDQ0NDQ0NDQ0NDQ0NDQ0NDQ0N/8AAEQgAcABwAwERAAIRAQMRAf/EABsAAAIDAQEBAAAAAAAAAAAAAAQFAQIDBgAI/8QANhAAAgEDAwMCBQIFAwUBAAAAAQIRAxIhAAQxBSJBE1EGFCMyYUKRJDNScYFiofA0Q8HR8RX/xAAZAQADAQEBAAAAAAAAAAAAAAABAgMABAX/xAA0EQACAgAEAwYEBQQDAAAAAAAAAQIRAxIhMUFRoRMiUmFxkRQygeEEYrHB0SNCQ/AzcvH/2gAMAwEAAhEDEQA/APqvqPVhSUMy1HlrYRQxGCZMlQFEZM+R+Y8J6I9KEHN1aXqR0j4jFZrRTrrAmXp2r+9xmfETOf6WtRO+A88LKrtMcHTkCfT0DWe9LRNZYUhrGLCgPbWNZIoDQoxIoaUBBpaNGJB0yRi1+nymskNplExF2nQGc/v6lQD6PpkyJ9QtEeYtzd7eP31oZb7115AlfACapu5gfKctLH1uOy2FDGT990uI7IB7tU/occ3QT+p5BFV9zb2/K3W0+fVgtn1eDIUG0IMk9xJGBpV2N630C8/CgraPWFMX+gal2Qt4QL+C1zFvOQB48SUl2ebS66hWatasYbSo1ovtDQLgpJUHzaSASPYkCeYHGoyq9Nh1dahKnQMXnWRirNrGAOr7wohYMiAAyzKxA4AMLmBmf8cZ0rVKykEm6YhX4k8/M0YgifTYgGYEkAYP5OTxPGlT8zoeH+V+5duuEEKa9G4tFppvJwTAAtN/ErzII7TwU2uIvZp65epXe9XZTmvSWbo+i54NoOPukgzBieLgDp23xfQ0YJrSPUvW64biRXQKWZFT0HmTFndGYJHiGJgHwGzPe+hlBV8vU6LayFAYgtGSBAJ8kCTA/EmPc866EjkdXpsc18RhLFNWtU26hxDJU9O4lWAVjBlYJNvEgH9Ig4Dlm7sb+lk8Sq1dCakdvYx+drwLCzHc5p9wC8r2h2XIYGRcPtLDXVPtbS7NX/13I9yn3upnRG2qm5N7XaEuldy8WpNxkC0gs105JlQvaFXTVjQVPDS+gq7OWqk/clN7tGa0byozMqIFXdOxg2WWgSbjaO8yxF8nL6OTHSt4ar0NeHdZuoTvBtrvSbc1VamGlRuKgbtWWuIHdaFZokwWf+wlDtqzKCp+SC1h7ZtfU90hdrTqqi7ncO6sEWm+5r1LiIUB7puIIyWOGDSfuGjidtKLbgqfGkvYEFhppJv3Z0+5qp6oBqlWuSKQqxcbXZRbyblBYr+oLJm3HDHNl0WnOjqdXuM79IkNZRtPQATf7RmEU3FMzN1itPOIbwSZkZnzzIcb20HjJLdWAp0qpBmuST7UaYj/AGz/AJx+NDs3zHeJF7R6s1OxqR/OM5z6aH+qPbiR/exeO6XyuqsGeN/KMl/fHOM//dVSJFwdEwg6xuWVAadH5gzFl1NYBBlpqYjwQJJBODwZ4MU33pUgTbS0VgFDeVCIOyCDsP8ANoFZvA8CfpqTUm3xCyTi0oQX+ToyacvD1RVeo1roGyUCfuO4o5yckBCfY+TM+0k1hv8AyP2YLnfyjnaVWNMs1FVYNimHQyJHddAA8mOYH5gSdZqUtOf2GV1bQRs0uWWpimSTKmwnDGCSsgz93JicwZGkap0naGWu6CUojmB/eBpdQ0SUEzAn3gT++gY0B0UgkFtMYqammEBeob2xC4V3gfYglm/AHn/wM6DdIaKt0KNr8TMzhPl9ykmLmVAo/JIcwB+QCfAMjSqTvYu8NJXmQ9u1UiaFtBmBaTY1NbDFrtLVmEu/6a5Y1BXakvaSAFtAX+q4kR90xaDd3B7VK3jNRVZbItO7zUZDpdRmLLuWH3cJTOWi1bWVlCU1CxHc1zkkFgdOpxS1h+ouVvaX6BFIOzEDcTCqsenSMN29xgDLWObcAXn+hda0lrDqwq/EMOj0iqkGqa3cWDEICA2QvYAIBmDHGPGkm7eir/fMaPrYbfqQ5VamiYD6olRl+iyIZ5dC4j2gMv7yf7aAklJruugFtrXk/VpRBgekZ+2ASbwJu7jiIFoGbtEnlne6B22e5tAFajdJk/LmIgQAPVOQQfP6v9Iuxsk+a9jejtK3msp94pKB7SJJYEZP3EExiOcNlnz6A7bPdeK9EZGPlzgZx/NMngTjGfEEi5cTmvYdbIMEAqMHbMsFtByYgSYgQOTxoJlYp1qBbrqVgHbUe5gvYt0TMs3AVFAkkn2ABJALQjm0tL1DJ0LE+Mef4bei2MGiuSSBC21CGIkkkG0BTn7Qa/D/AJ46+ZN4v5WK6m0oGrnY1C10Gr6FKPqOCxu9S4i4lmMYsk4CzZSmo/8AL9LfAnUb+ToHj4V27MU+XWAJzT+mxYWSP0l7ZQtFwXEwwmPbYiSeb+efsP2cHpX8BC/Bm38UaYHBAEBhyAwH3AGGAPDKrcgHQ+IxPEw9lDkMOndIp0STSRUuiYHMEkT/AJY/vqU8SU/mdjRio7IYLUnSWObDTGAupdWWkAXu7jaAqOxP+EDGPcnA1hJSUdwSt1amygNf3lYWypJ74Ewsr3DzEDJgGdAGaLRonSUBUgHsi3ufEAjILQ3J+6c55zojZUZnpCD9PDK/3NymFPPj24/GsDKjU9NQkEjKsXGTyTJPOc5gyMD2GsGkFFtYYEpPoDMWdSp1Qxsq0qYYoFupFrcEWx6qBy7EEHtItIh7hZSLjWsW/qSalzAd1Q3IsC16H6rmake83kkqoqyQtKRbfAZZOPtspYTu4v326cybU9KaCdlsaygB66uLkM+jDFVBuE+oZLmwliDADiO9Sk5Tw3tHr9hlGS3ZlttvuUi6tQKDvctTe4ZUsoc1bQsX95pi0WwsDtMp4T2i74a/tQqU1uwjcbPc33JV24XuARqFQ4PEkbhZcHF0RE9oukBSwkqlF36/YLU7tNe33G+xqkrlldhhmUQJHIAucrHFpYkeSedRlvsUTsJFTQGA+qbiqFHoLSZrhd6jsgA8xYjkt7AwP3xrJzcv7UvqRtatTHqCmOyWKsxhrsABlErbm4kGcWgZ0ArNxC7tFDlZ1jEepGjZir1dCw0BfMhRLMqjiSQBnjnE61pLUZKxDQ+HNsxi4VDAWDXJJIkliFcEu90MT+kAAKBBv8XLZP8A3+CLwEtWhoPh6ncr2ksihFN79oXgRdb/AHxnEzA0nazppPcGSO4K3wVRItKuREQa1fMwM/Uk8eZgyf1NLr8RiLj0QOyiE7r4Rp1cutR+xU/m1gIWQMI6rOTJABPmYGpx/EShon+hnhplqnwehUJFYKrMwAq1h90EglXDEBgHEsSrcEAkaPxEs2bS/RG7ONUG9L6MtBbKalVktEucnky5JzzzEyfJ0k8Rzdyeo8YqKpBd2ptjCzrOxFS0NUq0smPTqWFjHkxJAyQOJ98aFk5xT3f7AVfoS2getuVALG4bh5N0YLNJIFvaJgS3udFaiPDXN+5hR6IlpI3G5Iwpb5kkCWVhHgGYHvDRw0EgyR5v3L7npSFiTWriREDcuOF8AHkDPnwdYOSLe/UY7Xp1gWHqtYGHdULX3ZmpOXI8TgTgDRKqFVqe3m4gaKRQA6vUApy1JtxkfTCqxyD3QxAgCZ8+ACSNNGCno2l6gcnFWugBQ6gAfUXZ1lYKc+nRD8RaO/g/bN0ecjJd/h4J6TiI8aTWqY0o9fJUsaNdYiBapZpDnCq5Mi0A+JdMmWtDw0mkpJi5nxTGoOpDivq3WFpfcJm48E2hEvZmADG0ARIU9zKOWE+VjtKeo+ZRWpwK7b1HmnuKqrUU0meKhRaldDUpMrVo9N7KwWkqrarWAgMVDec0n/c/u/U5qTdpv/0fdPq9wq16ly3ValMn1FTsKkurCaaUgtNvSpklmpPVJeoKjDT7aykyi5yfodBsPjGm1RaVNkLPcQFYRC00qTwsyjoVtmVJbhGhu0TdJl1OLdIt8QbxAB6lJ68loC0hUtiCSQcDwB5njzr3I7I58SuKsU7jqVO0A7SsQCSF+VRghB5wSoJLMQQZ+84nufUnmWzi/Y9teoUypT5WqqkiVO1AUk5JKiZi0SY5C8mNZGUltlfsX6fu6buLdtVQ/cKjbYIBAJBnDA4iImSBA8bUMXG/l6Dfbb64EgOsGO5Ss4BkTyM8jEz7HW1OhOxR1XeeNNqgh9am7IPScU27e4oHwORBIyffMex1WLSfeV9BWm1oC/8A51eP+oEyDPoU8QWkR/qBAnxbPmNUc8PwdRcsvF0CqW3qqI9VWN0y1IYWUwLGpiYDiSDllMQhV5XBv5eouWS4jJquplBV1PpNOqZqUqdU2lQWVSQGi5QWBgNAkAwYEzA1xYmBnlmNUXurAK/Q0IIFGJ/UhpqwIUorK1wKuikhGGVHEam/wr8hu7yNG6SjFS1AEohpoTYbFIAZFN5IRgACowYEgwNL8I7vQNx5B+22qqSwpqpLXlu2SxWwsTk3W9szMY1vhXfA1q7SPdW3NQL9FVYzwzQPb28TcePt5zB9GKpEZuVaGC1q0GVoTmIqVCPtaJ+kJ77eAIUt5UXFATl5EepWj7aE5gX1AOVtk+mxwLiTb3EKIWSQQd7yPdO3FW4+qtNQIgq5a7AkZVSIP6jE5EYDMAxzX3g/cVsaJY5Le1pOgxRm3WSqCxbySoi4CAeWJPhfYAk+3nVHpsMhefi+qBPy1Q5iPVoyBBMmXA5gRPk+ACZ3LwlKjzLP8WVYB+XcyJP1KeMnBzzEH2Ex4OtryFqPM33HxM68UnfuC9pp58T3OuLsZjHdxOi75CacyKfxKx/7NVZBJk0u32kiqQbsgWzEZCysi3yNpzM6vxPUxbRdp570FniG7jwfKyIgzrW+QVXMb7XqE+/7H/aQJ0wgYtTQCKPiTdBbQdwdrJYyBSN9sEg+qrAR+IJnzGMRxH50BqrMoC7twaiwjenQuYjvZlDJ3MFxAEBQZBOdGhabXzPoGL0yrA/iHJDXXFKeRAFpCBQRyfeT+ANBobJLxF6uzclfrN2mT2KL/YNEdo9hE4mdGh8rdamO729S0j1ckg3emuABlQJiCYMmSMjM4DQafM5yhtXU91Q1BJOUUc8AQcAeAB/7K0CKa3dmu8vNMCnUWiZHeUDwPYBiBJMDJ4nzGuuFXqrFldaOgbbiouH3KMSViaVJeJuEBs3GB+ADGWlaPK9o0DXmD/LV8/xVMHvidun4tkeoMr58GT+IbNh+B+4lS8RruaNVrQm5VSFAb6NNi5BksJcWyMQAQM/4CcFblDrVBak9pEU9nXEzuASRC/w6C3GD9xkzHmCsiAYcByw/D1NllzL7Ta1gwL7hWW6So26rI/pBvYgfnJ/OllPDa0j1ClLi+g56ezAm9lbulYW2BAwe5rjM93biBGJMHXBDq+J0NF8aUcx3+/KDCVKmCYQAnBAjJUSZxngMcRoUBuuAto/EBJA+X3KDOSlMAYkkxUJH7ZOBOiTWI/CzbZ9fuEmnWUi6R6bH7ZwCB3ExiOSQPOsMp3wZA+IBaC1OupP6fSdiO4qASikAnmJ4OhYFPS2mB9c+ILRAp13+3K0mI7oOD5tBzE5BHONaxpTrg/YV0d1cJAYc8qVODHDAHPIxkQdAZOzLq9vpD1KbVxI7FS8zByRIEDOSYGPxrqw7zd1157E57aqxLskooCq7SskQf5PNvEWu0mQAPERPaCR1SeI98RP6kVlX9vQvT6PQqP6Z2jgAEB3pKEABJEG8nkkrAwT48B4mJGOZYn0vU2WDdZRl0ro1Fe6nSFMhiJKFTNtpInJBU23CZEicajiYk2qlK/rY8Yx4IO2+0WmLUAUSTA/Jk/udRk3J2yiVbF7tKE3oPnQZh9QbQGBurIDBNVqEXcNSAaQJn1FcG0CQRESdYWaXOilGBaPWJtJBF1MmoYIAbBYFT/QVyDM8awFWmoxU/wDPH/P8aBQ8z6IRL1fcaQAnnQsxtuN6tJLqjBVwJP54/MnVG6CouTpFtru1qC5DcM5HBjBj31lKxXFx0Z6nvVYkKVYiJAIJE8T7T4940bs1VuWaroimbVdYxS/WMb0DnQZjoKDaAwv69abb6B3GTEU0ezKyTeQFnnHNp/GgJOtO7f7Fdl0ykWDfLqjQKlxppgk4hhP1O0MY47czjWAox8I12u2VFtQBQJhQAAJMnAxkmdEsklsWrNoBZzHV6udJJigF2pmP/9k="
menu_delporto = "/9j/4AAQSkZJRgABAQAAAQABAAD/4QAqRXhpZgAASUkqAAgAAAABADEBAgAHAAAAGgAAAAAAAABHb29nbGUAAP/bAIQAAwICCwgIDQgICggICgoICAgKCAoICAoKCA0ICAgICAoICggICAgICAgICgoICggICAgKCQkICAsOCggOCAgJCAEDBAQGBQYKBgYKDw0KDg8NEA4QDw8QDQ0NDw0PDQ0PDw0NDQ8PDQ0ODQ0NDQ8NDQ0NDQ0NDQ0NDQ0NDQ0NDQ0N/8AAEQgAcABwAwERAAIRAQMRAf/EABsAAAIDAQEBAAAAAAAAAAAAAAQFAQIDBgAI/8QANhAAAgEDAwMCBQIFAwUBAAAAAQIRAxIhAAQxBSJBE1EGFCMyYUKRJDNScYFiofA0Q8HR8RX/xAAZAQADAQEBAAAAAAAAAAAAAAABAgMABAX/xAA0EQACAgAEAwYEBQQDAAAAAAAAAQIRAxIhMUFRoRMiUmFxkRQygeEEYrHB0SNCQ/AzcvH/2gAMAwEAAhEDEQA/APqvqPVhSUMy1HlrYRQxGCZMlQFEZM+R+Y8J6I9KEHN1aXqR0j4jFZrRTrrAmXp2r+9xmfETOf6WtRO+A88LKrtMcHTkCfT0DWe9LRNZYUhrGLCgPbWNZIoDQoxIoaUBBpaNGJB0yRi1+nymskNplExF2nQGc/v6lQD6PpkyJ9QtEeYtzd7eP31oZb7115AlfACapu5gfKctLH1uOy2FDGT990uI7IB7tU/occ3QT+p5BFV9zb2/K3W0+fVgtn1eDIUG0IMk9xJGBpV2N630C8/CgraPWFMX+gal2Qt4QL+C1zFvOQB48SUl2ebS66hWatasYbSo1ovtDQLgpJUHzaSASPYkCeYHGoyq9Nh1dahKnQMXnWRirNrGAOr7wohYMiAAyzKxA4AMLmBmf8cZ0rVKykEm6YhX4k8/M0YgifTYgGYEkAYP5OTxPGlT8zoeH+V+5duuEEKa9G4tFppvJwTAAtN/ErzII7TwU2uIvZp65epXe9XZTmvSWbo+i54NoOPukgzBieLgDp23xfQ0YJrSPUvW64biRXQKWZFT0HmTFndGYJHiGJgHwGzPe+hlBV8vU6LayFAYgtGSBAJ8kCTA/EmPc866EjkdXpsc18RhLFNWtU26hxDJU9O4lWAVjBlYJNvEgH9Ig4Dlm7sb+lk8Sq1dCakdvYx+drwLCzHc5p9wC8r2h2XIYGRcPtLDXVPtbS7NX/13I9yn3upnRG2qm5N7XaEuldy8WpNxkC0gs105JlQvaFXTVjQVPDS+gq7OWqk/clN7tGa0byozMqIFXdOxg2WWgSbjaO8yxF8nL6OTHSt4ar0NeHdZuoTvBtrvSbc1VamGlRuKgbtWWuIHdaFZokwWf+wlDtqzKCp+SC1h7ZtfU90hdrTqqi7ncO6sEWm+5r1LiIUB7puIIyWOGDSfuGjidtKLbgqfGkvYEFhppJv3Z0+5qp6oBqlWuSKQqxcbXZRbyblBYr+oLJm3HDHNl0WnOjqdXuM79IkNZRtPQATf7RmEU3FMzN1itPOIbwSZkZnzzIcb20HjJLdWAp0qpBmuST7UaYj/AGz/AJx+NDs3zHeJF7R6s1OxqR/OM5z6aH+qPbiR/exeO6XyuqsGeN/KMl/fHOM//dVSJFwdEwg6xuWVAadH5gzFl1NYBBlpqYjwQJJBODwZ4MU33pUgTbS0VgFDeVCIOyCDsP8ANoFZvA8CfpqTUm3xCyTi0oQX+ToyacvD1RVeo1roGyUCfuO4o5yckBCfY+TM+0k1hv8AyP2YLnfyjnaVWNMs1FVYNimHQyJHddAA8mOYH5gSdZqUtOf2GV1bQRs0uWWpimSTKmwnDGCSsgz93JicwZGkap0naGWu6CUojmB/eBpdQ0SUEzAn3gT++gY0B0UgkFtMYqammEBeob2xC4V3gfYglm/AHn/wM6DdIaKt0KNr8TMzhPl9ykmLmVAo/JIcwB+QCfAMjSqTvYu8NJXmQ9u1UiaFtBmBaTY1NbDFrtLVmEu/6a5Y1BXakvaSAFtAX+q4kR90xaDd3B7VK3jNRVZbItO7zUZDpdRmLLuWH3cJTOWi1bWVlCU1CxHc1zkkFgdOpxS1h+ouVvaX6BFIOzEDcTCqsenSMN29xgDLWObcAXn+hda0lrDqwq/EMOj0iqkGqa3cWDEICA2QvYAIBmDHGPGkm7eir/fMaPrYbfqQ5VamiYD6olRl+iyIZ5dC4j2gMv7yf7aAklJruugFtrXk/VpRBgekZ+2ASbwJu7jiIFoGbtEnlne6B22e5tAFajdJk/LmIgQAPVOQQfP6v9Iuxsk+a9jejtK3msp94pKB7SJJYEZP3EExiOcNlnz6A7bPdeK9EZGPlzgZx/NMngTjGfEEi5cTmvYdbIMEAqMHbMsFtByYgSYgQOTxoJlYp1qBbrqVgHbUe5gvYt0TMs3AVFAkkn2ABJALQjm0tL1DJ0LE+Mef4bei2MGiuSSBC21CGIkkkG0BTn7Qa/D/AJ46+ZN4v5WK6m0oGrnY1C10Gr6FKPqOCxu9S4i4lmMYsk4CzZSmo/8AL9LfAnUb+ToHj4V27MU+XWAJzT+mxYWSP0l7ZQtFwXEwwmPbYiSeb+efsP2cHpX8BC/Bm38UaYHBAEBhyAwH3AGGAPDKrcgHQ+IxPEw9lDkMOndIp0STSRUuiYHMEkT/AJY/vqU8SU/mdjRio7IYLUnSWObDTGAupdWWkAXu7jaAqOxP+EDGPcnA1hJSUdwSt1amygNf3lYWypJ74Ewsr3DzEDJgGdAGaLRonSUBUgHsi3ufEAjILQ3J+6c55zojZUZnpCD9PDK/3NymFPPj24/GsDKjU9NQkEjKsXGTyTJPOc5gyMD2GsGkFFtYYEpPoDMWdSp1Qxsq0qYYoFupFrcEWx6qBy7EEHtItIh7hZSLjWsW/qSalzAd1Q3IsC16H6rmake83kkqoqyQtKRbfAZZOPtspYTu4v326cybU9KaCdlsaygB66uLkM+jDFVBuE+oZLmwliDADiO9Sk5Tw3tHr9hlGS3ZlttvuUi6tQKDvctTe4ZUsoc1bQsX95pi0WwsDtMp4T2i74a/tQqU1uwjcbPc33JV24XuARqFQ4PEkbhZcHF0RE9oukBSwkqlF36/YLU7tNe33G+xqkrlldhhmUQJHIAucrHFpYkeSedRlvsUTsJFTQGA+qbiqFHoLSZrhd6jsgA8xYjkt7AwP3xrJzcv7UvqRtatTHqCmOyWKsxhrsABlErbm4kGcWgZ0ArNxC7tFDlZ1jEepGjZir1dCw0BfMhRLMqjiSQBnjnE61pLUZKxDQ+HNsxi4VDAWDXJJIkliFcEu90MT+kAAKBBv8XLZP8A3+CLwEtWhoPh6ncr2ksihFN79oXgRdb/AHxnEzA0nazppPcGSO4K3wVRItKuREQa1fMwM/Uk8eZgyf1NLr8RiLj0QOyiE7r4Rp1cutR+xU/m1gIWQMI6rOTJABPmYGpx/EShon+hnhplqnwehUJFYKrMwAq1h90EglXDEBgHEsSrcEAkaPxEs2bS/RG7ONUG9L6MtBbKalVktEucnky5JzzzEyfJ0k8Rzdyeo8YqKpBd2ptjCzrOxFS0NUq0smPTqWFjHkxJAyQOJ98aFk5xT3f7AVfoS2getuVALG4bh5N0YLNJIFvaJgS3udFaiPDXN+5hR6IlpI3G5Iwpb5kkCWVhHgGYHvDRw0EgyR5v3L7npSFiTWriREDcuOF8AHkDPnwdYOSLe/UY7Xp1gWHqtYGHdULX3ZmpOXI8TgTgDRKqFVqe3m4gaKRQA6vUApy1JtxkfTCqxyD3QxAgCZ8+ACSNNGCno2l6gcnFWugBQ6gAfUXZ1lYKc+nRD8RaO/g/bN0ecjJd/h4J6TiI8aTWqY0o9fJUsaNdYiBapZpDnCq5Mi0A+JdMmWtDw0mkpJi5nxTGoOpDivq3WFpfcJm48E2hEvZmADG0ARIU9zKOWE+VjtKeo+ZRWpwK7b1HmnuKqrUU0meKhRaldDUpMrVo9N7KwWkqrarWAgMVDec0n/c/u/U5qTdpv/0fdPq9wq16ly3ValMn1FTsKkurCaaUgtNvSpklmpPVJeoKjDT7aykyi5yfodBsPjGm1RaVNkLPcQFYRC00qTwsyjoVtmVJbhGhu0TdJl1OLdIt8QbxAB6lJ68loC0hUtiCSQcDwB5njzr3I7I58SuKsU7jqVO0A7SsQCSF+VRghB5wSoJLMQQZ+84nufUnmWzi/Y9teoUypT5WqqkiVO1AUk5JKiZi0SY5C8mNZGUltlfsX6fu6buLdtVQ/cKjbYIBAJBnDA4iImSBA8bUMXG/l6Dfbb64EgOsGO5Ss4BkTyM8jEz7HW1OhOxR1XeeNNqgh9am7IPScU27e4oHwORBIyffMex1WLSfeV9BWm1oC/8A51eP+oEyDPoU8QWkR/qBAnxbPmNUc8PwdRcsvF0CqW3qqI9VWN0y1IYWUwLGpiYDiSDllMQhV5XBv5eouWS4jJquplBV1PpNOqZqUqdU2lQWVSQGi5QWBgNAkAwYEzA1xYmBnlmNUXurAK/Q0IIFGJ/UhpqwIUorK1wKuikhGGVHEam/wr8hu7yNG6SjFS1AEohpoTYbFIAZFN5IRgACowYEgwNL8I7vQNx5B+22qqSwpqpLXlu2SxWwsTk3W9szMY1vhXfA1q7SPdW3NQL9FVYzwzQPb28TcePt5zB9GKpEZuVaGC1q0GVoTmIqVCPtaJ+kJ77eAIUt5UXFATl5EepWj7aE5gX1AOVtk+mxwLiTb3EKIWSQQd7yPdO3FW4+qtNQIgq5a7AkZVSIP6jE5EYDMAxzX3g/cVsaJY5Le1pOgxRm3WSqCxbySoi4CAeWJPhfYAk+3nVHpsMhefi+qBPy1Q5iPVoyBBMmXA5gRPk+ACZ3LwlKjzLP8WVYB+XcyJP1KeMnBzzEH2Ex4OtryFqPM33HxM68UnfuC9pp58T3OuLsZjHdxOi75CacyKfxKx/7NVZBJk0u32kiqQbsgWzEZCysi3yNpzM6vxPUxbRdp570FniG7jwfKyIgzrW+QVXMb7XqE+/7H/aQJ0wgYtTQCKPiTdBbQdwdrJYyBSN9sEg+qrAR+IJnzGMRxH50BqrMoC7twaiwjenQuYjvZlDJ3MFxAEBQZBOdGhabXzPoGL0yrA/iHJDXXFKeRAFpCBQRyfeT+ANBobJLxF6uzclfrN2mT2KL/YNEdo9hE4mdGh8rdamO729S0j1ckg3emuABlQJiCYMmSMjM4DQafM5yhtXU91Q1BJOUUc8AQcAeAB/7K0CKa3dmu8vNMCnUWiZHeUDwPYBiBJMDJ4nzGuuFXqrFldaOgbbiouH3KMSViaVJeJuEBs3GB+ADGWlaPK9o0DXmD/LV8/xVMHvidun4tkeoMr58GT+IbNh+B+4lS8RruaNVrQm5VSFAb6NNi5BksJcWyMQAQM/4CcFblDrVBak9pEU9nXEzuASRC/w6C3GD9xkzHmCsiAYcByw/D1NllzL7Ta1gwL7hWW6So26rI/pBvYgfnJ/OllPDa0j1ClLi+g56ezAm9lbulYW2BAwe5rjM93biBGJMHXBDq+J0NF8aUcx3+/KDCVKmCYQAnBAjJUSZxngMcRoUBuuAto/EBJA+X3KDOSlMAYkkxUJH7ZOBOiTWI/CzbZ9fuEmnWUi6R6bH7ZwCB3ExiOSQPOsMp3wZA+IBaC1OupP6fSdiO4qASikAnmJ4OhYFPS2mB9c+ILRAp13+3K0mI7oOD5tBzE5BHONaxpTrg/YV0d1cJAYc8qVODHDAHPIxkQdAZOzLq9vpD1KbVxI7FS8zByRIEDOSYGPxrqw7zd1157E57aqxLskooCq7SskQf5PNvEWu0mQAPERPaCR1SeI98RP6kVlX9vQvT6PQqP6Z2jgAEB3pKEABJEG8nkkrAwT48B4mJGOZYn0vU2WDdZRl0ro1Fe6nSFMhiJKFTNtpInJBU23CZEicajiYk2qlK/rY8Yx4IO2+0WmLUAUSTA/Jk/udRk3J2yiVbF7tKE3oPnQZh9QbQGBurIDBNVqEXcNSAaQJn1FcG0CQRESdYWaXOilGBaPWJtJBF1MmoYIAbBYFT/QVyDM8awFWmoxU/wDPH/P8aBQ8z6IRL1fcaQAnnQsxtuN6tJLqjBVwJP54/MnVG6CouTpFtru1qC5DcM5HBjBj31lKxXFx0Z6nvVYkKVYiJAIJE8T7T4940bs1VuWaroimbVdYxS/WMb0DnQZjoKDaAwv69abb6B3GTEU0ezKyTeQFnnHNp/GgJOtO7f7Fdl0ykWDfLqjQKlxppgk4hhP1O0MY47czjWAox8I12u2VFtQBQJhQAAJMnAxkmdEsklsWrNoBZzHV6udJJigF2pmP/9k="

In [ ]:
from google.colab import userdata
import google.generativeai as genai


# --- Configuration ---
# Replace with your actual Gemini API Key.
# In a real environment, you'd load this securely (e.g., from environment variables).
# For this Canvas environment, the __api_key__ variable will be provided at runtime.
GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY') # Leave this empty, Canvas will inject the API key.
genai.configure(api_key=GOOGLE_API_KEY)
multimodal_ocr_model_name = 'gemini-1.5-pro-latest'

image = '/9j/4AAQSkZJRgABAQAAAQABAAD/4QBoRXhpZgAASUkqAAgAAAACADEBAgAHAAAAJgAAAGmHBAABAAAALgAAAAAAAABQaWNhc2EAAAIAAJAHAAQAAAAwMjIwA5ACABQAAABMAAAAAAAAADIwMjI6MDY6MjYgMTQ6MDY6NDEA/9sAhAADAgINDQoICg0NCwoKCwgICggKCAoLCwgICwoKCg0ICggNCggKCgoKCAgICgkKCAoICAoKCggKDQ0KCg0ICA0IAQMEBAYFBgoGBgoPDQsOEA8PDw8PEA8PEA0PDw0PDw0QDw0NDg8PDQ0PDg8PDg0PDQ0NDw8NDw4NDw0PDw0NDQ3/wAARCACGAIYDAREAAhEBAxEB/8QAHQAAAQQDAQEAAAAAAAAAAAAAAgEDBAgFBgcACf/EAD8QAAICAQIFAwEGAwUGBwEAAAECAxESBCEABRMiMQZBUTIHFCNCYXEIUoEWJEORoTNicrHh8BVTc5Ky0fEJ/8QAGgEAAwEBAQEAAAAAAAAAAAAAAAECAwQFBv/EADMRAAEDAgQEBQMDBAMAAAAAAAEAAhEDIRIxQVEEE2GhcYGRsfAiweEUQtEyUmLxBSOC/9oADAMBAAIRAxEAPwDsX2w+tE0nLtXqJLKogBVfqcyMsaRrZG7tIEyJAGVnYcfeua6oMLcz87L5GmAHCVRP7LJdJq5eYy6lNRJzHUTSScs0+lzI6iozqGYLgad0T8VcAseRCjKvc4SlXoU5pkYBZzjFpMm2eul1jUcyoSdcv9eQK3vQ6walHg55qNdHr01DDlunOkwZ0ZEYMOly4o4nmjaNWZsfwiFIs36ZJofXwTQWES4zMEE7ukQL+a46zGPgvsY00kx65dirgfYxyDr6DTy6uKOSYyTmtRAuSBZWRKV4wVySMMNtwwNm748JwDHEMNkhLmjEL/ldJXkybdibeKQUP2224lNTo4K/6bcKULzqOAIQBOGhYfU+t4F8yxgDUrpSchimobHCBzdJI2ahUYqSXUCy63njbvrC05btuqg8z+0rTosju5VI9O+okZoZgI4UUs8rfhbIFU93yCPIrhGo1tz4+SYpOJgfbNNwfaJp2llhEg6sefVQo4KdOKCWS7QC0h1cDkAk1Mu3mrFVpJbNx/APsQr5bwMUW/P4TGj+0XTSdMJIspk00OpjEKvIzwTh+hKoRGOEvTcKa8rWxItiswxBmRPkckix17ZGPNBy/wBewu8EasxaZNQ8FxSKsi6do1nILIFGDzIu5BYt2hsWxbarXEATedDpmpNMgEn3GqzxTjoWSaK8CEBA4EIRH+nChWCql/8A9A/U+Oh0unH+PrFL/wDBCrN/pKYz/wDh45eHb9c7An7fddk/STPwrhX2WaWAaKOXRHWNz+Iyyf3bTyyRQxNJg7U2nkhP93KRl7PfICN9h9DRbUAw1gBQduQJMWyIdmPRc7yGgOaASItrtlnFyVv+jjjlXT6+bWaqfn0Kxl9G8EadIrKQEaMaGMgwROzOeoe7I/AGVV5YHUWMApEmHSTP/omLxZQ7C76rYhtPsT4Sr3+hUf7po+oblOmgMxAq5GQGSh7d5O3tx4sAG2Sd9c1nb4aUJDLw4UIGbhgJIXN7eP28/wBOCELRZvs6S9QBDGY58fvCyyMyamtP0C0yUUZjpwImyyLqqWbRcMxSAJtn3tC35k5n5M+6HXfZ5mjxNFE8cmiXRSK80/fpxYaJtyTaM4L5Zte7b7J1IEEEaR5JipBEHWdM1Ol9G49VkjiDzSRT6ks8v4s0aRqrMRuVCwRIFIorGLB4oMFyAL3PWMvYKceUnLLJYfT/AGdBTCRFp7hleTT5PO4gchxnDmSEOE8y0gUAMoG1BAUgMgN9c/VUa06nY5LI8h9FqhhYxxhoI5Y9OyNITGk5VtStyOxKyvFGxBv6F/lXi2UgIMZZeeah1SQQDnn5ZLYG42WSHgTCHHgVYUK8CYEL54/x9epS2u0cPvDpHkx+TO5As+LrTUPiz/MOM+GGZ8Pz7rrP9AG5z+eKTRcxXlyQz8o1E+qeaHpcyeSBJYoHjETRxr/c4zGGLOTkzEqqW3gn3aTP1LeVxYwBsYf2kzYm5vlpqVyVnFkGmBFyc7ifFdA0Ohgh10Wr0eui1+s1snR1SjpOsP3iVWZ06ZJTKWRVVWBGB2YY78FWpUqUeTVplrRkYImBrIvnNkObynYhqT3gdsMeavkU2/79uPLCcpiR+NAoJQ3w1KThhC9wQhNamelLUTQulFsa9gPc/pwkLm2u5E7MXEmpBbUPKy9FygiMUipDj94UWJJQ5ktraLEJHasMHU7zJXS18WgJ7l3pZ1bqmTUSjKa4Hi7fxMGLJ+OHBjZHELPNJgJmQKQI8RrCDMnW3ztdBeDaBp8y9baLMegtCY41iYStSgmSVAtlQqkN+K5LNWe21Hc351pggR891nVIN/nstjnTjdYSo78UAmhvhJrx4SqUNcCtfMP+I/mA1HqCeN3WOIanQ6RpTssUVRCaRyTjUTyysSSqgIbrfhcOIZIvc27fZdBbIa0nQnzuf4XSfRvq19FqjyXQ6vRz6fVT5S6yWLJkaaFRK1xa5Y2MEKLjuovYj3Pr1abeIo/rK7XAtsACALHq0m5J1WN2P5eExHW2fj/tdG+wP0aeXcyi5eskGrGqME0k6IVZEhXVFIx+O69gS7GV9TcAUR5/FcR+sAqXaG6ZyTEnIdPRZAco4Rrn6fcz7aK3vqTmpRQRheQvqMFGAI6hskb49q+2TLex382YyVBslavJ60YEX933JAHWHjYAtvfc2VUtUBvWTCsR6KsA6o9N6zbyx0yghgB1t81B7cqojJkBITssjuJpXiPRBYOq9ovUrtgqvp3YtIX7jeAAKYhdhdNbHIDyC3jhjEkQOqGL1c1Nk+mU4bC37W2vK/byRsL2sgC2clGAaSo+j9VyZAmSAqoVZRbCislSP9Aa8Nx9KWRtW5QJQWiNUun9VS/iEPA6hyU2kLCMuqrYRO76tmFg2DdXRJvdGFuyfh9SyFkJaGhj1AqTEUW7yh6ZtgqkLbAWdwduCSiB1WY/tVH7FjvW0Uh3F34j9qP7caYlngKZT1CpNASbkAHoyBbJ2FlP6lvpHuR44rElhUuReNVKg6+FirBSFfbFiLA3F/8AuFre+N3RqiOkiytpEyU1rtPJigRgGFZl97Fb1SgZE/mxAG5xOy8Zua6AAVQLZuFGlaazXRI2rZ78DL8w8tZHcaBA3okyQ8bKgW9V8v8A056JfmXOdVFG6Aza3XzrLIS0YjWWR0O25VgURQPAYew46KNVvDhrzcCOl4/la1W45DcoA7Qux8h5jqNdpT6dg02lV9JCobWPM6rIui1USsygaJyDq3AYKXHY7bnHu9NzGcM8cdUeSHGcIGWJpOeK8bxtuk4sql7RnlpqPHT7Bdk/hF5UsE7cvbTwjVaVdZJPqosGzYyqigN0kc4wzrF3GwIyPFceZxr+c/nNP0mIB8PwuZv0ww5gZqzvOualSiiMyXkTRAxAGx33NtSmgcQb/fzzOi1idViE9TPZHQawTsrLYW+0kmhZWmKg2uQ2KnIAJ2TwjdLF6gc3lCARiUylj33S/wBilliReybWSALk7JYRoVG1XrJ1AJhABKhj14+3KtyBewvuNkAAt9Ivh4iNEwwb9l7T+qJckDQgXnf4yWCGQKMbOx6mJIY94FhQ4pydu6nA3QpeZ+qJReEQItgrNKmJNNiPrWjYViBdqWG2x4RcdkYG7prV+smAPbGDjJWU6UHCjpKaP55CUvLYJd7gcPEfhTwD4EE/q+Ufli2qz107h8bkYkmxeTYkb2GBE4jt3Rgb8Cncn9Tdv4phRwq5VKhBau+hkcRkKClmPtvjbU126hzf7VI/tDHt3pvVHIYmyQKN1uQQBd2ONcQ3U4XbJv8AtFGRfUjr/wBRfff5+Pbzvww5u6MDtlC5lz9cGMbws4FqGkUKd9979hdeAWFEruQi8RYhUGGbgpuXnZpMMJWunCOoxpbY1m3j4Bfcjci2Bj2urDRrkhn5vIPELN48Og9hd2T4a1Ffy2ayW5c923dU1jTm5fJj0H6CecamYR9XT6NI9RrxlGHWDqXIYxIyh3MMMoCqcvA9xx6/CVGsqYSbkEN8TkrDXGCCRNreB+5BVhvW0A1qxS8n0E+lWFpV1kunaPTknBGhiIi1QLqotmXA4nD540pDkPLeOqB0j6QZdrfNtjZc2PGwupGL+t7wrPfwrjSMs02mWXIQ6ZJ5dQXMk7OGd2JkkcsWcZub8sN7BA8SoKgdFSBsBFvRViY4lzV1j1DzGLIrJEX2UWY8wd+3aiMV6h7yQAcwLxkxwJGqpoJuCsPDrIAaGnIAAS+gD5BBB2PbjtkWKtkFu2UO5GyrC7dSIYIAgP3cbuVZeijOtH6pAb2/NYZ9je+9VaLBKDMT3Tet53D23p2I3Hdpx2BEONCjX0hFU4e9fQRwsQ2Rgdv3Ra7V6dcriXucofwVBkIGZ81kCdwT9TlassuVktGiQDjr3RDnEBqMxGrUBTCCgOeAHuBh522Cbg1wpGUIwuznus0ORxXYjjv56a3/APH9ONICxkpTytNuxNvp7F2/bba/04cBElONoVsnFbPk4jf332+d/wB+CAlJSPpB8D9qH6/p+p/zPyeKQmZeWoQQQtEEEFRRB8ggjwfccOOiJKgc30oVHfp9Qqt4IoLtXsNv6+522BNApxAEwtGmTEocVRTJgFONnpqCSPgdqkj4sAVvtw5AEpySYlQJPX8ShTkyh1Vh2MdiAaIVSQQG3sAXkLtHC5Oqs1WnLdovmh/DxiJXXUyz6bls6PFq3jWQQ6h1Q9CDNI3GZeQtjdhQQayB49VjP+ovpiagIwjbWY20UOdhuSA3Xwgz7DuuhH1f931/3PQa2SDlOokg688iwmMNMAmodJJtNsUiVURWJAdaok8aFvPomrXZNUZC89PpB3JPghsUgGM/pkmegFxO2cq6X8NnoiDT6SX7vL14pdSzCQlT/s0SPEFVUdhiN2LDE/08WpUdUMvEHbvqs2U2sENM/IW267VQF5S0pV7AkHVdKKV7WAMcl7gAKYNdMCcQ5slbYXQLJhdVAyhTK5svQaaQOcjRWgwLCxioN2fFljdBzTr3TLXbL2EDArm9AM7DrS2AcQS3feJGNDcUTQ3a3Ld0odt2TaTwXl1mJLZD+8PXezYgKHqiQyqAtUtDxww5u/dIh23ZJ04CRUzEkriE1Eh3ZqU0r7F27ciN9xfnhgt37o+rbsstreQqxZiZASoU4yOBQJI8N72b+QaNgLVloWYMJpfTi0y3IVK4kNK5FbbC2OPjytH9dzawjJBcUQ5GthrksGwTLIfe/BevbxVVt4PDwhGMps+mo/hvmhJIB+tAPQvya8nfzwYQljKaPpaPbZtjYPVksGiPPU+GI/rxWAJ4ypmi5MqXjYs2bZm/yyY0N/AocMCFJJOaDXapUVpGcIiqWZm2CgeT/wBBvwOcGiTkm0SYC9LzEAAllANgCTtJoEsKajYVWJHwpPgHhyM04OScXmfgBlNgMBs1g+GH6H2I24UAp3AXzp/h/wDs5n5ly6TSCaDT6bSa1pVLQs8080sbhgzDUIoWJWFARE0wF7We8cRT4Itqlpc5wymABPhmY3WtSnzW4SIyv79ltfof1zqtdE/IY44ETTKctSXlXqQ6LUxBmCiKTu1UmPZ2jB3Ia1AY4ii2gf1jjOL9sZFzTF508FJDS3lDMAZ9c/T3gHNXQ+xP0h925fpIDWSo7PheJaWR3YglVO5f3VT+g48tz8ZlQG4RCmt6f+t+olEsSxhiIXclgWrcDe7N2SSTZ4jDrKvFpCT+zePl0/lXKCEAEkCh2izQwA9vG/Dw9UYunui0vp45NUiHcZqIIrrbtNAUDjdUN/24MKRcmtR6XpiVaNQ2ICnTx9tD2IxJs91G63quGGXn7ID03pvSpDXlDeSk46ZVY4mwb6p3VyCp8KfYkgh4PD0TxzupC8vm2uZCRlf4FBvGJYdbyPfFlBvwOKg7qZbsslpFYCmIY2dwuIr4rJvHzf8A1YUnoni3DSQs3DUoVbikkpfgQmJ9OGBBFggggiwQfIIOxBHkHhxurFkxNyxDiGUMgNqCNhsRY/XFjuPk778QWgiEw6DKg6v0zCT9CsAKXIbAe1AmtvF1dcIUwcwqxEar5yP6Dd4OX6+PRyQ8uh0OlGrnX7sks/RZvv8AqmWPUdR2bE4syiRsRtvx7lCs2mx1B7gahJhtyBIEC4gX8lfEUXVXiHW6SI0gbLffUcen1Bjm5TpZoUjTUJq5IIpEGb9IwREI7I1AtIQVNdh2Db+e0VKct4wybEAkG15jOMllxBLm4qXyPt6q/vLNDhBHH5KQIgDE9xVAO4+dyNz53vjzVotTm9MMVpogezBsdS42xCbWmwCKtV3JTFTk8nUjB07rQPvM9kv9m2VsliunzH44AJ6hkuunQuRiTt+avA4WCMh3RinXsg1PpdgkYVWFyq8oWVbGAcILZTlkG7m7msA3kAeGWWhAfBURPTrXvHNsKFaiM7AAKfpBDj2cHJaBDWAeFyzt3VYhv2U3luneNrEc7HpkAyTRsCWKAg+4xEIcNfmZ+03SWGlpmCpd9WZC2pZdv+9uN1zJvqcUEJc+HhQhd+HCEgfhwlC8X4IRCi8w5cJI5YyWAdGUlDiwDCjifY0djuPkHiHjEMJ1VtdhIIQPp+xksrYcZJSspa7ZKXEMCbBxNHcg728MNwhE3lYduRydv47bKBsq2aJJJzzHdlRJUtSJTfX1M+W7+5ViGyoh9mfPZtXLLyc68xcrWHUpCSmj3gilUaZVfoIzNLalz1CWUtV3Y9jjKdOhTHFYP+wkE3dmRJtOn8LRtR73lkWFjnJsZ18FvH2V6+TR8wPKNPJHPp35ho+o5hHUZpI4TqLYTEEJpguJW6o2CLrhrRxTRXqAgxYaWPhOc6rnc51LDTHqepJ/nzV4fUeoqNiCwNpuilmPcuQAXu7hYLA2oJI3Ucee7JbjNalNzhx4mYDfHLTO1C6AYlCzFcSCSwZgScro8RJ37LSBsg5jzZjR62IARhcUi5FdnLYoAVdrBjIHgdvyEnfsgAbeyKHnkjVjLESsIMuSsAWC07oOmTtJvhZ9htvZicciEi0agpg+qWuxLpiT1LV81PxHj2g0rEZEjuHgiweDGcwQqwDYrJaD1PuepJBRD49JjeQIOO5IJEbqcR3HyBV1Tal7keSkstYFZSXnsY/Oni/rXxtv5uqIN+KIPg8bY2jVZ4DlCXUc4QEqXVSKyDMARYsefkGx8714NVibqVGB0TC8OaJsA6EkgAKwJN/ABJ/XxsNzsDxeNu6WE7JYdarXiytVXiwNXdXR2ujXzXDBBySIIzTpHFJIb4EJnmDNhJhj1MH6fUvDOjhnW9ZVf6cZ1MWE4c9JVNiROSYpghNW9MQGpbO+IYpkB7AsoPzR8cUJhBiVgtRz+XEEQM5/lzRD5bfuJA7QjFSbBkrfpsTjjfEhvdaYWzEqmX2nczm51DD9z5eVTT6j+8ydTSg5Mi0gzkidqidyyhGHcvvx6XDU2/8AH1Dz6mYtGI2nwVYubTJp2N/POMuoC6t/CzzjRSyaOKLSiLUQxSPNM0MSvLLCoimkLxu2ReSS1c2zK3kWRxxcQ2tzHPc6WkyLnW4z6LGWjCw3cAO0ZbROStLznWMMAgUksQQx3rFt1BdbxfHIX9N+9ccxJ0WrQNViH9XOKuCQEvitMTe12KTbLwASDd5YAGpxuGivAN0v9qGqUmGWlAZQqks9kDFR5LA2fAGIB7b4rGbyClhE2KDTeqWqjFKD+GdkcLTsin6ku06hdlANLG2+3CFQnQo5Y3C8vrAUD05xZoZRjzQO9uMQLAyahf7Hhc3oUCmdwmR6xj/MjAfiVarf4Z3sBu3PYoPLXuBXD5g1CfLO6zQ06mjipsCrUeCBXtttW3HS0Ahc8kJfuC/yr5J+keT5PjyTufk8VhGyUleTSKDYVQfkKAfBHx8Ej9jwQESUa6YC6AHziAP86H68AA0QSSiy4pJAy8CFB5zzYRpJKxpI43dyBZCoCWNDc7DiHODAXHIKmguMBNy86ATNjgt0S9du5AyokAH5J8HejdLGAJNk8JmAmdV6hQAEugBJALbbgkMDe4IZWBU0bU7dpo5jd0YCqC6D1G+j5mnLeX6520c08JlcjSTAvOIhK4kXTAFY45Y22YAYHfzx6hYOJoHiOJYA4B0ZjLK07yreeWGspiJgTnqd50aR5z1Vm/4afsbj0WodY5vvAXRmMFlXKMGVCuRXzl0z+nbfvx4zuKdXEkRf5mk6mA8v1P3XdefzxWiS49wZlzsKcSt77AEFlNE2QCRsrViS3VW0H9qxiw6YHYoCyk2JCNjiNyHoZdRQv81nG8TRiYqh6Zk0WnVC2VIXjspI1ZbYDtJ3pl2oUMSaCjF/QBM90vrJhS4OXxgOgZjezq8zMy3t4ZyVPf8AA3K+4WrAblPdRidn9kScjB8PLXsFlNC/ZfNV7Udv6DgDBufVGI7D0SnkZ2/Fm82O8fFfy72D73/oOHg6lGLoEj8rICjqyDuoXgcrql7kJPg1Xd3HfxVYepSxToEX/gzUv4kgxUqDYs2R9dqVYgCgxSwCffcMt0koBGwQtyphv1n/AMo9v2/Dr/MH/nawk/uKC4bJZuXmhUjg75HYlrrfxQrHbEAC22trDwnIFTiGyll+N1CBm4EITL54EKO2gUgLQxFUpAxFbihVCjuPg8LDZOTmoWs5HHtaIR7dor/Qf9/58Llt2VB7hkVRHX+q4dTyw8r03L5X5hHp4oXl+76Z6MTxjWSh0maXxli+Fkum4y47eS6nW/VVKgDHG0E63aIjwWzXts3NwvEZGN/G1uqst/B16aEUWuDI0UyywxSiRCrBVjyiXfyAJGegAAZDtueOLiKnMfiBt8lcXDsLGQ7PVds55ydnax09loCVMqvLI353tRV1WX6ccLmk7Lta4BQJfStteGmC3/5bZgZAj5GQA8nYtvW+04OgVY7ZlMD063cpSDAlmYDqKWY4bneztGL38qp/IvFCn0CMY6qLqvTzsWZo4SzbOetLuL8/SBVeFNefbHuMBJkgepQKgFpKd0/pn8Mr0owbjFLI1HFOnfgN2xu9DIkGjZbcPl/TBA9fL58KXMvMoouSEdT8D6+mXqYbmMnA7kADeiooEEg2KHEYSP25xqrxf5dk0/pqh/smu2/xY7+hACTR/lNVZU5n/FcFcu1m90Y/8uye0WiKMSEkNDEEyKVdQKViBW5AF9t+PqxFaNbhNgeyguBESFmdLqiwsqyb+GKk/wBMWPHUwzmIWDhGRlOHjRQvFOGhADwIUPm+hzjljtkzjdM4zTpkpGSH2ZbsH2I4Tm4gQqaYIKjT8ubpqiMIyHU5Knb9YZwFLf4gtd2IGV0aricBiAY/2qxCZIUaflMtKMwazv8AKTZBXfGQ9gtare7J24WF26eJuypJzT0zNyPU6Xozw6mTVpJGxm0zqUiQxGRhjrD3O2O5YjbwaPHYyqz/AJGmWuaWht7EXMGP2+KzqgcPiqZk2HiT7enirifYBoH+7z6mUqZdZqBqX6alVW4IUVQCTWKxb9xBYk3vQ82wAaMhI7lU2Tc6x7LbOa66cSHBA0fZjRUMdpOoWLNsGZosKQkdOQm80AwJfNhb5+Fu0Ni5+fPdNarm83bUQJzOYDr2qFXEHyRmST+YgAbGxTJdoEYW6lDqNfMFY9LJgHIRd2k/EkCqrZAJjGI2zZGzyPalGgvfEx5eaMDcpTPNfUkgzqBzRpKsl/NMQEFAbFlsHyATXFY3f2owN3To5+wIBik/MDit7qoaxYWlcEhLGTNSkIbHBzTqCjANwpUnqGiBhL4uwtgbv+t2RGSNvDJdF1HFGrGhSwdQmJfVC1GSsgzLgdotMSQc6b3olayy9tyATmiAYKk0ySbhCPVif7w81eIO2X+/8rt85J/OtrntG6oUjooer9eIFyAY9rMLGIIXPwSCLJjYAGjtdeLo126I5RyKzOi5srrakMLINefJ9vg1sfB41puDrhYOaW2KLUSkCwC242Wr3O53IGw3O90NrNA6qUMicNCi6/UBEd2bFEVncnwqqLYn9gL4lzgASclbbmAmvvmwYmgSFGfbZLYqN6+tiAv82Qq8haxWTi8I4ubpQbJaPg5Cj80bo1+/BiETKktKol9jnKOWDTu2sLff4jMVjlfWRvAmA6aFFZYray1shanFnZa6uLqcQHnkD6IFwAQd79kYWinhrm5OsTY75xads1dX7CeXlOWaAHydOshyayOrbgWfZQ4UDYAAAUOOGpGIxkilOEE5p2XlZMjkrqFvMsY3TFqZmTwl5EEoBfaojskra8eHofJdYNosk0GhrMj72CVcteNguEQYUFJYKqtjfZRxO3EgG+aokdEscbkSMW1SKCqqFyJCpGgJa3YsclZsrtsjkGZgQHe4TEdFGlY0wz1VFy3dG7hVJFrZYkpRPaGzGIxru6jvoT6FAA2Cnwkr02Mjsm2IMRGIUMvdZv6mQg4msaAGYrQDDBxGPBQbyAAp39rYx+etwNwR8G9wNqNk+BxpzmbqOW7ZSV52hruG4Feb3Fj2vcb+OK5rd1Jpu2RNz6PfvXbY77ivNjyKPng5jd1PLOych1asLUhh8qQR/oSPPGjSDkoMgrVftB9eJpkDUGkc1GgIUsAe5mNEhUv4NkgDySNGi6guWuaH7cUIBaJx/wALhv8AmqcaYCsTVAsQl5Z/EBpJJWhUyGVcso8RkMSA/wCbcKWAsH5+DXMKzC8sBEibeC6XMe1ocWmDHfL4VP1f2pafuQ5G1IZHVTakbhhn4I2N7EcaEg2t4KQdVq/rv+IfR6eEGRZZPxIsIY1USORIpyGcqAiI1IxZwDjW5ZQeLiOIZQZ9Q8AM+50Xdw3Dvrv+jzJyHjZLo/tg0pWKWFJWBj7GLY0r4lhTM5BJjQMCoIwUXSgcbUyxzQ9mqwq42OLHZg3Ve/7H6uHVyc41MMWM00MohimLuliMaeOzGhzUIins3KtsAdulvEUnUBwtMmRNyI1JOpWfEUjzm1M2gQPGfyr6QRUqj4AFft/TjlyV5oW4pJLY/fbfhIST64Kt2dt9gdhv8e/9DwKugULRa8MMh4O4sEbe2xF8aBSRBTksnFAwkvRtwFEJ6XU8QGpEqM2kU+wP7gf/AFw4GynEUUUY8Ch+3ALZJEyqc/xP6aU66eRZsBGsSKjTdM1gp7LYKbZj2kje/N7+dxragh7HxbKY/BXqcHVogFlRszrE/kLnPoz1hP8AeIoic0d41IkABHUYAEMBdgG9zWxFDbjj4fjawcA4yDv/ACjiqXDFpLBB3H8LvfNpkgWSdFQAgtNiFHUCA+SB9SgGt/m+O+q4U5qNjr1XmURj+h3ksHyqJdQ51M2nwRcW07zsY3axuWjDAhAoUAS7nu7KILYNw1HY3t8F0kEDC0rhP2x+roZmVY41iERZcwqq7BbGNDfAbkWfPtuePN4lzahAaIj56Lqol7QRKn+gfXpjXpzHFVVelkArV7DfFSFWqYAEg33bnjq4biCwYX5aKalLEZ1XQPsC1up5kIBqdSzrDqtHOFMMQzxdSVJiWIKMMlBKyfV47QePpOLp0+GqNFJsSDNz9yV5rXOrl82DXCAOhV5XPHGt0DcCE06cMJFR3k2/Q+3FKUiDikIWXgQkLcCF4cWEiEgPDIUBIZKPBCCqOfxLEjX6pw31SYMuCjYQpl3r3tkqgUTtQ+BXg8awB2Iew99fNdlEmIPuuTtoyKon6gFv2Wsm+aJ+Rx5msLUmVM0v2iTlYYeowjRg0VYhkK7rRVVsAi6bKyd7BN7l7g3DNgoLGt+oC6Z1v2iai5EaV3DUWDkkefy2dvjYDb9hU8xxFyVbWg3WF16A4mgGIy/Tcn/U1Z/X3NcRhlbNnUo9JzBGu1N/Irf99xxrEpGQv//Z'
model = genai.GenerativeModel(multimodal_ocr_model_name)
images = [image]
idx = 0
for mime_type, menu in encoded_images:
  print(f'--- Extracting text from item {idx}')
  try:
      # Initialize the Generative Model

      # Send a simple prompt
      image_part = {
          "mime_type": mime_type, # Assuming your Base64 was generated from a JPEG. Change if PNG etc.
          "data": menu
      }

      ocr_prompt = """Extract all text from this image. Provide the text as best as you can, including any structure you observe (e.g., in columns, sections). If the image is too blurry simply reply with text 'Menu cannot be detected'"""
      print(f"\nSending image to model for text extraction with prompt: '{ocr_prompt}'")

      # Send the prompt with both text and image
      response = model.generate_content([ocr_prompt, image_part])

      # Print the model's extracted text
      print("\n--- Extracted Text from Image ---")
      print(response.text)
      print("\nSUCCESS! Text extraction from Base64 image using Gemini multimodal model complete. 🎉")
      idx +=1
  except Exception as e:
      print(f"\nFAILED to make a call to the Gemini API: {e}")
      print("This usually means your API key is incorrect, invalid, or you haven't enabled the Gemini API for your project.")
      print("Double-check your API key and ensure the Generative Language API is enabled in your Google Cloud project.")



In [ ]:
import pandas as pd
from google.colab import files, drive
import io
from PIL import Image # For image handling
import matplotlib.pyplot as plt # For displaying images
import os # For file system operations

print("--- Method 1: Interactive File Upload during Runtime ---")
# This method prompts you to upload files directly from your computer
# It's useful if you want the user to select a file when the notebook runs.
try:
    uploaded = files.upload() # This will open a file selection dialog
    for filename in uploaded.keys():
        print(f'User uploaded file: {filename}')

        # Example: Reading a text file
        if filename.endswith('.txt'):
            content = uploaded[filename].decode('utf-8')
            print(f'\nContent of {filename}:\n{content[:200]}...') # Print first 200 chars
        # Example: Reading a CSV file into a Pandas DataFrame
        elif filename.endswith('.csv'):
            df = pd.read_csv(io.StringIO(uploaded[filename].decode('utf-8')))
            print(f'\nDataFrame from {filename} (first 5 rows):\n')
            print(df.head())
        # Example: Reading an image file
        elif filename.endswith(('.png', '.jpg', '.jpeg', '.gif')):
            image = Image.open(io.BytesIO(uploaded[filename]))
            print(f'\nImage {filename} loaded. Size: {image.size}, Format: {image.format}')
            plt.imshow(image)
            plt.title(f'Uploaded Image: {filename}')
            plt.axis('off')
            plt.show()
        else:
            print(f"Unsupported file type for {filename}. Content not displayed.")

except Exception as e:
    print(f"An error occurred during interactive upload: {e}")
    print("This might happen if no file was selected or if the environment is not Colab.")



### Programmatically chekc image

In [ ]:
import base64
from IPython.display import Image, display

# IMPORTANT: Replace this placeholder with your actual Base64 image string.
# This string should typically start with "data:image/png;base64," or
# "data:image/jpeg;base64," etc., as generated by the previous Canvas.
# If you only have the pure Base64 encoded data (without the "data:image/..." prefix),
# you'll need to adjust the extraction logic below.

# Example placeholder Base64 string (a tiny transparent PNG)
# In a real scenario, you would copy the full markdown_code output
# from the 'colab_image_embed' Canvas for your desired image.
base64_image_string_with_prefix = "data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQABAAD/4QAqRXhpZgAASUkqAAgAAAABADEBAgAHAAAAGgAAAAAAAABHb29nbGUAAP/bAIQAAwICCwgIDQgICggICgoICAgKCAoICAoKCA0ICAgICAoICggICAgICAgICgoICggICAgKCQkICAsOCggOCAgJCAEDBAQGBQYKBgYKDw0KDg8NEA4QDw8QDQ0NDw0PDQ0PDw0NDQ8PDQ0ODQ0NDQ8NDQ0NDQ0NDQ0NDQ0NDQ0NDQ0N/8AAEQgAcABwAwERAAIRAQMRAf/EABsAAAIDAQEBAAAAAAAAAAAAAAQFAQIDBgAI/8QANhAAAgEDAwMCBQIFAwUBAAAAAQIRAxIhAAQxBSJBE1EGFCMyYUKRJDNScYFiofA0Q8HR8RX/xAAZAQADAQEBAAAAAAAAAAAAAAABAgMABAX/xAA0EQACAgAEAwYEBQQDAAAAAAAAAQIRAxIhMUFRoRMiUmFxkRQygeEEYrHB0SNCQ/AzcvH/2gAMAwEAAhEDEQA/APqvqPVhSUMy1HlrYRQxGCZMlQFEZM+R+Y8J6I9KEHN1aXqR0j4jFZrRTrrAmXp2r+9xmfETOf6WtRO+A88LKrtMcHTkCfT0DWe9LRNZYUhrGLCgPbWNZIoDQoxIoaUBBpaNGJB0yRi1+nymskNplExF2nQGc/v6lQD6PpkyJ9QtEeYtzd7eP31oZb7115AlfACapu5gfKctLH1uOy2FDGT990uI7IB7tU/occ3QT+p5BFV9zb2/K3W0+fVgtn1eDIUG0IMk9xJGBpV2N630C8/CgraPWFMX+gal2Qt4QL+C1zFvOQB48SUl2ebS66hWatasYbSo1ovtDQLgpJUHzaSASPYkCeYHGoyq9Nh1dahKnQMXnWRirNrGAOr7wohYMiAAyzKxA4AMLmBmf8cZ0rVKykEm6YhX4k8/M0YgifTYgGYEkAYP5OTxPGlT8zoeH+V+5duuEEKa9G4tFppvJwTAAtN/ErzII7TwU2uIvZp65epXe9XZTmvSWbo+i54NoOPukgzBieLgDp23xfQ0YJrSPUvW64biRXQKWZFT0HmTFndGYJHiGJgHwGzPe+hlBV8vU6LayFAYgtGSBAJ8kCTA/EmPc866EjkdXpsc18RhLFNWtU26hxDJU9O4lWAVjBlYJNvEgH9Ig4Dlm7sb+lk8Sq1dCakdvYx+drwLCzHc5p9wC8r2h2XIYGRcPtLDXVPtbS7NX/13I9yn3upnRG2qm5N7XaEuldy8WpNxkC0gs105JlQvaFXTVjQVPDS+gq7OWqk/clN7tGa0byozMqIFXdOxg2WWgSbjaO8yxF8nL6OTHSt4ar0NeHdZuoTvBtrvSbc1VamGlRuKgbtWWuIHdaFZokwWf+wlDtqzKCp+SC1h7ZtfU90hdrTqqi7ncO6sEWm+5r1LiIUB7puIIyWOGDSfuGjidtKLbgqfGkvYEFhppJv3Z0+5qp6oBqlWuSKQqxcbXZRbyblBYr+oLJm3HDHNl0WnOjqdXuM79IkNZRtPQATf7RmEU3FMzN1itPOIbwSZkZnzzIcb20HjJLdWAp0qpBmuST7UaYj/AGz/AJx+NDs3zHeJF7R6s1OxqR/OM5z6aH+qPbiR/exeO6XyuqsGeN/KMl/fHOM//dVSJFwdEwg6xuWVAadH5gzFl1NYBBlpqYjwQJJBODwZ4MU33pUgTbS0VgFDeVCIOyCDsP8ANoFZvA8CfpqTUm3xCyTi0oQX+ToyacvD1RVeo1roGyUCfuO4o5yckBCfY+TM+0k1hv8AyP2YLnfyjnaVWNMs1FVYNimHQyJHddAA8mOYH5gSdZqUtOf2GV1bQRs0uWWpimSTKmwnDGCSsgz93JicwZGkap0naGWu6CUojmB/eBpdQ0SUEzAn3gT++gY0B0UgkFtMYqammEBeob2xC4V3gfYglm/AHn/wM6DdIaKt0KNr8TMzhPl9ykmLmVAo/JIcwB+QCfAMjSqTvYu8NJXmQ9u1UiaFtBmBaTY1NbDFrtLVmEu/6a5Y1BXakvaSAFtAX+q4kR90xaDd3B7VK3jNRVZbItO7zUZDpdRmLLuWH3cJTOWi1bWVlCU1CxHc1zkkFgdOpxS1h+ouVvaX6BFIOzEDcTCqsenSMN29xgDLWObcAXn+hda0lrDqwq/EMOj0iqkGqa3cWDEICA2QvYAIBmDHGPGkm7eir/fMaPrYbfqQ5VamiYD6olRl+iyIZ5dC4j2gMv7yf7aAklJruugFtrXk/VpRBgekZ+2ASbwJu7jiIFoGbtEnlne6B22e5tAFajdJk/LmIgQAPVOQQfP6v9Iuxsk+a9jejtK3msp94pKB7SJJYEZP3EExiOcNlnz6A7bPdeK9EZGPlzgZx/NMngTjGfEEi5cTmvYdbIMEAqMHbMsFtByYgSYgQOTxoJlYp1qBbrqVgHbUe5gvYt0TMs3AVFAkkn2ABJALQjm0tL1DJ0LE+Mef4bei2MGiuSSBC21CGIkkkG0BTn7Qa/D/AJ46+ZN4v5WK6m0oGrnY1C10Gr6FKPqOCxu9S4i4lmMYsk4CzZSmo/8AL9LfAnUb+ToHj4V27MU+XWAJzT+mxYWSP0l7ZQtFwXEwwmPbYiSeb+efsP2cHpX8BC/Bm38UaYHBAEBhyAwH3AGGAPDKrcgHQ+IxPEw9lDkMOndIp0STSRUuiYHMEkT/AJY/vqU8SU/mdjRio7IYLUnSWObDTGAupdWWkAXu7jaAqOxP+EDGPcnA1hJSUdwSt1amygNf3lYWypJ74Ewsr3DzEDJgGdAGaLRonSUBUgHsi3ufEAjILQ3J+6c55zojZUZnpCD9PDK/3NymFPPj24/GsDKjU9NQkEjKsXGTyTJPOc5gyMD2GsGkFFtYYEpPoDMWdSp1Qxsq0qYYoFupFrcEWx6qBy7EEHtItIh7hZSLjWsW/qSalzAd1Q3IsC16H6rmake83kkqoqyQtKRbfAZZOPtspYTu4v326cybU9KaCdlsaygB66uLkM+jDFVBuE+oZLmwliDADiO9Sk5Tw3tHr9hlGS3ZlttvuUi6tQKDvctTe4ZUsoc1bQsX95pi0WwsDtMp4T2i74a/tQqU1uwjcbPc33JV24XuARqFQ4PEkbhZcHF0RE9oukBSwkqlF36/YLU7tNe33G+xqkrlldhhmUQJHIAucrHFpYkeSedRlvsUTsJFTQGA+qbiqFHoLSZrhd6jsgA8xYjkt7AwP3xrJzcv7UvqRtatTHqCmOyWKsxhrsABlErbm4kGcWgZ0ArNxC7tFDlZ1jEepGjZir1dCw0BfMhRLMqjiSQBnjnE61pLUZKxDQ+HNsxi4VDAWDXJJIkliFcEu90MT+kAAKBBv8XLZP8A3+CLwEtWhoPh6ncr2ksihFN79oXgRdb/AHxnEzA0nazppPcGSO4K3wVRItKuREQa1fMwM/Uk8eZgyf1NLr8RiLj0QOyiE7r4Rp1cutR+xU/m1gIWQMI6rOTJABPmYGpx/EShon+hnhplqnwehUJFYKrMwAq1h90EglXDEBgHEsSrcEAkaPxEs2bS/RG7ONUG9L6MtBbKalVktEucnky5JzzzEyfJ0k8Rzdyeo8YqKpBd2ptjCzrOxFS0NUq0smPTqWFjHkxJAyQOJ98aFk5xT3f7AVfoS2getuVALG4bh5N0YLNJIFvaJgS3udFaiPDXN+5hR6IlpI3G5Iwpb5kkCWVhHgGYHvDRw0EgyR5v3L7npSFiTWriREDcuOF8AHkDPnwdYOSLe/UY7Xp1gWHqtYGHdULX3ZmpOXI8TgTgDRKqFVqe3m4gaKRQA6vUApy1JtxkfTCqxyD3QxAgCZ8+ACSNNGCno2l6gcnFWugBQ6gAfUXZ1lYKc+nRD8RaO/g/bN0ecjJd/h4J6TiI8aTWqY0o9fJUsaNdYiBapZpDnCq5Mi0A+JdMmWtDw0mkpJi5nxTGoOpDivq3WFpfcJm48E2hEvZmADG0ARIU9zKOWE+VjtKeo+ZRWpwK7b1HmnuKqrUU0meKhRaldDUpMrVo9N7KwWkqrarWAgMVDec0n/c/u/U5qTdpv/0fdPq9wq16ly3ValMn1FTsKkurCaaUgtNvSpklmpPVJeoKjDT7aykyi5yfodBsPjGm1RaVNkLPcQFYRC00qTwsyjoVtmVJbhGhu0TdJl1OLdIt8QbxAB6lJ68loC0hUtiCSQcDwB5njzr3I7I58SuKsU7jqVO0A7SsQCSF+VRghB5wSoJLMQQZ+84nufUnmWzi/Y9teoUypT5WqqkiVO1AUk5JKiZi0SY5C8mNZGUltlfsX6fu6buLdtVQ/cKjbYIBAJBnDA4iImSBA8bUMXG/l6Dfbb64EgOsGO5Ss4BkTyM8jEz7HW1OhOxR1XeeNNqgh9am7IPScU27e4oHwORBIyffMex1WLSfeV9BWm1oC/8A51eP+oEyDPoU8QWkR/qBAnxbPmNUc8PwdRcsvF0CqW3qqI9VWN0y1IYWUwLGpiYDiSDllMQhV5XBv5eouWS4jJquplBV1PpNOqZqUqdU2lQWVSQGi5QWBgNAkAwYEzA1xYmBnlmNUXurAK/Q0IIFGJ/UhpqwIUorK1wKuikhGGVHEam/wr8hu7yNG6SjFS1AEohpoTYbFIAZFN5IRgACowYEgwNL8I7vQNx5B+22qqSwpqpLXlu2SxWwsTk3W9szMY1vhXfA1q7SPdW3NQL9FVYzwzQPb28TcePt5zB9GKpEZuVaGC1q0GVoTmIqVCPtaJ+kJ77eAIUt5UXFATl5EepWj7aE5gX1AOVtk+mxwLiTb3EKIWSQQd7yPdO3FW4+qtNQIgq5a7AkZVSIP6jE5EYDMAxzX3g/cVsaJY5Le1pOgxRm3WSqCxbySoi4CAeWJPhfYAk+3nVHpsMhefi+qBPy1Q5iPVoyBBMmXA5gRPk+ACZ3LwlKjzLP8WVYB+XcyJP1KeMnBzzEH2Ex4OtryFqPM33HxM68UnfuC9pp58T3OuLsZjHdxOi75CacyKfxKx/7NVZBJk0u32kiqQbsgWzEZCysi3yNpzM6vxPUxbRdp570FniG7jwfKyIgzrW+QVXMb7XqE+/7H/aQJ0wgYtTQCKPiTdBbQdwdrJYyBSN9sEg+qrAR+IJnzGMRxH50BqrMoC7twaiwjenQuYjvZlDJ3MFxAEBQZBOdGhabXzPoGL0yrA/iHJDXXFKeRAFpCBQRyfeT+ANBobJLxF6uzclfrN2mT2KL/YNEdo9hE4mdGh8rdamO729S0j1ckg3emuABlQJiCYMmSMjM4DQafM5yhtXU91Q1BJOUUc8AQcAeAB/7K0CKa3dmu8vNMCnUWiZHeUDwPYBiBJMDJ4nzGuuFXqrFldaOgbbiouH3KMSViaVJeJuEBs3GB+ADGWlaPK9o0DXmD/LV8/xVMHvidun4tkeoMr58GT+IbNh+B+4lS8RruaNVrQm5VSFAb6NNi5BksJcWyMQAQM/4CcFblDrVBak9pEU9nXEzuASRC/w6C3GD9xkzHmCsiAYcByw/D1NllzL7Ta1gwL7hWW6So26rI/pBvYgfnJ/OllPDa0j1ClLi+g56ezAm9lbulYW2BAwe5rjM93biBGJMHXBDq+J0NF8aUcx3+/KDCVKmCYQAnBAjJUSZxngMcRoUBuuAto/EBJA+X3KDOSlMAYkkxUJH7ZOBOiTWI/CzbZ9fuEmnWUi6R6bH7ZwCB3ExiOSQPOsMp3wZA+IBaC1OupP6fSdiO4qASikAnmJ4OhYFPS2mB9c+ILRAp13+3K0mI7oOD5tBzE5BHONaxpTrg/YV0d1cJAYc8qVODHDAHPIxkQdAZOzLq9vpD1KbVxI7FS8zByRIEDOSYGPxrqw7zd1157E57aqxLskooCq7SskQf5PNvEWu0mQAPERPaCR1SeI98RP6kVlX9vQvT6PQqP6Z2jgAEB3pKEABJEG8nkkrAwT48B4mJGOZYn0vU2WDdZRl0ro1Fe6nSFMhiJKFTNtpInJBU23CZEicajiYk2qlK/rY8Yx4IO2+0WmLUAUSTA/Jk/udRk3J2yiVbF7tKE3oPnQZh9QbQGBurIDBNVqEXcNSAaQJn1FcG0CQRESdYWaXOilGBaPWJtJBF1MmoYIAbBYFT/QVyDM8awFWmoxU/wDPH/P8aBQ8z6IRL1fcaQAnnQsxtuN6tJLqjBVwJP54/MnVG6CouTpFtru1qC5DcM5HBjBj31lKxXFx0Z6nvVYkKVYiJAIJE8T7T4940bs1VuWaroimbVdYxS/WMb0DnQZjoKDaAwv69abb6B3GTEU0ezKyTeQFnnHNp/GgJOtO7f7Fdl0ykWDfLqjQKlxppgk4hhP1O0MY47czjWAox8I12u2VFtQBQJhQAAJMnAxkmdEsklsWrNoBZzHV6udJJigF2pmP/9k="

def display_base64_image(base64_str):
    """
    Decodes a Base64 string (with or without a data URI prefix)
    and displays it as an image in an IPython/Colab environment.
    """
    try:
        # Check if the string contains the data URI prefix (e.g., "data:image/png;base64,")
        if "base64," in base64_str:
            # Split the string to get only the Base64 data part
            _, base64_data = base64_str.split("base64,", 1)
        else:
            # Assume the entire string is pure Base64 data
            base64_data = base64_str

        # Decode the Base64 string into bytes
        image_bytes = base64.b64decode(base64_data)

        # Display the image using IPython.display.Image
        display(Image(data=image_bytes))
        print("\nImage displayed successfully from Base64 string.")

    except Exception as e:
        print(f"Error displaying image from Base64 string: {e}")
        print("Please ensure the Base64 string is valid and correctly formatted.")

# Call the function with your Base64 string
display_base64_image(base64_image_string_with_prefix)

# You can also use this if you have the full Markdown code string:
# markdown_code_from_previous_output = "![Alt Text](data:image/png;base64,YOUR_VERY_LONG_BASE64_STRING_HERE)"
# display_base64_image(markdown_code_from_previous_output)
